In [ ]:
import os
import sys
import math
import h5py
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm  #to get python's normal library of colormaps
import matplotlib.colors as mplc
import matplotlib.ticker as ticker
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec

import cartopy.crs as ccrs
import cartopy.feature as cfeature
#from cartopy.util import add_cyclic_point
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from datetime import datetime
from datetime import timedelta

import metpy.calc as mpcalc
import metpy.plots as mplots
from metpy.units import units

import scipy.stats
from scipy.stats import norm
import scipy.signal as sig

from PIL import Image
import icartt            #needed to read .ict files

import time

# import warnings
# warnings.filterwarnings("ignore")  #hides "MatplotlibDeprecationWarning" with pcolormesh

# tstart = time.time()


### For CPEX-CV AGU Paper (Figure 1)

In [ ]:
#adapted from CPEXCV_APR.ipynb (see this script for version separated into separate cells)

#the only cell with variables you need to change, unless you are not Ben (will need to change cell #3 too)
file_date = '20220906'           #which date to create the APR plot for
start_time0 = '20220906,161555'  #start date,time for the APR plot
end_time0 = '20220906,163100'    #end date,time for the APR plot
full = False                     #do the chosen time ranges below encapsulate the entire convective module?

#other users will also have to change day_folder, apr_folder, dawn_csv_path, and drop_csv_path filepaths
    #dawn_csv_path and drop_csv_path are based on created files from CPEXCV_DAWN.ipynb and CPEXCV_dropsonde(_metrics).ipynb

start_time = datetime.strptime(start_time0, '%Y%m%d,%H%M%S')
end_time = datetime.strptime(end_time0, '%Y%m%d,%H%M%S')

#locations of the APR folder and APR files
day_folder = os.path.join(os.getcwd(), file_date)
apr_folder = os.path.join(day_folder, 'APR_files')

#load the final DAWN and final Dropsonde CSVs
dawn_csv_path = os.path.join(day_folder, 'final_dawn_' + file_date + '.csv')
dawn_csv = pd.read_csv(dawn_csv_path)

drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
drop_csv = pd.read_csv(drop_csv_path)

# #example CPEX-CV APR file
# test = xr.open_dataset('/Users/ben/Desktop/CPEX/Coding/20220906/APR_files/cpexcv-APR3_DC8_20220906_R0_S20220906a121242_E20220906a121851_KUsKAsWns.nc')
# test

#create a list of all the given day's desired range's APR files:
for x in os.listdir(apr_folder):
    if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
        os.remove(os.path.join(apr_folder, x))

#find the starting APR file in apr_folder
first_file_index = None 

#sorted() makes sure the code goes through the files in alphabetical (chronological) order
#GOING THROUGH THE FILES IN CHRONOLOGICAL ORDER IS ESSENTIAL FOR THIS CELL TO WORK PROPERLY!!
for i, x in enumerate(sorted(os.listdir(apr_folder))):  
    file_start_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
    file_end_time = datetime.strptime(x[46:54] + x[55:61], '%Y%m%d%H%M%S')

    if start_time <= file_start_time:  #if start_time is before the APR file start time and not within any previous APR file's time ranges
        first_file_index = i
        break
    elif (start_time >= file_start_time) and (start_time < file_end_time):
        first_file_index = i
        break
    else:
        continue
if first_file_index == None:
    sys.exit('Requested start_time is beyond all available APR files')
    
#find the ending APR file in apr_folder
last_file_index = None    

#sorted() makes sure the code goes through the files in alphabetical (chronological) order
#GOING THROUGH THE FILES IN CHRONOLOGICAL ORDER IS ESSENTIAL FOR THIS CELL TO WORK PROPERLY!!
for i, x in enumerate(sorted(os.listdir(apr_folder))):  
    file_start_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
    file_end_time = datetime.strptime(x[46:54] + x[55:61], '%Y%m%d%H%M%S')

    if end_time <= file_start_time:  #if end_time is before the APR file start time and not within any previous APR file's time ranges
        last_file_index = i - 1
        break
    elif (end_time > file_start_time) and (end_time <= file_end_time):
        last_file_index = i
        break
    else:
        continue
if last_file_index == None:  #the end_time is after all available APR file ranges
    last_file_index = len(sorted(os.listdir(apr_folder))) - 1  #the last available APR file's index
if last_file_index == -1:
    sys.exit('Requested end_time is before all available APR files')

apr_file_list = sorted(os.listdir(apr_folder))[first_file_index:last_file_index + 1]
#apr_file_list

#Calculate time range in minutes, have a tick for each minute
num_minutes = (end_time - start_time).total_seconds() // 60
fig_length = num_minutes * 2

#Calculate height range of the plot, based on the maximum height of all DAWN profiles
height_max = dawn_csv['Height [m]'].max()
y_max = math.ceil(height_max / 1000) * 1000  #round height_max up to the nearest 1000

#matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
matplotlib.rcParams['axes.labelsize'] = 45
matplotlib.rcParams['axes.titlesize'] = 55
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['xtick.labelsize'] = 37
matplotlib.rcParams['ytick.labelsize'] = 37
matplotlib.rcParams['legend.fontsize'] = 35
#matplotlib.rcParams['legend.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'
#matplotlib.rcParams['hatch.linewidth'] = 0.3

#plotting a time-series plot of just Ku-band reflectivity vs. height
ylim=[0,y_max]
vlim=[-10,40]
vel_lim=[-10,10]

#Low resolution ('lores') radar variables in APR hdf files
ku_band = 'lores_zhh14' #Ku-band reflectivity
ka_band = 'lores_zhh35' #Ka-band reflectivity
vel = 'lores_vel14c' #Mean Doppler Velocity from Ku-band (surface Doppler velocity is subtracted and free of aliasing)
w_band = 'lores_z95s' #W-band reflectivity
w_band_nadir = 'hi2lo_z95n' #W-band reflectivity at nadir, upscaled to same resolution as 'lores'
    #^^^for quantitative analysis, will likely want to switch to using 'hires_z95n'

if full:
    plot_save_name = file_date + '_SinglePanel_full.png'
else:
    plot_save_name = file_date + '_SinglePanel_' + start_time0[-6:] + '-' + end_time0[-6:] + '.png'

#fig,axes = plt.subplots(1,1,figsize=(fig_length,14))
fig,axes = plt.subplots(1,1,figsize=(48,14))   #use for AGU Paper Figure 12b (to match figsize for Figure 12a from CPEX_2017_APR.ipynb)

for file in apr_file_list:
    
    #find the APR file of interest (for the desired datetime)
    apr_filepath = os.path.join(apr_folder, file)
    apr_file = xr.open_dataset(apr_filepath)

    #grab the radar variables of interest
    ku_good = False
    if ku_band in apr_file.keys():
        try:   #some APR files, at least in the preliminary data, have corrupted Ku-band data; 
               #if so, skip plotting the Ku-band for that file
               #corrupted: "OSError: Can't read data (inflate() failed)"
            ku_data = apr_file[ku_band][:]

            #mask the missing Ku-band data (values of -99.99)
            ku_masked = np.ma.masked_where(ku_data <= -99, ku_data)
            ku_masked = np.ma.masked_where(np.isnan(ku_masked), ku_masked)  #masks NaN values (not masked in previous line)
            ku_good = True
        except:
            ku_good = False
    
    #if Ku-band is available
    if ku_good:            

        #Convert 'lo-res' APR times to datetimes
        time = apr_file['time'][:]  #For 'lores': Time of scan, in seconds since midnight UTC of [YYYY-mm-DD]
        alt3d = apr_file['lores_alt3D'][:]

        time_dates = np.empty(time.shape, dtype=object)
        for i in np.arange(0, time.shape[0]):
            #hour, second automatically revert to midnight (hour = 0, seconds = 0) for '%Y%m%d'
            time_dates[i] = datetime.strptime(file_date, '%Y%m%d') + timedelta(seconds = float(time[i].values))

        #Create a time at each gate (assuming time is the same for each ray of a given scan and down each ray)      
        time_gate = np.empty(alt3d.shape, dtype=object)
        for i in np.arange(0, alt3d.shape[0]):
            time_gate[i,:,:] = time_dates[i]   #assign the same time to all of a given scan's rays and height bins     

        time3d = np.copy(time_gate)

#         print (time3d[0,0,:])  #times should be the same
#         print ('')
#         print (time3d[0,:,0])  #times should be the same
#         print ('')
#         print (time3d[:,0,0])  #times should be different
#         sys.exit()

        #plot the APR data factoring in the aircraft roll (ray adjustment)
        #choose the "pseudo-nadir" ray factoring in aircraft roll
        roll = apr_file['lores_roll'][:]
        ray_angles = np.arange(-25,25.01,25/12)  #in degrees; 25 rays for each scan, with the middle (13th) scan being zero degrees
        for scan in range(roll.shape[0]):
            if (time_dates[scan] >= start_time) and (time_dates[scan] <= end_time):
                ac_roll = np.nanmean(roll[scan,:])  #roll varies slightly w/ray, so take the average roll value for a given scan and use that for ray adjustment
                ray_use = np.argmin(np.abs(ray_angles - ac_roll))  #the index of the ray whose angle is closest to that of ac_roll    

                #scan + 2 (and not scan + 1) is needed because pcolormesh colors the grid cell from the  
                #grid cell's time to the subsequent grid cell's time.  If a subsequent grid cell does not exist,  
                #then pcolormesh cannot/doesn't color the grid cell (remember, slicing is right side EXCLUSIVE, 
                #so scan:scan + 1 is only 1 element and thus doesn't have a subsequent cell!)
                #by this same logic, scan:scan + 2 will only color one grid cell, since the 2nd (and last) 
                #element/grid cell doesn't have a subsequent grid cell

                if ku_good:
                    pm0 = axes.pcolormesh(time3d[scan:scan+2,ray_use,:], alt3d[scan:scan+2,ray_use,:],
                                          ku_masked[scan:scan+2,ray_use,:], cmap=cm.Spectral_r, vmin=vlim[0], vmax=vlim[1])

                else:
                    pass

    apr_file.close()

cbar0 = plt.colorbar(pm0, ax=axes, pad = 0.01, extend = 'both')
cbar0.set_label('Z [dBZ]')
axes.set_title('b) Ku-band Reflectivity (Case 10, 06 September 2022)')

# dawn_skip = slice(None,None,7)   #only plot every 7th value
# drop_skip = slice(None,None,20)  #only plot every 20th value
dawn_skip = slice(None,None,10)   #only plot every 10th value
drop_skip = slice(None,None,50)  #only plot every 50th value

#set the plot start time as the beginning time of the first APR file and the
    #plot end time as the end time of the last APR file:
# range_start = datetime.strptime(apr_file_list[0][32:40] + apr_file_list[0][41:47], '%Y%m%d%H%M%S') 
# range_end = datetime.strptime(apr_file_list[-1][49:57] + apr_file_list[-1][58:64], '%Y%m%d%H%M%S')
# range_start = start_time
# range_end = end_time
range_start = datetime.strptime(start_time0, '%Y%m%d,%H%M%S') 
range_end = datetime.strptime(end_time0, '%Y%m%d,%H%M%S')

for xx, dawn_profile in enumerate(dawn_csv['Time [UTC]'].unique()):
    if xx % 2 == 0:  #skip plotting every other DAWN profile
        dawn_csv_prof = dawn_csv[dawn_csv['Time [UTC]'] == dawn_profile]
        axes.barbs(pd.to_datetime(dawn_csv_prof['Time [UTC]'])[dawn_skip], dawn_csv_prof['Height [m]'][dawn_skip], 
                   dawn_csv_prof['U Comp of Wind [m/s]'][dawn_skip], dawn_csv_prof['V Comp of Wind [m/s]'][dawn_skip], 
                   fill_empty = True, length = 10, pivot='middle', sizes=dict(emptybarb=0.1), barbcolor = 'k')
    else:
        pass

# axes.barbs(pd.to_datetime(dawn_csv['Time [UTC]'])[dawn_skip], dawn_csv['Height [m]'][dawn_skip], 
#            dawn_csv['U Comp of Wind [m/s]'][dawn_skip], dawn_csv['V Comp of Wind [m/s]'][dawn_skip], 
#            fill_empty = True, length = 10, pivot='middle', sizes=dict(emptybarb=0.1), barbcolor = 'k')
axes.barbs(pd.to_datetime(drop_csv['Time [UTC]'])[drop_skip], drop_csv['Height [m]'][drop_skip], 
           drop_csv['U Comp of Wind [m/s]'][drop_skip], drop_csv['V Comp of Wind [m/s]'][drop_skip], 
           fill_empty = True, length = 10, pivot='middle', sizes=dict(emptybarb=0.1), barbcolor = 'b')

##########################

axes.set_ylabel('Altitude [m]')
axes.set_xlabel('Time [UTC]')
axes.set_ylim([ylim[0],ylim[1]])
axes.tick_params(axis='x', rotation = 50)
axes.tick_params(length = 15, width = 5)
axes.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%H:%M:%S"))
axes.xaxis.set_major_locator(ticker.MaxNLocator(num_minutes))      #sets number of ticks
axes.set_xlim([np.datetime64(range_start),np.datetime64(range_end)])    
    #use the above line to narrow the plot's time range (even within a file!!)
        #range_start and range_end must be a datetime object or a string with the 
            #format: 'YYYY-MM-DD HH:MM:SS' or 'YYYY-MM-DDTHH:MM:SS'

plt.tight_layout()

#save the figure
#plt.savefig(os.path.join(day_folder, plot_save_name), bbox_inches = 'tight')
plt.savefig('/Users/ben/Desktop/Figure1b.png', bbox_inches = 'tight')
#plt.show()  #if you want to also show the image in the output cell, plt.show() must come after plt.savefig() in order for the image to save properly
plt.close()


##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
from PIL import Image
im = Image.open('/Users/ben/Desktop/Figure1b.png')

try:
    im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
except:
    #use this for older version of PIL/Pillow if the above line doesn't work, 
    #though this line will have isolated, extremely minor image effects due to 
    #only using 256 colors instead of the 3-element RGB scale
    im2 = im.convert('P')

im2.save('/Users/ben/Desktop/Figure1b.png')
im.close()
im2.close()



### For CPEX-CV AGU Paper (Figure 2)

In [ ]:
#see ERA5_Analysis.ipynb for version separated into separate cells

#adapted from CPEXCV_BAMS_Figures.ipynb (Figure 7)
#plotting ERA5 reanalysis 850 hPa streamlines (and maybe RH), MIMIC TPW (if not ERA5 RH) and GPM IMERG

# case_dict_agu = {0: ['20220909', '2022-09-09', 16, [-60, -10, 0, 30], 'Isolated'],
#                 1: ['20220907', '2022-09-07', 15, [-60, -10, 0, 30], 'Organized']}  #flight track lat/lon extent [West,East,South,North]
case_dict_agu = {0: ['20220909', '2022-09-09', 16, [-27.5, -12.5, 10, 20], 'Isolated'],
                1: ['20220907', '2022-09-07', 15, [-50, -20, 5, 25], 'Organized']}  #flight track lat/lon extent [West,East,South,North]

stream_pres = 700  #hPa; pressure level to plot ERA5 streamlines for

#set some baseline plot displays

#matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
matplotlib.rcParams['axes.labelsize'] = 40
matplotlib.rcParams['axes.titlesize'] = 41
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['xtick.labelsize'] = 40
matplotlib.rcParams['ytick.labelsize'] = 40
matplotlib.rcParams['legend.fontsize'] = 40
matplotlib.rcParams['legend.facecolor'] = 'w'
matplotlib.rcParams['axes.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'
matplotlib.rcParams['hatch.linewidth'] = 0.3

data_proj = ccrs.PlateCarree()
group_fig = plt.figure(figsize = (24,34))

for key, case_info in case_dict_agu.items():
    
    file_date = case_info[0]
    file_date_prior = str(int(case_info[0]) - 1)
    case_num = case_info[1]   
    hr = case_info[2]
    campaign_extent = case_info[3]
    convective_type = case_info[4]
    
    ###get locations of the dropsonde/Navigation folder and read the appropriate files in
    day_folder = os.path.join(os.getcwd(), file_date)

#     #dropsonde data
#     drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
#     drop_csv = pd.read_csv(drop_csv_path)

    if file_date[:4] == '2017':
        campaign = 'CPEX'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
    elif file_date[:4] == '2021':
        campaign = 'CPEXAW'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
    elif file_date[:4] == '2022':
        campaign = 'CPEXCV'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
    else:
        pass

    #ERA5 data
    era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
    era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
    #era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure_BAMS.nc')
    ds = xr.open_dataset(era5_path)

    #Navigation data
    nav_folder = os.path.join(day_folder, 'Nav_files')
    
    for x in os.listdir(nav_folder):
        if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
            os.remove(os.path.join(nav_folder, x))

    nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

    if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

        nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
        flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
        flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

    else:  #for CPEX navigation files, open the CSV file with pandas
           #(Navigation files for CPEX (2017) are originally .kmz not .ict,
           #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

        nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
        flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
        flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    


    # ###calculate each dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

    # drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

    # for sonde_datetime in drop_csv['Time [UTC]'].unique():
    #     drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
    #     drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
    #     drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

    #     sonde_info = [drop_mean_lon, drop_mean_lat, sonde_datetime[-8:-3]]
    #     drop_coords_and_time.append(sonde_info)

    # ###calculate each NEAR-STORM dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted
    # df_drop = pd.read_csv(drop_metric_filepath)
    # df_drop_use = df_drop[df_drop['Date'] == int(file_date)].copy()

    # for x in range(len(df_drop_use)):
    #     date = str(df_drop_use['Date'].iloc[x])
    #     time = str(df_drop_use['Time'].iloc[x]).zfill(6)
    #     sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

    #     drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
    #     drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
    #     drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

    #     sonde_info = [drop_mean_lon, drop_mean_lat, time[:4]]
    #     drop_coords_and_time.append(sonde_info)
        
    hr2 = str(hr).zfill(2)

    #MIMIC TPW data
        ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
    tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
    tpw_path = os.path.join(tpw_folder, 'comp' + file_date + '.' + hr2 + '0000.nc')
    ds_tpw = xr.open_dataset(tpw_path)

    #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
        ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
        ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
        #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
    imerg_folder = os.path.join(day_folder, 'IMERG_files')

    for x in os.listdir(imerg_folder):
        if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
            os.remove(os.path.join(imerg_folder, x))

        #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
        elif (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
            imerg_file = x
            break
        else:
            imerg_file = 'Could not find the desired IMERG file'

    #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
    assert (file_date in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'

    imerg_path = os.path.join(imerg_folder, imerg_file)
    ds_imerg = h5py.File(imerg_path, 'r')

    imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
    imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
    imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 

    imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
    imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)

    #mask blank data
    imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
    imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)

    #creat the plot
    ax = group_fig.add_subplot(2, 1, key+1, projection = data_proj)
    
    uwnd = ds.u.sel(time = file_date).sel(level = stream_pres)  #zonal winds (m/s)
    uwnd = uwnd.sel(time = uwnd.time.dt.hour.isin(hr))       #zonal winds (m/s)

    vwnd = ds.v.sel(time = file_date).sel(level = stream_pres)  #meridional winds (m/s)
    vwnd = vwnd.sel(time = vwnd.time.dt.hour.isin(hr))       #meridional winds (m/s)
    
    rh = ds.r.sel(time = file_date).sel(level = stream_pres)  #meridional winds (m/s)
    rh = rh.sel(time = rh.time.dt.hour.isin(hr))       #meridional winds (m/s)

    ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
        ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
            ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
    #uwnd_smoothed = mpcalc.smooth_n_point(uwnd,9,10)
    #vwnd_smoothed = mpcalc.smooth_n_point(vwnd,9,10)

    #ax.set_title('%s %s UTC MIMIC TPW, GPM IMERG, and ERA5 %i hPa Streamlines' % (case_num, hr2, stream_pres))
    if convective_type == 'Isolated':
        ax.set_title('a) GPM IMERG and MIMIC TPW for an %s Convective Case' % (convective_type))
    else:
        ax.set_title('b) GPM IMERG and MIMIC TPW for an %s Convective Case' % (convective_type))
    #ax.set_title('%s %s UTC ERA5 %i hPa Relative Humidity, \nGPM IMERG, and ERA5 %i hPa Streamlines' % (case_num, hr2, stream_pres, stream_pres))
    ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

    # Add land, coastlines, and borders
    #ax.add_feature(cfeature.LAND, facecolor='0.8')
    ax.coastlines(ls = '-', linewidth = 5, color = 'k')

    #plot MIMIC TPW
    tpw_levels = np.arange(0, 70.5, 1)
    pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
                      extend = 'max', cmap = cm.jet, transform = data_proj, zorder = 0)
#             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
#                                 cmap = cm.jet, transform = data_proj, zorder = 0)

#     #plot ERA5 RH
#     tpw_levels = np.arange(0, 100.1, 2)  #actually RH levels, but keeping the tpw_levels name because we use it elsewhere
#     pm0 = ax.contourf(ds.longitude, ds.latitude, rh[0].values, levels = tpw_levels,
#                       extend = 'max', cmap = cm.jet, transform = data_proj, alpha = 0.6, zorder = 1)
# #             pm0 = ax.pcolormesh(ds.longitude, ds.latitude, rh[0].values, vmin = 0, vmax = 70,
# #                                 cmap = cm.jet, transform = data_proj, zorder = 1)

    #plot IMERG Rain Rate
    pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
                      levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
                      norm = 'log', extend = 'max', 
                      cmap = cm.jet, transform = data_proj, zorder = 1)
#             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
#                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
#                                 cmap = cm.jet, transform = data_proj, zorder = 1)

    #gridlines
    gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, linewidth = 2, 
                      color = 'gray', alpha = 0.5, linestyle = '--', zorder = 3)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size':40, 'color':'black'}
    gl.ylabel_style = {'size':40, 'color':'black'}

    # #plot ERA5 streamlines
    # ax.streamplot(ds.longitude, ds.latitude, uwnd[0].values, vwnd[0].values,
    #               color = 'white', linewidth = 4, density = 1.0, 
    #               arrowsize = 6, arrowstyle = '->', 
    #               transform = data_proj, zorder = 3)  

    #plot flight track
    ax.plot(flight_lon, flight_lat, color = 'darkmagenta', linewidth = 5, zorder = 4)

#     #plot dropsonde locations for the given flight
#     for sonde in drop_coords_and_time:
#         #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 300)
#         if sonde[2] in ['15:10', '15:43', '16:16', '16:42']:
#             ax.scatter(sonde[0], sonde[1], marker = '*', color = 'white', edgecolor = 'k', zorder = 5, s = 6000)
#         else:
#             ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', zorder = 5, s = 6000)

#     #same as above, but labeling the dropsondes by the order that they appear in the
#         #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
#     #for z, sonde in enumerate(drop_coords_and_time):
#         #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5)

    #plotting the colorbars
    #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
    #cbar0.set_label('TPW [mm]')
    #cbar0.ax.yaxis.set_ticks_position('left')
    #cbar0.ax.yaxis.set_label_position('left')

    #this works with GeoAxes (i.e., Cartopy's map projections)
    if key == len(case_dict_agu) - 1:

        #MIMIC TPW colorbar
        ticks_tpw = np.arange(0, 70.5, 10, dtype = int)
        #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
        cax0 = group_fig.add_axes([ax.get_position().x1 + 0.10, ax.get_position().y0, 0.04, 0.77])
        cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
        cbar0.set_label('TPW [mm]')
        cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
        cbar0.ax.yaxis.set_ticks_position('left')
        cbar0.ax.yaxis.set_label_position('left')

        # #ERA5 RH colorbar
        # ticks_rh = np.arange(0, 100.5, 10, dtype = int)
        # #cax = group_fig.add_axes([ax.get_position().x1+0.05, ax.get_position().y0, 0.02, ax.get_position().height])
        # cax0 = group_fig.add_axes([ax.get_position().x1 + 0.10, ax.get_position().y0, 0.04, 0.77])
        # cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_rh)
        # cbar0.set_label('Relative Humidity [%]')
        # cbar0.ax.set_yticklabels(list(map(str, list(ticks_rh))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
        # cbar0.ax.yaxis.set_ticks_position('left')
        # cbar0.ax.yaxis.set_label_position('left')

        #IMERG colorbar
        ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40], dtype = float)
        cax1 = group_fig.add_axes([ax.get_position().x1 + 0.10, ax.get_position().y0, 0.04, 0.77])
        cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
        cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
        cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
        cbar1.ax.yaxis.set_ticks_position('right')
        cbar1.ax.yaxis.set_label_position('right')
                
    ds_tpw.close()
    ds_imerg.close()
    ds.close()

#plt.tight_layout()
#plt.subplots_adjust(hspace = 0.3)

#save the figure

#need to manually write out the whole filepath; for some reason, you can't use os.path.join() unless you're 
    #saving in the script's directory or you're saving within a subdirectory of the script's directory
plot_save_name = '/Users/ben/Desktop/Figure2.png'

plt.savefig(plot_save_name, bbox_inches = 'tight')
#plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
#plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
plt.close()

##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
im = Image.open(plot_save_name)

try:
    im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
except:
    #use this for older version of PIL/Pillow if the above line doesn't work, 
    #though this line will have isolated, extremely minor image effects due to 
    #only using 256 colors instead of the 3-element RGB scale
    im2 = im.convert('P')

im2.save(plot_save_name)
im.close()
im2.close()


### For CPEX-CV AGU Paper (Figure 3)

In [ ]:
#adapted from CPEXCV_APR.ipynb and Figure 1 above
    #see run_ecco_v_apr3_CPEXCV_AGU_paper.m, run_ecco_v_apr3_save1D.m, and eccoCases-cpexcv-apr3-full-flights_AGU.txt
    #for the ECCO-V portion of this figure (Figure 3b-d)

#the only cell with variables you need to change, unless you are not Ben (will need to change cell #3 too)
file_date = '20220907'           #which date to create the APR plot for
start_time0 = '20220907,165055'  #start date,time for the APR plot
end_time0 = '20220907,171258'    #end date,time for the APR plot
full = False                     #do the chosen time ranges below encapsulate the entire convective module?

#other users will also have to change day_folder, apr_folder, dawn_csv_path, and drop_csv_path filepaths
    #dawn_csv_path and drop_csv_path are based on created files from CPEXCV_DAWN.ipynb and CPEXCV_dropsonde(_metrics).ipynb

start_time = datetime.strptime(start_time0, '%Y%m%d,%H%M%S')
end_time = datetime.strptime(end_time0, '%Y%m%d,%H%M%S')

#locations of the APR folder and APR files
day_folder = os.path.join(os.getcwd(), file_date)
apr_folder = os.path.join(day_folder, 'APR_files')

#load the final DAWN and final Dropsonde CSVs
dawn_csv_path = os.path.join(day_folder, 'final_dawn_' + file_date + '.csv')
dawn_csv = pd.read_csv(dawn_csv_path)

drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
drop_csv = pd.read_csv(drop_csv_path)

# #example CPEX-CV APR file
# test = xr.open_dataset('/Users/ben/Desktop/CPEX/Coding/20220906/APR_files/cpexcv-APR3_DC8_20220906_R0_S20220906a121242_E20220906a121851_KUsKAsWns.nc')
# test

#create a list of all the given day's desired range's APR files:
for x in os.listdir(apr_folder):
    if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
        os.remove(os.path.join(apr_folder, x))

#find the starting APR file in apr_folder
first_file_index = None 

#sorted() makes sure the code goes through the files in alphabetical (chronological) order
#GOING THROUGH THE FILES IN CHRONOLOGICAL ORDER IS ESSENTIAL FOR THIS CELL TO WORK PROPERLY!!
for i, x in enumerate(sorted(os.listdir(apr_folder))):  
    file_start_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
    file_end_time = datetime.strptime(x[46:54] + x[55:61], '%Y%m%d%H%M%S')

    if start_time <= file_start_time:  #if start_time is before the APR file start time and not within any previous APR file's time ranges
        first_file_index = i
        break
    elif (start_time >= file_start_time) and (start_time < file_end_time):
        first_file_index = i
        break
    else:
        continue
if first_file_index == None:
    sys.exit('Requested start_time is beyond all available APR files')
    
#find the ending APR file in apr_folder
last_file_index = None    

#sorted() makes sure the code goes through the files in alphabetical (chronological) order
#GOING THROUGH THE FILES IN CHRONOLOGICAL ORDER IS ESSENTIAL FOR THIS CELL TO WORK PROPERLY!!
for i, x in enumerate(sorted(os.listdir(apr_folder))):  
    file_start_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
    file_end_time = datetime.strptime(x[46:54] + x[55:61], '%Y%m%d%H%M%S')

    if end_time <= file_start_time:  #if end_time is before the APR file start time and not within any previous APR file's time ranges
        last_file_index = i - 1
        break
    elif (end_time > file_start_time) and (end_time <= file_end_time):
        last_file_index = i
        break
    else:
        continue
if last_file_index == None:  #the end_time is after all available APR file ranges
    last_file_index = len(sorted(os.listdir(apr_folder))) - 1  #the last available APR file's index
if last_file_index == -1:
    sys.exit('Requested end_time is before all available APR files')

apr_file_list = sorted(os.listdir(apr_folder))[first_file_index:last_file_index + 1]
#apr_file_list

#Calculate time range in minutes, have a tick for each minute
num_minutes = (end_time - start_time).total_seconds() // 60
fig_length = num_minutes * 2

#Calculate height range of the plot, based on the maximum height of all DAWN profiles
height_max = dawn_csv['Height [m]'].max()
y_max = math.ceil(height_max / 1000)  #round height_max up to the nearest 1000

#matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
matplotlib.rcParams['axes.labelsize'] = 45
matplotlib.rcParams['axes.titlesize'] = 55
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['xtick.labelsize'] = 37
matplotlib.rcParams['ytick.labelsize'] = 37
matplotlib.rcParams['legend.fontsize'] = 35
#matplotlib.rcParams['legend.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'
#matplotlib.rcParams['hatch.linewidth'] = 0.3

#plotting a time-series plot of just Ku-band reflectivity vs. height
ylim=[0,y_max]
vlim=[-10,40]
vel_lim=[-10,10]

#Low resolution ('lores') radar variables in APR hdf files
ku_band = 'lores_zhh14' #Ku-band reflectivity
ka_band = 'lores_zhh35' #Ka-band reflectivity
vel = 'lores_vel14c' #Mean Doppler Velocity from Ku-band (surface Doppler velocity is subtracted and free of aliasing)
w_band = 'lores_z95s' #W-band reflectivity
w_band_nadir = 'hi2lo_z95n' #W-band reflectivity at nadir, upscaled to same resolution as 'lores'
    #^^^for quantitative analysis, will likely want to switch to using 'hires_z95n'

if full:
    plot_save_name = file_date + '_SinglePanel_full.png'
else:
    plot_save_name = file_date + '_SinglePanel_' + start_time0[-6:] + '-' + end_time0[-6:] + '.png'

#fig,axes = plt.subplots(1,1,figsize=(fig_length,14))
fig,axes = plt.subplots(1,1,figsize=(48,14))   #use for AGU Paper Figure 12b (to match figsize for Figure 12a from CPEX_2017_APR.ipynb)

for file in apr_file_list:
    
    #find the APR file of interest (for the desired datetime)
    apr_filepath = os.path.join(apr_folder, file)
    apr_file = xr.open_dataset(apr_filepath)

    #grab the radar variables of interest
    ku_good = False
    if ku_band in apr_file.keys():
        try:   #some APR files, at least in the preliminary data, have corrupted Ku-band data; 
               #if so, skip plotting the Ku-band for that file
               #corrupted: "OSError: Can't read data (inflate() failed)"
            ku_data = apr_file[ku_band][:]

            #mask the missing Ku-band data (values of -99.99)
            ku_masked = np.ma.masked_where(ku_data <= -99, ku_data)
            ku_masked = np.ma.masked_where(np.isnan(ku_masked), ku_masked)  #masks NaN values (not masked in previous line)
            ku_good = True
        except:
            ku_good = False
    
    #if Ku-band is available
    if ku_good:            

        #Convert 'lo-res' APR times to datetimes
        time = apr_file['time'][:]  #For 'lores': Time of scan, in seconds since midnight UTC of [YYYY-mm-DD]
        alt3d = apr_file['lores_alt3D'][:]

        time_dates = np.empty(time.shape, dtype=object)
        for i in np.arange(0, time.shape[0]):
            #hour, second automatically revert to midnight (hour = 0, seconds = 0) for '%Y%m%d'
            time_dates[i] = datetime.strptime(file_date, '%Y%m%d') + timedelta(seconds = float(time[i].values))

        #Create a time at each gate (assuming time is the same for each ray of a given scan and down each ray)      
        time_gate = np.empty(alt3d.shape, dtype=object)
        for i in np.arange(0, alt3d.shape[0]):
            time_gate[i,:,:] = time_dates[i]   #assign the same time to all of a given scan's rays and height bins     

        time3d = np.copy(time_gate)

#         print (time3d[0,0,:])  #times should be the same
#         print ('')
#         print (time3d[0,:,0])  #times should be the same
#         print ('')
#         print (time3d[:,0,0])  #times should be different
#         sys.exit()

        #plot the APR data factoring in the aircraft roll (ray adjustment)
        #choose the "pseudo-nadir" ray factoring in aircraft roll
        roll = apr_file['lores_roll'][:]
        ray_angles = np.arange(-25,25.01,25/12)  #in degrees; 25 rays for each scan, with the middle (13th) scan being zero degrees
        for scan in range(roll.shape[0]):
            if (time_dates[scan] >= start_time) and (time_dates[scan] <= end_time):
                ac_roll = np.nanmean(roll[scan,:])  #roll varies slightly w/ray, so take the average roll value for a given scan and use that for ray adjustment
                ray_use = np.argmin(np.abs(ray_angles - ac_roll))  #the index of the ray whose angle is closest to that of ac_roll    

                #scan + 2 (and not scan + 1) is needed because pcolormesh colors the grid cell from the  
                #grid cell's time to the subsequent grid cell's time.  If a subsequent grid cell does not exist,  
                #then pcolormesh cannot/doesn't color the grid cell (remember, slicing is right side EXCLUSIVE, 
                #so scan:scan + 1 is only 1 element and thus doesn't have a subsequent cell!)
                #by this same logic, scan:scan + 2 will only color one grid cell, since the 2nd (and last) 
                #element/grid cell doesn't have a subsequent grid cell

                if ku_good:
                    pm0 = axes.pcolormesh(time3d[scan:scan+2,ray_use,:], alt3d[scan:scan+2,ray_use,:] / 1000,
                                          ku_masked[scan:scan+2,ray_use,:], cmap=cm.Spectral_r, vmin=vlim[0], vmax=vlim[1])

                else:
                    pass

    apr_file.close()

cbar0 = plt.colorbar(pm0, ax=axes, pad = 0.01, extend = 'both')
cbar0.set_label('Z [dBZ]')
#axes.set_title('ECCO-V Classifications and APR-3 Ku-band Reflectivity (Case 12, 07 September 2022)')

# dawn_skip = slice(None,None,7)   #only plot every 7th value
# drop_skip = slice(None,None,20)  #only plot every 20th value
dawn_skip = slice(None,None,10)   #only plot every 10th value
drop_skip = slice(None,None,50)  #only plot every 50th value

#set the plot start time as the beginning time of the first APR file and the
    #plot end time as the end time of the last APR file:
# range_start = datetime.strptime(apr_file_list[0][32:40] + apr_file_list[0][41:47], '%Y%m%d%H%M%S') 
# range_end = datetime.strptime(apr_file_list[-1][49:57] + apr_file_list[-1][58:64], '%Y%m%d%H%M%S')
# range_start = start_time
# range_end = end_time
range_start = datetime.strptime(start_time0, '%Y%m%d,%H%M%S') 
range_end = datetime.strptime(end_time0, '%Y%m%d,%H%M%S')

for xx, dawn_profile in enumerate(dawn_csv['Time [UTC]'].unique()):
    if xx % 2 == 0:  #skip plotting every other DAWN profile
        dawn_csv_prof = dawn_csv[dawn_csv['Time [UTC]'] == dawn_profile]
        axes.barbs(pd.to_datetime(dawn_csv_prof['Time [UTC]'])[dawn_skip], dawn_csv_prof['Height [m]'][dawn_skip] / 1000, 
                   dawn_csv_prof['U Comp of Wind [m/s]'][dawn_skip], dawn_csv_prof['V Comp of Wind [m/s]'][dawn_skip], 
                   fill_empty = True, length = 10, pivot='middle', sizes=dict(emptybarb=0.1), barbcolor = 'k')
    else:
        pass

# axes.barbs(pd.to_datetime(dawn_csv['Time [UTC]'])[dawn_skip], dawn_csv['Height [m]'][dawn_skip], 
#            dawn_csv['U Comp of Wind [m/s]'][dawn_skip], dawn_csv['V Comp of Wind [m/s]'][dawn_skip], 
#            fill_empty = True, length = 10, pivot='middle', sizes=dict(emptybarb=0.1), barbcolor = 'k')
axes.barbs(pd.to_datetime(drop_csv['Time [UTC]'])[drop_skip], drop_csv['Height [m]'][drop_skip] / 1000, 
           drop_csv['U Comp of Wind [m/s]'][drop_skip], drop_csv['V Comp of Wind [m/s]'][drop_skip], 
           fill_empty = True, length = 10, pivot='middle', sizes=dict(emptybarb=0.1), barbcolor = 'b')

##########################

axes.set_ylabel('Altitude [km]')
axes.set_xlabel('Time [UTC]')
axes.set_ylim([ylim[0],ylim[1]])
axes.tick_params(axis='x', rotation = 50)
axes.tick_params(length = 15, width = 5)
axes.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%H:%M:%S"))
axes.xaxis.set_major_locator(ticker.MaxNLocator(num_minutes))      #sets number of ticks
axes.set_xlim([np.datetime64(range_start),np.datetime64(range_end)])    
    #use the above line to narrow the plot's time range (even within a file!!)
        #range_start and range_end must be a datetime object or a string with the 
            #format: 'YYYY-MM-DD HH:MM:SS' or 'YYYY-MM-DDTHH:MM:SS'

plt.tight_layout()

#save the figure
#plt.savefig(os.path.join(day_folder, plot_save_name), bbox_inches = 'tight')
plt.savefig('/Users/ben/Desktop/Figure3a.png', bbox_inches = 'tight')
#plt.show()  #if you want to also show the image in the output cell, plt.show() must come after plt.savefig() in order for the image to save properly
plt.close()


##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
from PIL import Image
im = Image.open('/Users/ben/Desktop/Figure3a.png')

try:
    im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
except:
    #use this for older version of PIL/Pillow if the above line doesn't work, 
    #though this line will have isolated, extremely minor image effects due to 
    #only using 256 colors instead of the 3-element RGB scale
    im2 = im.convert('P')

im2.save('/Users/ben/Desktop/Figure3a.png')
im.close()
im2.close()


### For CPEX-CV AGU Paper (Figures 4, 6, and 7)

In [ ]:
#adapted from metric_boxwhisker_4panel_CPEXCV.py

# matplotlib.rcParams['axes.labelsize'] = 14
# matplotlib.rcParams['axes.titlesize'] = 14
# matplotlib.rcParams['xtick.labelsize'] = 12
# matplotlib.rcParams['ytick.labelsize'] = 12
# matplotlib.rcParams['legend.fontsize'] = 12
# matplotlib.rcParams['legend.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'

TC_days = [20170619, 20170620, 20210826, 20210828, 20210901, 20210904, 20220923]  #including TDs as TCs (from both CPEX and CPEX-AW and CPEX-CV)

#drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
df = pd.read_csv(drop_filepath)
df_noTC = df[~df['Date'].isin(TC_days)].copy()  #filter out cases associated with a TD/TC

#dawn_filepath = os.path.join(os.getcwd(), 'DAWN_Shear_Calculations.csv')
dawn_filepath = os.path.join(os.getcwd(), 'DAWN_Shear_Calculations_CPEXCV.csv')
df_dawn = pd.read_csv(dawn_filepath)

df1 = pd.concat([df, df_dawn], ignore_index = True)  #concatenates fields with same heading
df1_noTC = df1[~df1['Date'].isin(TC_days)].copy()  #filter out cases associated with a TD/TC

#df1_noTC = df_noTC.copy() #not sure what this was for....probably can delete

rh_layers = ['Deep Layer RH [%]', 'PBL RH [%]', 'Mid Layer RH [%]', 'Upper Layer RH [%]']
speed_shear_layers = ['SHARPpy Direct Method Deep Layer Speed Shear [kts]', 'SHARPpy Direct Method PBL Speed Shear [kts]', 
                      'SHARPpy Direct Method Mid Layer Speed Shear [kts]', 'SHARPpy Direct Method Upper Layer Speed Shear [kts]']
cape_layers = ['Deep Layer MUCAPE [J/kg]', 'Deep Layer MLCAPE [J/kg]', 'Above FZL MUCAPE [J/kg]', 'Above FZL MLCAPE [J/kg]']

use_alpha = 0.25  #alpha for box plots with all sondes, not just inflow sondes

#for convective type comparison of All Lifecycles dropsondes
def box_plot_onlyIsoOrg_RH(plotting_metric_name, ax):  #parameter will be a data column from df
    
    if plotting_metric_name == 'Upper Layer RH [%]':  #include "In Precip" sondes if "Partially In Precip" == Yes
        df_use0 = df_noTC[df_noTC['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
        df_use1 = df_use0[(df_use0['Environment Falling In'] != 'In Precip') | (df_use0['Partially In Precip'] == 'Yes')].copy()  #filter out In Precip, unless "Partially In Precip" == Yes
        df_use = df_use1[(df_use1['Low-level Inflow Sonde'] == 'Yes') | (df_use1['Mid-level Inflow Sonde'] == 'Yes')].copy()
        #df_use = df_noTC[(df_noTC['Environment Falling In'] != 'In Precip') & (df_noTC['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
    else:
        df_use1 = df_noTC[(df_noTC['Environment Falling In'] != 'In Precip') & (df_noTC['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
        df_use = df_use1[(df_use1['Low-level Inflow Sonde'] == 'Yes') | (df_use1['Mid-level Inflow Sonde'] == 'Yes')].copy()

    #all sondes
    df_iso_all = df_use1[df_use1['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all = df_use1[df_use1['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all = df_use1[df_use1['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso = df_use[df_use['Primary Convective Type'] == 'Isolated'].copy()
    df_org = df_use[df_use['Primary Convective Type'] == 'Organized'].copy()
    #df_scat = df_use[df_use['Primary Convective Type'] == 'Scattered'].copy()
    
    #need to filter out NaN values (using dropna()), otherwise the boxplot() won't create anything
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_scat[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                 patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated', 'Organized', 'Scattered'])
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
    #                  df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values, 
    #                  df_scat[plotting_metric_name].dropna().values, df_scat_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                  patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized', 'Scattered\n(Inflow)', 'Scattered'])
    bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
                     df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
                     patch_artist = True, vert = True, widths = 0.65, tick_labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized'])
    
        
    print (f'Isolated {plotting_metric_name} median:', df_iso[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median:', df_org[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median:', df_scat[plotting_metric_name].median(skipna = True))

    # colors = ['red', 'blue', 'black']
    #colors = ['red', 'red', 'blue', 'blue', 'black', 'black']
    colors = ['red', 'red', 'blue', 'blue']
    nums = list(range(len(colors)))
     
    for ii, patch, color in zip(nums, bp['boxes'], colors):
        if ii % 2 == 0:
            patch.set_facecolor(color)
        else:
            patch.set_facecolor(color)
            patch.set_alpha(use_alpha)

    #changing color and linewidth of medians
    for ii, median in enumerate(bp['medians']):
        if ii % 2 == 0:
            median.set(color = 'white', linewidth = 3)
        else:
            #median.set(color = 'white', alpha = use_alpha, linewidth = 3)
            median.set(color = 'white', linewidth = 3)

    #changing color and linewidth of whiskers
    for ii, whisker in enumerate(bp['whiskers']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 whiskers, not 4
            whisker.set(color = 'k', linewidth = 2, linestyle = "-")
        else:
            whisker.set(color = 'k', alpha = use_alpha, linewidth = 2, linestyle = "-")
     
    #changing color and linewidth of caps
    for ii, cap in enumerate(bp['caps']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 caps, not 4
            cap.set(color = 'k', linewidth = 2)
        else:
            cap.set(color = 'k', alpha = use_alpha, linewidth = 2)
     
    #changing style of fliers
    for ii, flier, color in zip(nums, bp['fliers'], colors):
        if ii % 2 == 0:
            flier.set(marker = 'o', color = 'k', markersize = 13, markerfacecolor = color)
        else:
            flier.set(marker = 'o', color = 'k', alpha = use_alpha, markersize = 13, markerfacecolor = color)
            
    ax.grid(True, axis = 'y')
    ax.set_title(plotting_metric_name[:-4], fontsize = 33, fontweight = 'bold')
    #ax.set_xlabel('Convective Type', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[%]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([15,100])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
 
    
#group_fig = plt.figure(figsize=(36,24))
group_fig = plt.figure(figsize=(22,24))

for i, layer in enumerate(rh_layers):
    ax = group_fig.add_subplot(2,2,i+1)
    box_plot_onlyIsoOrg_RH(layer, ax)

#custom legend
# legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 's', markersize = 13, label='Isolated (In Precip Profiles Excluded)'),
#                    Line2D([], [], color='blue', linewidth = 0, marker = 's', markersize = 13, label='Organized (In Precip Profiles Excluded)')]
# group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 25)

group_fig.text(0.5, 0.484, 'In Precip Profiles Excluded', horizontalalignment='center', verticalalignment='center', 
               fontsize = 40, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
plt.subplots_adjust(hspace = 0.45, wspace = 0.35)
plt.savefig('/Users/ben/Desktop/RH_box_4panel.png', bbox_inches = 'tight')
plt.close()
print ('')


#for convective type comparison of All Lifecycles dropsondes
def box_plot_onlyIsoOrg_SS(plotting_metric_name, ax):  #parameter will be a data column from df
    
    df_use1 = df_noTC[df_noTC['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
    df_use = df_use1[(df_use1['Low-level Inflow Sonde'] == 'Yes') | (df_use1['Mid-level Inflow Sonde'] == 'Yes')].copy()
    
    #all sondes
    df_iso_all = df_use1[df_use1['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all = df_use1[df_use1['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all = df_use1[df_use1['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso = df_use[df_use['Primary Convective Type'] == 'Isolated'].copy()
    df_org = df_use[df_use['Primary Convective Type'] == 'Organized'].copy()
    #df_scat = df_use[df_use['Primary Convective Type'] == 'Scattered'].copy()
    
    #need to filter out NaN values (using dropna()), otherwise the boxplot() won't create anything
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_scat[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                 patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated', 'Organized', 'Scattered'])
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
    #                  df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values, 
    #                  df_scat[plotting_metric_name].dropna().values, df_scat_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                  patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized', 'Scattered\n(Inflow)', 'Scattered'])
    bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
                     df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
                     patch_artist = True, vert = True, widths = 0.65, tick_labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized'])
        
    print (f'Isolated {plotting_metric_name} median:', df_iso[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median:', df_org[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median:', df_scat[plotting_metric_name].median(skipna = True))

    # colors = ['red', 'blue', 'black']
    #colors = ['red', 'red', 'blue', 'blue', 'black', 'black']
    colors = ['red', 'red', 'blue', 'blue']
    nums = list(range(len(colors)))
     
    for ii, patch, color in zip(nums, bp['boxes'], colors):
        if ii % 2 == 0:
            patch.set_facecolor(color)
        else:
            patch.set_facecolor(color)
            patch.set_alpha(use_alpha)

    #changing color and linewidth of medians
    for ii, median in enumerate(bp['medians']):
        if ii % 2 == 0:
            median.set(color = 'white', linewidth = 3)
        else:
            #median.set(color = 'white', alpha = use_alpha, linewidth = 3)
            median.set(color = 'white', linewidth = 3)        

    #changing color and linewidth of whiskers
    for ii, whisker in enumerate(bp['whiskers']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 whiskers, not 4
            whisker.set(color = 'k', linewidth = 2, linestyle = "-")
        else:
            whisker.set(color = 'k', alpha = use_alpha, linewidth = 2, linestyle = "-")
     
    #changing color and linewidth of caps
    for ii, cap in enumerate(bp['caps']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 caps, not 4
            cap.set(color = 'k', linewidth = 2)
        else:
            cap.set(color = 'k', alpha = use_alpha, linewidth = 2)
     
    #changing style of fliers
    for ii, flier, color in zip(nums, bp['fliers'], colors):
        if ii % 2 == 0:
            flier.set(marker = 'o', color = 'k', markersize = 13, markerfacecolor = color)
        else:
            flier.set(marker = 'o', color = 'k', alpha = use_alpha, markersize = 13, markerfacecolor = color)
            
    ax.grid(True, axis = 'y')
    ax.set_title(plotting_metric_name[22:-18] + ' Shear', fontsize = 33, fontweight = 'bold')
    #ax.set_xlabel('Convective Type', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[kts]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([0,50])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
 
#group_fig = plt.figure(figsize=(36,24))
group_fig = plt.figure(figsize=(22,24))

for i, layer in enumerate(speed_shear_layers):
    ax = group_fig.add_subplot(2,2,i+1)
    box_plot_onlyIsoOrg_SS(layer, ax)

#custom legend
# legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 's', markersize = 13, label='Isolated'),
#                    Line2D([], [], color='blue', linewidth = 0, marker = 's', markersize = 13, label='Organized')]
# group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 25)

group_fig.text(0.5, 0.484, 'In Precip Profiles Included', horizontalalignment='center', verticalalignment='center', 
               fontsize = 40, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
plt.subplots_adjust(hspace = 0.45, wspace = 0.35)
plt.savefig('/Users/ben/Desktop/SS_box_4panel.png', bbox_inches = 'tight')
plt.close()
print ('')


#for convective type comparison of All Lifecycles dropsondes
def box_plot_onlyIsoOrg_CAPE(plotting_metric_name, ax):  #parameter will be a data column from df
    
    df_use1 = df_noTC[(df_noTC['Environment Falling In'] != 'In Precip') & (df_noTC['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
    df_use = df_use1[(df_use1['Low-level Inflow Sonde'] == 'Yes') | (df_use1['Mid-level Inflow Sonde'] == 'Yes')].copy()
    
    #all sondes
    df_iso_all = df_use1[df_use1['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all = df_use1[df_use1['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all = df_use1[df_use1['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso = df_use[df_use['Primary Convective Type'] == 'Isolated'].copy()
    df_org = df_use[df_use['Primary Convective Type'] == 'Organized'].copy()
    #df_scat = df_use[df_use['Primary Convective Type'] == 'Scattered'].copy()
    
    #need to filter out NaN values (using dropna()), otherwise the boxplot() won't create anything
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_scat[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                 patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated', 'Organized', 'Scattered'])
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
    #                  df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values, 
    #                  df_scat[plotting_metric_name].dropna().values, df_scat_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                  patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized', 'Scattered\n(Inflow)', 'Scattered'])
    bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
                     df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
                     patch_artist = True, vert = True, widths = 0.65, tick_labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized'])
        
    print (f'Isolated {plotting_metric_name} median:', df_iso[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median:', df_org[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median:', df_scat[plotting_metric_name].median(skipna = True))

    # colors = ['red', 'blue', 'black']
    #colors = ['red', 'red', 'blue', 'blue', 'black', 'black']
    colors = ['red', 'red', 'blue', 'blue']
    nums = list(range(len(colors)))
     
    for ii, patch, color in zip(nums, bp['boxes'], colors):
        if ii % 2 == 0:
            patch.set_facecolor(color)
        else:
            patch.set_facecolor(color)
            patch.set_alpha(use_alpha)

    #changing color and linewidth of medians
    for ii, median in enumerate(bp['medians']):
        if ii % 2 == 0:
            median.set(color = 'white', linewidth = 3)
        else:
            #median.set(color = 'white', alpha = use_alpha, linewidth = 3)
            median.set(color = 'white', linewidth = 3)        

    #changing color and linewidth of whiskers
    for ii, whisker in enumerate(bp['whiskers']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 whiskers, not 4
            whisker.set(color = 'k', linewidth = 2, linestyle = "-")
        else:
            whisker.set(color = 'k', alpha = use_alpha, linewidth = 2, linestyle = "-")
     
    #changing color and linewidth of caps
    for ii, cap in enumerate(bp['caps']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 caps, not 4
            cap.set(color = 'k', linewidth = 2)
        else:
            cap.set(color = 'k', alpha = use_alpha, linewidth = 2)
     
    #changing style of fliers
    for ii, flier, color in zip(nums, bp['fliers'], colors):
        if ii % 2 == 0:
            flier.set(marker = 'o', color = 'k', markersize = 13, markerfacecolor = color)
        else:
            flier.set(marker = 'o', color = 'k', alpha = use_alpha, markersize = 13, markerfacecolor = color)
            
    ax.grid(True, axis = 'y')
    if 'Above FZL' in plotting_metric_name:
        ax.set_title('Upper Layer' + plotting_metric_name[9:-7], fontsize = 33, fontweight = 'bold')
    else:
        ax.set_title(plotting_metric_name[:-7], fontsize = 33, fontweight = 'bold')
    #ax.set_xlabel('Convective Type', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[J kg$^{-1}$]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([15,100])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    
    # #make the deep layer plot axes range the same and the above FZL plot axes the same for easier comparison
    # if plotting_metric_name[:4] == 'Deep':
    #     ax.set_ylim([-30,1450])
    # else:
    #     ax.set_ylim([-30,1000])
 
#group_fig = plt.figure(figsize=(36,24))
group_fig = plt.figure(figsize=(22,24))

for i, layer in enumerate(cape_layers):
    ax = group_fig.add_subplot(2,2,i+1)
    box_plot_onlyIsoOrg_CAPE(layer, ax)
    
#custom legend
# legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 's', markersize = 13, label='Isolated (In Precip Profiles Excluded)'),
#                    Line2D([], [], color='blue', linewidth = 0, marker = 's', markersize = 13, label='Organized (In Precip Profiles Excluded)')]
# group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 25)

group_fig.text(0.5, 0.484, 'In Precip Profiles Excluded', horizontalalignment='center', verticalalignment='center', 
               fontsize = 40, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
plt.subplots_adjust(hspace = 0.45, wspace = 0.35)
plt.savefig('/Users/ben/Desktop/CAPE_box_4panel.png', bbox_inches = 'tight')
plt.close()
print ('')


#for convective type comparison of All Lifecycles dropsondes
def box_plot_onlyIsoOrg_PBL(plotting_metric_name):  #parameter will be a data column from df

    #group_fig = plt.figure(figsize=(40,21))
    group_fig = plt.figure(figsize=(32,21))
    
    #plot the PBL Top box and whisker plot
    ax = group_fig.add_subplot(1,2,1)
    
    df_use1 = df_noTC[(df_noTC['Environment Falling In'] != 'In Precip') & (df_noTC['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
    df_use = df_use1[(df_use1['Low-level Inflow Sonde'] == 'Yes') | (df_use1['Mid-level Inflow Sonde'] == 'Yes')].copy()
    
    #all sondes
    df_iso_all = df_use1[df_use1['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all = df_use1[df_use1['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all = df_use1[df_use1['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso = df_use[df_use['Primary Convective Type'] == 'Isolated'].copy()
    df_org = df_use[df_use['Primary Convective Type'] == 'Organized'].copy()
    #df_scat = df_use[df_use['Primary Convective Type'] == 'Scattered'].copy()
    
    #need to filter out NaN values (using dropna()), otherwise the boxplot() won't create anything
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_scat[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                 patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated', 'Organized', 'Scattered'])
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
    #                  df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values, 
    #                  df_scat[plotting_metric_name].dropna().values, df_scat_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                  patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized', 'Scattered\n(Inflow)', 'Scattered'])
    bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
                     df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
                     patch_artist = True, vert = True, widths = 0.65, tick_labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized'])
        
    print (f'Isolated {plotting_metric_name} median:', df_iso[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median:', df_org[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median:', df_scat[plotting_metric_name].median(skipna = True))

    # colors = ['red', 'blue', 'black']
    #colors = ['red', 'red', 'blue', 'blue', 'black', 'black']
    colors = ['red', 'red', 'blue', 'blue']
    nums = list(range(len(colors)))
     
    for ii, patch, color in zip(nums, bp['boxes'], colors):
        if ii % 2 == 0:
            patch.set_facecolor(color)
        else:
            patch.set_facecolor(color)
            patch.set_alpha(use_alpha)

    #changing color and linewidth of medians
    for ii, median in enumerate(bp['medians']):
        if ii % 2 == 0:
            median.set(color = 'white', linewidth = 3)
        else:
            #median.set(color = 'white', alpha = use_alpha, linewidth = 3)
            median.set(color = 'white', linewidth = 3)        

    #changing color and linewidth of whiskers
    for ii, whisker in enumerate(bp['whiskers']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 whiskers, not 4
            whisker.set(color = 'k', linewidth = 2, linestyle = "-")
        else:
            whisker.set(color = 'k', alpha = use_alpha, linewidth = 2, linestyle = "-")
     
    #changing color and linewidth of caps
    for ii, cap in enumerate(bp['caps']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 caps, not 4
            cap.set(color = 'k', linewidth = 2)
        else:
            cap.set(color = 'k', alpha = use_alpha, linewidth = 2)
     
    #changing style of fliers
    for ii, flier, color in zip(nums, bp['fliers'], colors):
        if ii % 2 == 0:
            flier.set(marker = 'o', color = 'k', markersize = 13, markerfacecolor = color)
        else:
            flier.set(marker = 'o', color = 'k', alpha = use_alpha, markersize = 13, markerfacecolor = color)
            
    ax.grid(True, axis = 'y')
    ax.set_title('PBL Depth', fontsize = 33, fontweight = 'bold')
    ax.set_xlabel('Convective Type', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[mb]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([1010,900])
    ax.invert_yaxis()
    ax.set_yticks(np.arange(1010,899,-10))
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    ax.text(0.50, 0.97, 'In Precip Profiles Excluded', horizontalalignment='center', verticalalignment='center', 
            transform=ax.transAxes, fontsize = 30, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 5})
    
    #plot the PBL Top scatter plot
    ax = group_fig.add_subplot(1,2,2)
    
    plotting_metric = df[plotting_metric_name]
    xlist = []
    for i in range(len(df)):
        xstring = str(df['Case'][i])
        xlist.append(xstring)
    
    # legend_elements_PBL = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 13, label='Isolated (Clear)'),
    #                       Line2D([], [], color='red', linewidth = 0, marker = '$C$', markersize = 13, label='Isolated (In Cloud)'),
    #                       Line2D([], [], color='red', linewidth = 0, marker = '$P$', markersize = 13, label='Isolated (In Precip)'),
    #                       Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 13, label='Organized (Clear)'),
    #                       Line2D([], [], color='blue', linewidth = 0, marker = '$C$', markersize = 13, label='Organized (In Cloud)'),
    #                       Line2D([], [], color='blue', linewidth = 0, marker = '$P$', markersize = 13, label='Organized (In Precip)'),
    #                       Line2D([], [], color='black', linewidth = 0, marker = 'o', markersize = 13, label='Scattered (Clear)'),
    #                       Line2D([], [], color='black', linewidth = 0, marker = '$C$', markersize = 13, label='Scattered (In Cloud)'),
    #                       Line2D([], [], color='black', linewidth = 0, marker = '$P$', markersize = 13, label='Scattered (In Precip)')]
    
    # legend_elements_PBL = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 13, label='Isolated'),
    #                Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 13, label='Organized'),
    #                Line2D([], [], color='black', linewidth = 0, marker = 'o', markersize = 13, label='Scattered')]

    legend_elements_PBL = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 13, label='Isolated (Clear)'),
                          Line2D([], [], color='red', linewidth = 0, marker = '$C$', markersize = 13, label='Isolated (In Cloud)'),
                          Line2D([], [], color='red', linewidth = 0, marker = '$P$', markersize = 13, label='Isolated (In Precip)'),
                          Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 13, label='Organized (Clear)'),
                          Line2D([], [], color='blue', linewidth = 0, marker = '$C$', markersize = 13, label='Organized (In Cloud)'),
                          Line2D([], [], color='blue', linewidth = 0, marker = '$P$', markersize = 13, label='Organized (In Precip)')]
    
    #the lighter shades account for TS Cindy organized cases and, further, TS Cindy cases that were away from the main organized convection and instead near the cyclonic center
    for j in range(len(plotting_metric)):
        #if (df['Convective Lifecycle'][j] != 'Weakening') and (df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip'):
        if df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip':
        #if (df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip') and (df['Low-level Inflow Sonde'][j] == 'Yes' or df['Mid-level Inflow Sonde'][j] == 'Yes'):
            if df['Primary Convective Type'][j] == 'Isolated':
                color = 'red'
                if df['Date'][j] in TC_days:
                    continue
            elif df['Primary Convective Type'][j] == 'Organized':
                color = 'blue'
                if df['Date'][j] in TC_days:
                    continue
            elif df['Primary Convective Type'][j] == 'Scattered':
                color = 'black'
                if df['Date'][j] in TC_days:
                    continue               
            else:
                continue
                  
            if df['Environment Falling In'][j] == 'In Precip':
                if df['Partially In Precip'][j] == 'Yes':
                    mark = '$*P$'
                    mark = '$P$'
                else:
                    mark = '$P$'
                outline = None
            elif df['Environment Falling In'][j] == 'In Cloud':
                if df['Falling Through Weak Stratiform (Onion-ish Profile (Typically On Outer Edge of Storm))'][j] == 'Yes':
                    mark = '$*C$'
                    mark = '$C$'
                else:
                    mark = '$C$'
                outline = None
            else:
                mark = 'o'
                outline = 'black'

            #if the metric is not NaN (need this filter, otherwise Python raises an error when showing/saving the figure)
            if not np.isnan(plotting_metric[j]):  #could also use pd.isna(plotting_metric[j])
                if df['Low-level Inflow Sonde'][j] == 'Yes' or df['Mid-level Inflow Sonde'][j] == 'Yes':               
                    ax.scatter(xlist[j], plotting_metric[j], c = color, s = 150, marker = mark, edgecolor = outline)
                else:
                    ax.scatter(xlist[j], plotting_metric[j], c = color, alpha = use_alpha, s = 150, marker = mark, edgecolor = outline)
            
    ax.grid(True)
    ax.set_title('PBL Depth', fontsize = 33, fontweight = 'bold')
    ax.set_xlabel('Case', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[mb]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([1010,900])
    ax.invert_yaxis()
    ax.set_yticks(np.arange(1010,899,-10))
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    ax.legend(handles = legend_elements_PBL, fontsize = 18, loc = 'upper left', ncol = 1)

box_plot_onlyIsoOrg_PBL('PBL Top [mb]')
plt.subplots_adjust(wspace = 0.20)
plt.savefig('/Users/ben/Desktop/PBL_box_scatter.png', bbox_inches = 'tight')
plt.close()
print ('')

#for convective type comparison of All Lifecycles dropsondes
def box_plot_onlyIsoOrg_SSwDAWN(plotting_metric_name):  #parameter will be a data column from df1

    #group_fig = plt.figure(figsize=(40,21))
    group_fig = plt.figure(figsize=(32,21))
    ax = group_fig.add_subplot(1,2,1)
    
    df_use1 = df1_noTC[df1_noTC['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
    df_use = df_use1[(df_use1['Low-level Inflow Sonde'] != 'No') | (df_use1['Mid-level Inflow Sonde'] != 'No')].copy()
    
    #all sondes
    df_iso_all = df_use1[df_use1['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all = df_use1[df_use1['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all = df_use1[df_use1['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso = df_use[df_use['Primary Convective Type'] == 'Isolated'].copy()
    df_org = df_use[df_use['Primary Convective Type'] == 'Organized'].copy()
    #df_scat = df_use[df_use['Primary Convective Type'] == 'Scattered'].copy()
    
    #need to filter out NaN values (using dropna()), otherwise the boxplot() won't create anything
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_scat[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                 patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated', 'Organized', 'Scattered'])
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
    #                  df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values, 
    #                  df_scat[plotting_metric_name].dropna().values, df_scat_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                  patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized', 'Scattered\n(Inflow)', 'Scattered'])
    bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
                     df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
                     patch_artist = True, vert = True, widths = 0.65, tick_labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized'])
        
    print (f'Isolated {plotting_metric_name} median:', df_iso[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median:', df_org[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median:', df_scat[plotting_metric_name].median(skipna = True))

    # colors = ['red', 'blue', 'black']
    #colors = ['red', 'red', 'blue', 'blue', 'black', 'black']
    colors = ['red', 'red', 'blue', 'blue']
    nums = list(range(len(colors)))
     
    for ii, patch, color in zip(nums, bp['boxes'], colors):
        if ii % 2 == 0:
            patch.set_facecolor(color)
        else:
            patch.set_facecolor(color)
            patch.set_alpha(use_alpha)

    #changing color and linewidth of medians
    for ii, median in enumerate(bp['medians']):
        if ii % 2 == 0:
            median.set(color = 'white', linewidth = 3)
        else:
            #median.set(color = 'white', alpha = use_alpha, linewidth = 3)
            median.set(color = 'white', linewidth = 3)        

    #changing color and linewidth of whiskers
    for ii, whisker in enumerate(bp['whiskers']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 whiskers, not 4
            whisker.set(color = 'k', linewidth = 2, linestyle = "-")
        else:
            whisker.set(color = 'k', alpha = use_alpha, linewidth = 2, linestyle = "-")
     
    #changing color and linewidth of caps
    for ii, cap in enumerate(bp['caps']):
        if ii in [0, 1, 4, 5, 8, 9]:  #8 caps, not 4
            cap.set(color = 'k', linewidth = 2)
        else:
            cap.set(color = 'k', alpha = use_alpha, linewidth = 2)
     
    #changing style of fliers
    for ii, flier, color in zip(nums, bp['fliers'], colors):
        if ii % 2 == 0:
            flier.set(marker = 'o', color = 'k', markersize = 13, markerfacecolor = color)
        else:
            flier.set(marker = 'o', color = 'k', alpha = use_alpha, markersize = 13, markerfacecolor = color)
        
    ax.grid(True, axis = 'y')
    ax.set_title('a) Deep Layer Shear (Dropsonde & DAWN, 0.5km - 7.6km)', fontsize = 29.5, fontweight = 'bold')
    ax.set_xlabel('Convective Type', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[kts]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([0,50])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    ax.text(0.2405, 0.97, 'In Precip Profiles Included', horizontalalignment='center', verticalalignment='center', 
            transform=ax.transAxes, fontsize = 30, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 5})
    
    #plot the DAWN/dropsonde speed shear scatter plot
    ax = group_fig.add_subplot(1,2,2)
    
    plotting_metric = df1[plotting_metric_name]   #both dropsonde and DAWN data
    xlist = []
    for i in range(len(df1)):
        xstring = str(df1['Case'][i])
        xlist.append(xstring)
    
    #custom legend
    # legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 15, label='Isolated'),
    #                     Line2D([], [], color='red', linewidth = 0, marker = '$P$', markersize = 15, label='Isolated (In Precip)'),
    #                     Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 15, label='Organized'),
    #                     Line2D([], [], color='blue', linewidth = 0, marker = '$P$', markersize = 15, label='Organized (In Precip)'),
    #                     Line2D([], [], color='black', linewidth = 0, marker = 'o', markersize = 15, label='Scattered'),
    #                     Line2D([], [], color='black', linewidth = 0, marker = '$P$', markersize = 15, label='Scattered (In Precip)')]
    legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 13, label='Isolated (Clear)'),
                       Line2D([], [], color='red', linewidth = 0, marker = '$C$', markersize = 13, label='Isolated (In Cloud)'),
                       Line2D([], [], color='red', linewidth = 0, marker = '$P$', markersize = 13, label='Isolated (In Precip)'),
                       Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 13, label='Organized (Clear)'),
                       Line2D([], [], color='blue', linewidth = 0, marker = '$C$', markersize = 13, label='Organized (In Cloud)'),
                       Line2D([], [], color='blue', linewidth = 0, marker = '$P$', markersize = 13, label='Organized (In Precip)')]
    
    
    #the lighter shades account for TS Cindy organized cases and, further, TS Cindy cases that were away from the main organized convection and instead near the cyclonic center
    for j in range(len(plotting_metric)):
        #if (df1['Convective Lifecycle'][j] != 'Weakening') and (df1['Environment Falling In'][j] == 'Clear Near' or df1['Environment Falling In'][j] == 'In Cloud' or df1['Environment Falling In'][j] == 'In Precip'):
        if df1['Environment Falling In'][j] == 'Clear Near' or df1['Environment Falling In'][j] == 'In Cloud' or df1['Environment Falling In'][j] == 'In Precip':
        #if (df1['Environment Falling In'][j] == 'Clear Near' or df1['Environment Falling In'][j] == 'In Cloud' or df1['Environment Falling In'][j] == 'In Precip') and (df1['Low-level Inflow Sonde'][j] != 'No' or df1['Mid-level Inflow Sonde'][j] != 'No'):
            if df1['Primary Convective Type'][j] == 'Isolated':
                color = 'red'
                if df1['Date'][j] in TC_days:
                    continue
            elif df1['Primary Convective Type'][j] == 'Organized':
                color = 'blue'
                if df1['Date'][j] in TC_days:
                    continue
            # elif df1['Primary Convective Type'][j] == 'Scattered':
            #     color = 'black'
            #     if df1['Date'][j] in TC_days:
            #         continue 
            else:
                continue
                  
            if df1['Environment Falling In'][j] == 'In Precip':
                if df1['Partially In Precip'][j] == 'Yes':
                    mark = '$*P$'
                    mark = '$P$'
                else:
                    mark = '$P$'
                outline = None
            elif df1['Environment Falling In'][j] == 'In Cloud':
                if df1['Falling Through Weak Stratiform (Onion-ish Profile (Typically On Outer Edge of Storm))'][j] == 'Yes':
                    mark = '$*C$'
                    mark = '$C$'
                else:
                    mark = '$C$'
                outline = None
            else:
                mark = 'o'
                outline = 'black'

            #if the metric is not NaN (need this filter, otherwise Python raises an error when showing/saving the figure)
            if not np.isnan(plotting_metric[j]):  #could also use pd.isna(plotting_metric[j]) 
                if df1['Low-level Inflow Sonde'][j] != 'No' or df1['Mid-level Inflow Sonde'][j] != 'No':               
                    ax.scatter(xlist[j], plotting_metric[j], c = color, s = 150, marker = mark, edgecolor = outline)
                else:
                    ax.scatter(xlist[j], plotting_metric[j], c = color, alpha = use_alpha, s = 150, marker = mark, edgecolor = outline)
    
    ax.grid(True)
    ax.set_title('b) Deep Layer Shear (Dropsonde & DAWN, 0.5km - 7.6km)', fontsize = 29.5, fontweight = 'bold')
    ax.set_xlabel('Case', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[kts]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([0,50])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    ax.legend(handles = legend_elements, fontsize = 22, ncol = 1, loc = 'upper left')

box_plot_onlyIsoOrg_SSwDAWN('500m Bottom Cap Deep Layer Speed Shear [kts]')
plt.subplots_adjust(wspace = 0.20)
plt.savefig('/Users/ben/Desktop/SSwDAWN_box_scatter.png', bbox_inches = 'tight')
plt.close()


### For CPEX-CV AGU Paper (Figures 5 and 8)

In [ ]:
#adapted from metric_scatter_4panel_allCPEX.py

# matplotlib.rcParams['axes.labelsize'] = 14
# matplotlib.rcParams['axes.titlesize'] = 14
# matplotlib.rcParams['xtick.labelsize'] = 12
# matplotlib.rcParams['ytick.labelsize'] = 12
# matplotlib.rcParams['legend.fontsize'] = 12
#matplotlib.rcParams['legend.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'

#drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
df = pd.read_csv(drop_filepath)

# #change the old index (which still references non-sorted rows) to a new index (which references the sorted rows)
# new_index = np.arange(0, len(df1), 1)
# df_sort = df.sort_values(['Region', 'Case']).set_index(new_index)
# #^^^orders by region first, then by case within each region group

TC_days = ['20170619','20170620','20210826','20210828','20210901','20210904', '20220923']  #including TDs as TCs (from both CPEX and CPEX-AW and CPEX-CV)

rh_layers = ['Deep Layer RH [%]', 'PBL RH [%]', 'Mid Layer RH [%]', 'Upper Layer RH [%]']
speed_shear_layers = ['SHARPpy Direct Method Deep Layer Speed Shear [kts]', 'SHARPpy Direct Method PBL Speed Shear [kts]', 
                      'SHARPpy Direct Method Mid Layer Speed Shear [kts]', 'SHARPpy Direct Method Upper Layer Speed Shear [kts]']
cape_layers = ['Deep Layer MUCAPE [J/kg]', 'Deep Layer MLCAPE [J/kg]', 'Above FZL MUCAPE [J/kg]', 'Above FZL MLCAPE [J/kg]']

use_alpha = 0.25  #alpha for non-inflow sondes
#use_alpha = 1.0

#custom legend
legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 15, label='Isolated (Clear)'),
                    Line2D([], [], color='red', linewidth = 0, marker = '$C$', markersize = 15, label='Isolated (In Cloud)'),
                    Line2D([], [], color='red', linewidth = 0, marker = '$P$', markersize = 15, label='Isolated (In Precip)'),
                    Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 15, label='Organized (Clear)'),
                    Line2D([], [], color='blue', linewidth = 0, marker = '$C$', markersize = 15, label='Organized (In Cloud)'),
                    Line2D([], [], color='blue', linewidth = 0, marker = '$P$', markersize = 15, label='Organized (In Precip)')]

# legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 15, label='Isolated (Clear)'),
#                     Line2D([], [], color='red', linewidth = 0, marker = '$C$', markersize = 15, label='Isolated (In Cloud)'),
#                     Line2D([], [], color='red', linewidth = 0, marker = '$P$', markersize = 15, label='Isolated (In Precip)'),
#                     Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 15, label='Organized (Clear)'),
#                     Line2D([], [], color='blue', linewidth = 0, marker = '$C$', markersize = 15, label='Organized (In Cloud)'),
#                     Line2D([], [], color='blue', linewidth = 0, marker = '$P$', markersize = 15, label='Organized (In Precip)'),
#                     Line2D([], [], color='black', linewidth = 0, marker = 'o', markersize = 15, label='Scattered (Clear)'),
#                     Line2D([], [], color='black', linewidth = 0, marker = '$C$', markersize = 15, label='Scattered (In Cloud)'),
#                     Line2D([], [], color='black', linewidth = 0, marker = '$P$', markersize = 15, label='Scattered (In Precip)')]

# legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 15, label='Isolated'),
#                    Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 15, label='Organized'),
#                    Line2D([], [], color='black', linewidth = 0, marker = 'o', markersize = 15, label='Scattered')]

#######################################################################################################

#IMPORTANT:  HOW TO FIX 'KeyError: 0' error:  NEED .iloc[j] AND NOT JUST [j] BECAUSE [j] refers to the index, 
#not the integer position (IP), and groupby function only adjusts IP when making groups and not index!
 
#for convective type comparison of All Lifecycles dropsondes
def make_plot_onlyIsoOrg_RH(plotting_metric_name, ax):  #parameter will be a data column from df
    
    plotting_metric = df[plotting_metric_name]
    xlist = []
    for i in range(len(df)):
        xstring = str(df['Case'][i])
        xlist.append(xstring)
    
    for j in range(len(plotting_metric)):
        #if (df['Convective Lifecycle'][j] != 'Weakening') and (df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip'):
        if df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip':
        #if (df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip') and (df['Low-level Inflow Sonde'][j] == 'Yes' or df['Mid-level Inflow Sonde'][j] == 'Yes'):
            if df['Primary Convective Type'][j] == 'Isolated':
                color = 'red'
                if str(df['Date'][j]) in TC_days:
                    continue
            elif df['Primary Convective Type'][j] == 'Organized':
                color = 'blue'
                if str(df['Date'][j]) in TC_days:
                    continue
            # elif df['Primary Convective Type'][j] == 'Scattered':
            #     color = 'black'
            #     if str(df['Date'][j]) in TC_days:
            #         continue                
            else:
                continue                
            
            if df['Environment Falling In'][j] == 'In Precip':
                if df['Partially In Precip'][j] == 'Yes':
                    mark = '$*P$'
                    mark = '$P$'
                else:
                    mark = '$P$'
                outline = None
            elif df['Environment Falling In'][j] == 'In Cloud':
                if df['Falling Through Weak Stratiform (Onion-ish Profile (Typically On Outer Edge of Storm))'][j] == 'Yes':
                    mark = '$*C$'
                    mark = '$C$'
                else:
                    mark = '$C$'
                outline = None
            else:
                mark = 'o'
                outline = 'black'
            
            #if the metric is not NaN (need this filter, otherwise Python raises an error when showing/saving the figure)
            if not np.isnan(plotting_metric[j]):  #could also use pd.isna(plotting_metric[j])
                if df['Low-level Inflow Sonde'][j] == 'Yes' or df['Mid-level Inflow Sonde'][j] == 'Yes':
                    ax.scatter(xlist[j], plotting_metric[j], c = color, s = 150, marker = mark, edgecolor = outline)
                else:
                    ax.scatter(xlist[j], plotting_metric[j], c = color, alpha = use_alpha, s = 150, marker = mark, edgecolor = outline)
            else:   #use this if you want each case to show up in the plot, regardless if it has data or not
                ax.scatter(xlist[j], 80, c = color, alpha = 0.0, s = 150)
            
    ax.grid(True)
    ax.set_title(plotting_metric_name[:-4], fontsize = 33, fontweight = 'bold')
    ax.set_xlabel('Case', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[%]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([15,100])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
 
#group_fig = plt.figure(figsize=(15,24))   #CPEX(-AW) with isolated and organized cases
group_fig = plt.figure(figsize=(32,24))   #CPEX-CV with isolated, organized, (and scattered if you want) cases

for i, layer in enumerate(rh_layers):
    ax = group_fig.add_subplot(2,2,i+1)
    make_plot_onlyIsoOrg_RH(layer, ax)

#group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 28, title = '$\\bf{Convective Type}$', title_fontsize = 30, ncol = 3)
group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 28, ncol = 2)
plt.subplots_adjust(hspace = 0.6, wspace = 0.22)
plt.savefig('/Users/ben/Desktop/RH_4panel.png', bbox_inches = 'tight')
plt.close()


def make_plot_onlyIsoOrg_SS(plotting_metric_name, ax):  #parameter will be a data column from df
    
    plotting_metric = df[plotting_metric_name]
    xlist = []
    for i in range(len(df)):
        xstring = str(df['Case'][i])
        xlist.append(xstring)
    
    for j in range(len(plotting_metric)):
        #if (df['Convective Lifecycle'][j] != 'Weakening') and (df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip'):
        if df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip':
        #if (df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip') and (df['Low-level Inflow Sonde'][j] == 'Yes' or df['Mid-level Inflow Sonde'][j] == 'Yes'):
            if df['Primary Convective Type'][j] == 'Isolated':
                color = 'red'
                if str(df['Date'][j]) in TC_days:
                    continue
            elif df['Primary Convective Type'][j] == 'Organized':
                color = 'blue'
                if str(df['Date'][j]) in TC_days:
                    continue
            # elif df['Primary Convective Type'][j] == 'Scattered':
            #     color = 'black'
            #     if str(df['Date'][j]) in TC_days:
            #         continue                
            else:
                continue              
                  
            if df['Environment Falling In'][j] == 'In Precip':
                if df['Partially In Precip'][j] == 'Yes':
                    mark = '$*P$'
                    mark = '$P$'
                else:
                    mark = '$P$'
                outline = None
            elif df['Environment Falling In'][j] == 'In Cloud':
                if df['Falling Through Weak Stratiform (Onion-ish Profile (Typically On Outer Edge of Storm))'][j] == 'Yes':
                    mark = '$*C$'
                    mark = '$C$'
                else:
                    mark = '$C$'
                outline = None
            else:
                mark = 'o'
                outline = 'black'
        
            #if the metric is not NaN (need this filter, otherwise Python raises an error when showing/saving the figure)
            if not np.isnan(plotting_metric[j]):  #could also use pd.isna(plotting_metric[j])
                if df['Low-level Inflow Sonde'][j] == 'Yes' or df['Mid-level Inflow Sonde'][j] == 'Yes':
                    ax.scatter(xlist[j], plotting_metric[j], c = color, s = 150, marker = mark, edgecolor = outline)
                else:
                    ax.scatter(xlist[j], plotting_metric[j], c = color, alpha = use_alpha, s = 150, marker = mark, edgecolor = outline)
            else:   #use this if you want each case to show up in the plot, regardless if it has data or not
                ax.scatter(xlist[j], 10, c = color, alpha = 0.0, s = 150)
                
    ax.grid(True)
    ax.set_title(plotting_metric_name[22:-18] + ' Shear', fontsize = 33, fontweight = 'bold')
    ax.set_xlabel('Case', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[kts]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([0,50])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
 
#group_fig = plt.figure(figsize=(15,24))   #CPEX(-AW) with isolated and organized cases
group_fig = plt.figure(figsize=(32,24))   #CPEX-CV with isolated, organized, (and scattered if you want) cases

for i, layer in enumerate(speed_shear_layers):
    ax = group_fig.add_subplot(2,2,i+1)
    make_plot_onlyIsoOrg_SS(layer, ax)

#group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 28, title = '$\\bf{Convective Type}$', title_fontsize = 30, ncol = 3)
group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 28, ncol = 2)
plt.subplots_adjust(hspace = 0.6, wspace = 0.22)
plt.savefig('/Users/ben/Desktop/SS_4panel.png', bbox_inches = 'tight')
plt.close()


def make_plot_onlyIsoOrg_CAPE(plotting_metric_name, ax):  #parameter will be a data column from df
    
    plotting_metric = df[plotting_metric_name]
    xlist = []
    for i in range(len(df)):
        xstring = str(df['Case'][i])
        xlist.append(xstring)
    
    for j in range(len(plotting_metric)):
        #if (df['Convective Lifecycle'][j] != 'Weakening') and (df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip'):
        if df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip':
        #if (df['Environment Falling In'][j] == 'Clear Near' or df['Environment Falling In'][j] == 'In Cloud' or df['Environment Falling In'][j] == 'In Precip') and (df['Low-level Inflow Sonde'][j] == 'Yes' or df['Mid-level Inflow Sonde'][j] == 'Yes'):            
            if df['Primary Convective Type'][j] == 'Isolated':
                color = 'red'
                if str(df['Date'][j]) in TC_days:
                    continue
            elif df['Primary Convective Type'][j] == 'Organized':
                color = 'blue'
                if str(df['Date'][j]) in TC_days:
                    continue
            # elif df['Primary Convective Type'][j] == 'Scattered':
            #     color = 'black'
            #     if str(df['Date'][j]) in TC_days:
            #         continue                
            else:
                continue
                  
            if df['Environment Falling In'][j] == 'In Precip':
                if df['Partially In Precip'][j] == 'Yes':
                    mark = '$*P$'
                    mark = '$P$'
                else:
                    mark = '$P$'
                outline = None
            elif df['Environment Falling In'][j] == 'In Cloud':
                if df['Falling Through Weak Stratiform (Onion-ish Profile (Typically On Outer Edge of Storm))'][j] == 'Yes':
                    mark = '$*C$'
                    mark = '$C$'
                else:
                    mark = '$C$'
                outline = None
            else:
                mark = 'o'
                outline = 'black'

            #if the metric is not NaN (need this filter, otherwise Python raises an error when showing/saving the figure)
            if not np.isnan(plotting_metric[j]):  #could also use pd.isna(plotting_metric[j])
                if df['Low-level Inflow Sonde'][j] == 'Yes' or df['Mid-level Inflow Sonde'][j] == 'Yes':
                    ax.scatter(xlist[j], plotting_metric[j], c = color, s = 150, marker = mark, edgecolor = outline)
                else:
                    ax.scatter(xlist[j], plotting_metric[j], c = color, alpha = use_alpha, s = 150, marker = mark, edgecolor = outline)
            else:   #use this if you want each case to show up in the plot, regardless if it has data or not
                ax.scatter(xlist[j], 200, c = color, alpha = 0.0, s = 150)
                
    ax.grid(True)
    if 'Above FZL' in plotting_metric_name:
        ax.set_title('Upper Layer' + plotting_metric_name[9:-7], fontsize = 33, fontweight = 'bold')
    else:
        ax.set_title(plotting_metric_name[:-7], fontsize = 33, fontweight = 'bold')
    ax.set_xlabel('Case', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[J kg$^{-1}$]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    
    # #make the deep layer plot axes range the same and the above FZL plot axes the same for easier comparison
    # if plotting_metric_name[:4] == 'Deep':
    #     ax.set_ylim([-30,1450])
    # else:
    #     ax.set_ylim([-30,1000])

 
#group_fig = plt.figure(figsize=(15,24))   #CPEX(-AW) with isolated and organized cases
group_fig = plt.figure(figsize=(32,24))   #CPEX-CV with isolated, organized, (and scattered if you want) cases

for i, layer in enumerate(cape_layers):
    ax = group_fig.add_subplot(2,2,i+1)
    make_plot_onlyIsoOrg_CAPE(layer, ax)

#group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 28, title = '$\\bf{Convective Type}$', title_fontsize = 30, ncol = 3)
group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 28, ncol = 2)
plt.subplots_adjust(hspace=0.6, wspace = 0.22)
plt.savefig('/Users/ben/Desktop/CAPE_4panel.png', bbox_inches = 'tight')
plt.close()


### For CPEX-CV AGU Paper (Figure 9)

In [ ]:
#CFADs (with and without convective-stratiform partitioning)
#adapted from CFAD_AGUpapers_only_pseudonadir_ray_and_ECCO.py

matplotlib.rcParams['font.family'] = 'arial'
# matplotlib.rcParams['axes.labelsize'] = 14
# matplotlib.rcParams['axes.titlesize'] = 14
# matplotlib.rcParams['xtick.labelsize'] = 12
# matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['legend.fontsize'] = 24
#matplotlib.rcParams['legend.facecolor'] = 'w'

#create the dictionary of cases for which you want to plot a total CFAD and create the height/reflectivity bin edges

#CPEX-CV cases (Cases 1-7 are Isolated; Cases 8-22 are Organized (ignore Case 19 (TC)); Cases 23-24 are Scattered)
case1_dict = {1: ['20220909','161000','171000']}
case2_dict = {2: ['20220909','173500','191000']}
case3_dict = {3: ['20220910','190500','194200']}
case4_dict = {4: ['20220910', '202500', '204500']}
case5_dict = {5: ['20220916', '155500', '163500' ]}
case6_dict = {6: ['20220920', '071000', '073500']}
case7_dict = {7: ['20220920', '083000', '090000']}
case8_dict = {8: ['20220906', '110000', '120000']}
case9_dict = {'9a': ['20220906', '133000', '143000'], '9b': ['20220906', '153000', '161000']}
case10_dict = {10: ['20220906', '161000', '180000']}
case11_dict = {11: ['20220907', '130000', '134500']}
case12_dict = {'12a': ['20220907', '134500', '151300'], '12b': ['20220907', '161800', '174500']}
case13_dict = {13: ['20220907', '151300', '161800']}
case14_dict = {'14a': ['20220914', '101000', '115500'], '14b': ['20220914', '134000', '142800']}
case15_dict = {'15a': ['20220914','115500','131000'], '15b': ['20220914','142800','164500']}
case16_dict = {16: ['20220916','143000','154000']}
case17_dict = {17: ['20220916','164000','183500']}
case18_dict = {'18a': ['20220922', '054000', '061500'], '18b': ['20220922', '063500', '073600'], '18c': ['20220922', '080000', '083000']}
case19_dict = {19: ['20220923', '092000', '143000']}
case20_dict = {20: ['20220926', '072000', '111500']}
case21_dict = {21: ['20220929', '103500', '134500']}
case22_dict = {22: ['20220930', '134800', '143000']}
case23_dict = {23: ['20220910', '153500', '183000']}
case24_dict = {'24a': ['20220930', '092000', '102000'], '24b': ['20220930', '111000', '123000'], '24c': ['20220930', '131800', '134000']}

# #CPEX(-AW) cases
# case1_dict = {1: ['20170610','194655','221900']}
# case2_dict = {2: ['20170624','180000','194800']}
# case3_dict = {3: ['20170624','201200','220000']}
# case4_dict = {4: ['20170615', '184840', '205000']}
# case5_dict = {5: ['20170616', '182451', '220600' ]}
# case6_dict = {6: ['20170601', '175849', '220700']}
# case7_dict = {7: ['20170606', '185211', '215000']}
# case8_dict = {8: ['20170617', '184650', '220000']}
# case9_dict = {9: ['20170619', '173229', '223000']}
# case10_dict = {10: ['20170620', '171600', '220600']}
# case11_dict = {11: ['20170602', '174415', '221221']}
# case12_dict1 = {12: ['20170611', '170000', '174500']}
# case12_dict2 = {12: ['20170611', '210000', '214500']}
# case13_dict = {13: ['20170611', '180100', '203400']}
# case14_dict = {14: ['20210821', '221800', '234145']}
# case16_dict = {16: ['20210824', '181545', '195745']}

height_edges = np.arange(1500, 8001, 500)
dbz_edges = np.arange(-20, 70.1, 5)
vel_edges = np.arange(-25, 25.1, 2)
# dbz_edges = np.arange(-20, 60.1, 5)
# vel_edges = np.arange(-13, 15.1, 2)

def plot_CFAD(cfad_array, contours, colorbar_label, save_label, var_label, var_centers_meshgrid, height_centers_meshgrid, var_edges, case_dict, median_height_profile_above1500m, medianDBZ_profile_above1500m, Q1DBZ_profile_above1500m, Q3DBZ_profile_above1500m, dbz_plot = True):
    
    """Plot a contourf CFAD given a CFAD 2-D array, contour levels, plot/image labels, and variable/height meshgrids"""
    
    #plot the CFAD
    fig, ax = plt.subplots(1,1, figsize=(21,21))
    cmap = mplc.ListedColormap(['#ffffff', '#d8fcfa', '#bef5f6', '#aaedf1', '#98e4ec', '#89dae7', '#7cd0e2', 
                                '#70c6dd', '#65bcd9', '#5cb2d4', '#53a8cf', '#4b9dca', '#4393c5', '#3c89c0', 
                                '#357ebb', '#2f74b6', '#286ab1', '#2160ac', '#1956a7', '#0f4ca2', '#00429d'])
    
    #the cmap below omits the white fill at the beginnning
    # cmap = mplc.ListedColormap(['#d8fcfa', '#bef5f6', '#aaedf1', '#98e4ec', '#89dae7', '#7cd0e2', 
    #                             '#70c6dd', '#65bcd9', '#5cb2d4', '#53a8cf', '#4b9dca', '#4393c5', '#3c89c0', 
    #                             '#357ebb', '#2f74b6', '#286ab1', '#2160ac', '#1956a7', '#0f4ca2', '#00429d'])
    
    #cs = ax.pcolormesh(dbz_meshgrid, height_meshgrid, cfad_array, cmap = cmap)
    #cs = ax.contour(dbz_centers_meshgrid, height_centers_meshgrid, cfad_array, levels = contours, cmap = cmap, linewidths = 1)
    cs = ax.contourf(var_centers_meshgrid, height_centers_meshgrid, cfad_array, levels = contours, cmap = cmap)
    if dbz_plot:
        ax.plot(medianDBZ_profile_above1500m, median_height_profile_above1500m, color = 'k', linestyle = '-', linewidth = 5, label = 'Case ' + ','.join(map(str, list(case_dict.keys()))) + ' Median Profile') 
        ax.plot(Q1DBZ_profile_above1500m, median_height_profile_above1500m, color = 'k', linestyle = '--', linewidth = 5, label = 'Case ' + ','.join(map(str, list(case_dict.keys()))) + ' Q1 and Q3 Profiles')
        ax.plot(Q3DBZ_profile_above1500m, median_height_profile_above1500m, color = 'k', linestyle = '--', linewidth = 5)
        ax.legend(loc = 'upper right')
    ax.set_ylabel('Altitude [m]', fontsize=30, fontweight = 'bold')
    ax.set_xlabel(var_label, fontsize=30, fontweight = 'bold')
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    #ax.set_title('Total CFAD for Case ' + ','.join(map(str, list(case_dict.keys()))), fontsize=35, fontweight = 'bold')
    ax.set_title('Case ' + ','.join(map(str, list(case_dict.keys()))) + ' Normalized CFAD', fontsize=35, fontweight = 'bold')
    ax.set_ylim([height_edges[0] + 250, height_edges[-1] - 250])
    #ax.set_xlim([var_edges[0],var_edges[-1]])
    ax.set_xlim([5,55])  #for Ku-band
    
    #set the colorbar axis
    cax = fig.add_axes([ax.get_position().x0, ax.get_position().y0 - 0.08,
                       ax.get_position().x1-ax.get_position().x0, 0.02])    #Left, bottom, width, height (all [0,1])

    #create the colorbar
    cbar = plt.colorbar(cs, cax = cax, orientation = 'horizontal')
    cbar.ax.tick_params(length = 10, width = 3, labelsize = 23)
    cbar.set_label(label = colorbar_label, fontsize = 30, fontweight = 'bold')
    
    #save the figure
    plt.savefig(''.join(['/Users/ben/Desktop/', save_label]), bbox_inches = 'tight')
    plt.close()


def CFAD(case_dict, height_bin_edges, dbz_bin_edges, vel_bin_edges, normalize = True):
    
    """"Calculate and plot Ku-band and Doppler Velocity CFAD 2-D arrays for 
        the given cases and their respective time ranges
    
    Parameters
    ----------
    case_dict:  dictionary of cases for which to calculate the total CFAD; 
                keys should be case numbers;
                values should be 3-element lists of case date and start/end times, 
                with date strings formatted as YYYYMMDD and time strings formatted as HHMMSS
    
    height_bin_edges:  a 1-D array of height [m] bin edges used to create the 2-D CFAD array/histogram    

    dbz_bin_edges:  a 1-D array of reflectivity [dBZ] bin edges used to create the 2-D CFAD array/histogram 

    vel_bin_edges:  a 1-D array of Doppler velocity [m/s] bin edges used to create the 2-D CFAD array/histogram       
                        
    normalized:  True/False; determines whether to normalize the CFAD array by maximum bin at any level 
                 (method from Zagrodnik et al., 2019) and create an additional, normalized CFAD plot
                        
    return:  2-D (normalized) CFAD arrays and plots """

    assert type(case_dict) == dict, "case_dict must be a dictionary"
    
    total_apr_profiles = 0 
    apr_profile_roll10 = 0
    new_ku_array = True
    new_vel_array = True
    
    for key in case_dict:
        print ('Processing Case {}...'.format(key))
        
        first_good_Ku_file_index = 0    #used to determine first usable Ku-band file for CPEX-CV cases below; without this, an error is brought up due to the first APR file in apr_files_use only being W-band data (i.e., _Wn.nc)
        
        #grab the case's date and start/end times from the dictionary
        assert type(case_dict[key]) == list, "A key's values must be a list of date, start time, and end time" 
        desired_date = case_dict[key][0]
        start_time = case_dict[key][1]
        end_time = case_dict[key][2]
        assert start_time < end_time, "Start time must precede end time in a key's list"
        
        #grab the ECCO-V convective-stratiform classification file for the given date
        use_file_list = []
        for file in os.listdir(os.path.join(os.getcwd(), 'ECCO-V_output', 'ECCO-V_classification_1D')):
            if desired_date in file:
                use_file_list.append(file)
        assert len(use_file_list) == 1, "Found either 0 or multiple ECCO-V files for the given date"
        use_file = use_file_list[0]
        
        ecco_filepath = os.path.join(os.getcwd(), 'ECCO-V_output', 'ECCO-V_classification_1D', use_file)
        ecco_df = pd.read_csv(ecco_filepath, sep = ',', dtype = str)
        ecco_df['Full_Datetime'] = pd.to_datetime(ecco_df['Year'].str.zfill(4) + ecco_df['Month'].str.zfill(2) + ecco_df['Day'].str.zfill(2) + ecco_df['Hour'].str.zfill(2) + ecco_df['Minute'].str.zfill(2) + ecco_df['Second'].astype(float).round().astype(int).astype(str), format = '%Y%m%d%H%M%S')
        
        #find the APR files of interest (for the desired date and time ranges)
        apr_folder = os.path.join(desired_date, 'APR_files')
        apr_file_list = sorted(os.listdir(apr_folder))
        
        #angles for each of the 24 rays in a given scan (used for ray adjustment; index order goes from left to right in the scan when looking ahead in the direction that the aircraft is headed)
        if desired_date[:4] == '2017':
            ray_angles = np.linspace(-25,25,24)[:-1]  #in degrees; omits 24th ray, which doesn't have data for Ku/Ka bands
        
            #find the list of APR files that have data for the desired time range
            apr_files_use = []
            first_file = 'blank'
            for file in apr_file_list:        #sorted() makes sure the code goes through the files in alphabetical order
                if file[0:3] == '.DS':         #delete possible .DS_Store files
                    os.remove(os.path.join(apr_folder, file))
                elif file[22:28] <= start_time:
                    first_file = file     #first_file will always be the file immediately before (or equal to) range_start
                elif file[22:28] >= end_time:
                    continue
                else:
                    if (first_file not in apr_files_use) and (first_file != 'blank'):
                        apr_files_use.append(first_file)
                    apr_files_use.append(file)
                    
            if apr_files_use == []:             #accounts for if start_time is greater than all of the file times, but still within the last file's time range; also accounts for start/end times equaling the times of adjacent files
                apr_files_use.append(first_file)
                
        elif desired_date[:4] == '2021':
            ray_angles = np.linspace(-25,25,25)
            
            #create a list of all the given day's desired range's APR files:    #for APR_plots.py
            start_time1 = datetime.strptime(desired_date + start_time, '%Y%m%d%H%M%S')
            end_time1 = datetime.strptime(desired_date + end_time, '%Y%m%d%H%M%S')
            
            for x in os.listdir(apr_folder):
                if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
                    os.remove(os.path.join(apr_folder, x))
            
            #find the starting APR file in apr_folder
            first_file_index = None       
            for i, x in enumerate(apr_file_list):
                file_start_time = datetime.strptime(x[13:21] + x[22:28], '%Y%m%d%H%M%S')
                file_end_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
            
                if start_time1 <= file_start_time:  #if start_time1 is before the APR file start time and not within any previous APR file's time ranges
                    first_file_index = i
                    break
                elif (start_time1 >= file_start_time) and (start_time1 < file_end_time):
                    first_file_index = i
                    break
                else:
                    continue
            if first_file_index == None:
                sys.exit('Requested start_time is beyond all available APR files')
                
            #find the ending APR file in apr_folder
            last_file_index = None       
            for i, x in enumerate(apr_file_list):  
                file_start_time = datetime.strptime(x[13:21] + x[22:28], '%Y%m%d%H%M%S')
                file_end_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
            
                if end_time1 <= file_start_time:  #if end_time1 is before the APR file start time and not within any previous APR file's time ranges
                    last_file_index = i - 1
                    break
                elif (end_time1 > file_start_time) and (end_time1 <= file_end_time):
                    last_file_index = i
                    break
                else:
                    continue
            if last_file_index == None:  #the end_time is after all available APR files
                last_file_index = len(apr_file_list) - 1  #the last available APR file's index
            if last_file_index == -1:
                sys.exit('Requested end_time is before all available APR files')
            
            apr_files_use = apr_file_list[first_file_index:last_file_index + 1]
        
        elif desired_date[:4] == '2022':
            ray_angles = np.linspace(-25,25,25)
            
            #create a list of all the given day's desired range's APR files:    #for APR_plots.py
            start_time1 = datetime.strptime(desired_date + start_time, '%Y%m%d%H%M%S')
            end_time1 = datetime.strptime(desired_date + end_time, '%Y%m%d%H%M%S')
            
            #create a list of all the given day's desired range's APR files:
            for x in os.listdir(apr_folder):
                if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
                    os.remove(os.path.join(apr_folder, x))
            
            #find the starting APR file in apr_folder
            first_file_index = None 
            
            #sorted() makes sure the code goes through the files in alphabetical (chronological) order
            #GOING THROUGH THE FILES IN CHRONOLOGICAL ORDER IS ESSENTIAL FOR THIS CELL TO WORK PROPERLY!!
            for i, x in enumerate(apr_file_list):  
                file_start_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
                file_end_time = datetime.strptime(x[46:54] + x[55:61], '%Y%m%d%H%M%S')
            
                if start_time1 <= file_start_time:  #if start_time is before the APR file start time and not within any previous APR file's time ranges
                    first_file_index = i
                    break
                elif (start_time1 >= file_start_time) and (start_time1 < file_end_time):
                    first_file_index = i
                    break
                else:
                    continue
            if first_file_index == None:
                sys.exit('Requested start_time is beyond all available APR files')
                
            #find the ending APR file in apr_folder
            last_file_index = None    
            
            #sorted() makes sure the code goes through the files in alphabetical (chronological) order
            #GOING THROUGH THE FILES IN CHRONOLOGICAL ORDER IS ESSENTIAL FOR THIS CELL TO WORK PROPERLY!!
            for i, x in enumerate(apr_file_list):  
                file_start_time = datetime.strptime(x[29:37] + x[38:44], '%Y%m%d%H%M%S')
                file_end_time = datetime.strptime(x[46:54] + x[55:61], '%Y%m%d%H%M%S')
            
                if end_time1 <= file_start_time:  #if end_time is before the APR file start time and not within any previous APR file's time ranges
                    last_file_index = i - 1
                    break
                elif (end_time1 > file_start_time) and (end_time1 <= file_end_time):
                    last_file_index = i
                    break
                else:
                    continue
            if last_file_index == None:  #the end_time is after all available APR file ranges
                last_file_index = len(apr_file_list) - 1  #the last available APR file's index
            if last_file_index == -1:
                sys.exit('Requested end_time is before all available APR files')
            
            apr_files_use = apr_file_list[first_file_index:last_file_index + 1]            
        
        else:
            sys.exit('Not a CPEX or CPEX-AW or CPEX-CV case')
    
        #print (apr_files_use)  #sanity check to make sure you grabbed the right files

        for apr_filepath in apr_files_use:
            
            if desired_date[:4] == '2017' or desired_date[:4] == '2021':    #CPEX(-AW)
            
                #Low resolution ('lores') radar variables in APR hdf files
                ku_band = 'zhh14' #Ku-band reflectivity
                vel = 'vel14c' #mean Doppler Velocity dealiased and from Ku&Ka band
            
                apr_file = h5py.File(os.path.join(apr_folder, apr_filepath), 'r')
                
                if ('lores' in apr_file.keys()) and (ku_band in apr_file['lores'].keys()):
                    
                    try:   #some CPEX-AW APR files have corrupted Ku-band data; if so, skip the Ku-band for 
                           #that file (corrupted: "OSError: Can't read data (inflate() failed)")
                        ku_data = apr_file['lores'][ku_band][:]
                    except:
                        apr_file.close()
                        continue  #both Ku-band and velocity CFADs rely on Ku-band data availability
    
                    try:   #some CPEX-AW APR files have corrupted velocity data; if so, skip the Ku-band for 
                           #that file (corrupted: "OSError: Can't read data (inflate() failed)")
                        vel_data = apr_file['lores'][vel][:]
                        vel_good = True
                    except:
                        vel_good = False                    
                
                    #grab the radar variables of interest
                    time = apr_file['lores']['scantime'][:]
                    alt3d = apr_file['lores']['alt3D'][:]
                    roll = apr_file['lores']['roll'][:]
                    
                    if apr_filepath == apr_files_use[0] or apr_filepath == apr_files_use[-1]:
                    #the complete APR file time range may not need to be used, so need to locate the closest time (and corresponding index) to the desired start/end time
                        
                        #Convert APR times to datetimes
                        time_dates = np.empty(time.shape[1], dtype=object)
                        for j in np.arange(0, time.shape[1]):
                            #tmp = datetime(time[12,j])
                            tmp = datetime.utcfromtimestamp(time[12,j])  #12 could be any ray, as it is the ray number and each ray of a given scan has the same time
                            time_dates[j] = tmp
                        
                        unique_apr_times = time_dates  #all the times in the given APR file
                        
                        if len(apr_files_use) == 1:  #i.e. apr_filepath == apr_files_use[0] and apr_filepath == apr_files_use[-1]
                        
                            #find the closest time (and its corresponding index) to the desired start_time
                            desired_start_time = datetime.strptime(desired_date + start_time, '%Y%m%d%H%M%S')
                            start_time_idx = np.argmin(abs(unique_apr_times - desired_start_time))
                        
                            #find the closest time (and its corresponding index) to the desired end_time
                            desired_end_time = datetime.strptime(desired_date + end_time, '%Y%m%d%H%M%S')
                            end_time_idx = np.argmin(abs(unique_apr_times - desired_end_time))
                                    
                        elif apr_filepath == apr_files_use[0]:  #i.e. the first APR file in the apr_files_use list
                            end_time_idx = time.shape[1] - 1
                            
                            #find the closest time (and its corresponding index) to the desired start_time
                            desired_start_time = datetime.strptime(desired_date + start_time, '%Y%m%d%H%M%S')
                            start_time_idx = np.argmin(abs(unique_apr_times - desired_start_time))
                                    
                        else:  #apr_filepath == apr_files_use[-1]  #i.e. the last APR file in the apr_files_use list
                            start_time_idx = 0
                            
                            #find the closest time (and its corresponding index) to the desired end_time
                            desired_end_time = datetime.strptime(desired_date + end_time, '%Y%m%d%H%M%S')
                            end_time_idx = np.argmin(abs(unique_apr_times - desired_end_time))                
                            
                    else:  #the entire APR file is within the desired time range, so set the start/end indices to the first/last indices of the APR file
                        start_time_idx = 0
                        end_time_idx = time.shape[1] - 1
                            
                    #numpy.histogram2d(dBZ, z) (Zagrodnik et al., 2019) CFAD method:

                    #loop through the given APR file’s valid times/scans/profiles
                    for time_idx in range(start_time_idx, end_time_idx + 1):          #time_idx represents the scan number
                    
                        profile_time = datetime.utcfromtimestamp(time[12, time_idx])  #12 could be any ray, as it is the ray number and each ray of a given scan has the same time
                        ecco_ip = np.argmin(abs(ecco_df['Full_Datetime'] - profile_time))  #abs() necessary to properly subtract datetime objects
                                                                                           #returns the integer position of the minimum value
                        #print (ecco_df['Full_Datetime'].iloc[ecco_index])           #to check that the correct time was chosen
                        classification = ecco_df['Classification'].iloc[ecco_ip]     #grab the classification of the ECCO-V time closest to the given profile time (may/should be the exact time)
                        
                        if classification not in ['0', '1', '2']:
                            raise ValueError(f'{classification} is not a valid ECCO-V classification.')
                    
                        # #skip profiles with an ECCO-V classification of "no classification" (0) or "stratiform" (1)
                        # #i.e., only use "convective" (2) profiles
                        # if classification == '0' or classification == '1':
                        #     continue
                        
                        #skip profiles with an ECCO-V classification of "no classification" (0) or "convective" (2)
                        #i.e., only use "stratiform" (1) profiles
                        if classification == '0' or classification == '2':
                            continue
                        
                        total_apr_profiles += 1
                        
                        #choose the "nadir" ray factoring in aircraft roll
                        ac_roll = np.nanmean(roll[:,time_idx])  #roll varies slightly w/ray, so take the average roll value for a given scan and use that for ray adjustment
                        ray_use = np.argmin(np.abs(ray_angles - ac_roll))  #the index of the ray whose angle is closest to that of ac_roll
                        
                        if abs(ac_roll) >= 10:
                            apr_profile_roll10 += 1
                        
                        prof_height = alt3d[:,ray_use,time_idx]
                        prof_dbz = ku_data[:,ray_use,time_idx]
                        
                        #grab the height/dbz data above 1.5km
                        idx_1500m = np.argmin(np.abs(prof_height - 1500))
                        prof_height_above1500m = np.flip(prof_height[:idx_1500m + 1])  #height profile above 1.5km, in ascending order
                        prof_dbz_above1500m = np.flip(prof_dbz[:idx_1500m + 1])  #Ku-band profile above 1.5km, in ascending order
                        
                        #store all the profile's height/dbz data in long 1-D concatenated arrays
                        if new_ku_array:  #if this is the first qualifying time and ray of the first APR file of the first case in case_dict
                            height_concat = prof_height.copy()
                            dbz_concat = prof_dbz.copy()
                            
                            #create a DataFrame of values above 1.5km, in order to calculate meadian/quartile reflectivity profile for the given case
                                #NOTE: WE ARE ASSUMING THAT EACH ROW CORRESPONDS TO A NEAR IDENTICAL HEIGHT LEVEL
                                    #this is the best we can do, and should be fine given that each profile is starting from ~1500m and the height resolution is the same for each profile     
                            height_profiles_above1500m_df = pd.DataFrame(prof_height_above1500m)  #height profile from 1500m to top of profile
                            dbz_profiles_above1500m_df = pd.DataFrame(prof_dbz_above1500m)        #dbz profile from 1500m to top of profile
                            new_ku_array = False
                        else:
                            height_concat = np.concatenate((height_concat, prof_height))
                            dbz_concat = np.concatenate((dbz_concat, prof_dbz))
                            
                            #NOTE: WE ARE ASSUMING THAT EACH ROW CORRESPONDS TO A NEAR IDENTICAL HEIGHT LEVEL
                                #this is the best we can do, and should be fine given that each profile is starting from ~1500m and the height resolution is the same for each profile
                            height_profiles_above1500m_df = pd.concat((height_profiles_above1500m_df, pd.Series(prof_height_above1500m)), axis = 1, ignore_index = True)  #add profile as a new column to the df; differences in profile lengths are filled in with NaNs
                            dbz_profiles_above1500m_df = pd.concat((dbz_profiles_above1500m_df, pd.Series(prof_dbz_above1500m)), axis = 1, ignore_index = True)  #add profile as a new column to the df; differences in profile lengths are filled in with NaNs
                        
                        if vel_good:
                            prof_vel = vel_data[:,ray_use,time_idx]
                            
                            #store all the profile's velocity data in long 1-D concatenated arrays if 
                            #the profile has Ku data > 0 dBZ above 1.5km (i.e. omits clear profiles)
                            
                            if np.nanmax(prof_dbz_above1500m) > 0:
                                if new_vel_array:  #if this is the first qualifying time and ray of the first APR file of the first case in case_dict
                                    vel_concat = prof_vel.copy()
                                    height_vel_concat = prof_height.copy()
                                    new_vel_array = False
                                else:
                                    vel_concat = np.concatenate((vel_concat, prof_vel))
                                    height_vel_concat = np.concatenate((height_vel_concat, prof_height))
                            else:  #clear profiles (which may have noisy velocity data) are omitted from the velocity CFAD
                                pass
                        else:
                            pass
                else:
                    pass
                    
                apr_file.close()
                
            elif desired_date[:4] == '2022':    #CPEX-CV
            
                ku_band = 'lores_zhh14' #Ku-band reflectivity
                vel = 'lores_vel14c' #Mean Doppler Velocity from Ku-band (surface Doppler velocity is subtracted and free of aliasing)
            
                apr_file = xr.open_dataset(os.path.join(apr_folder, apr_filepath))
                
                if ku_band in apr_file.keys():
                    
                    try:   #some APR files, at least in the preliminary data, have corrupted Ku-band data; 
                           #if so, skip plotting the Ku-band for that file 
                           #corrupted: "OSError: Can't read data (inflate() failed)"
                        ku_data = apr_file[ku_band][:]
                    except:
                        first_good_Ku_file_index += 1    #used to determine first usable Ku-band file for CPEX-CV cases below; without this, an error is brought up due to the first APR file in apr_files_use only being W-band data (i.e., _Wn.nc)
                        apr_file.close()
                        continue  #both Ku-band and velocity CFADs rely on Ku-band data availability
    
                    try:   #some APR files, at least in the preliminary data, have corrupted velocity data; 
                           #if so, skip plotting the Ku-band for that file 
                           #corrupted: "OSError: Can't read data (inflate() failed)"
                        vel_data = apr_file[vel][:]
                        vel_good = True
                    except:
                        vel_good = False                    
                
                    #grab the radar variables of interest
                    time = apr_file['time'][:]          #For 'lores': Time of scan, in seconds since midnight UTC of [YYYY-mm-DD]
                    alt3d = apr_file['lores_alt3D'][:]
                    roll = apr_file['lores_roll'][:]
                    
                    if apr_filepath == apr_files_use[first_good_Ku_file_index] or apr_filepath == apr_files_use[-1]:
                    #the complete APR file time range may not need to be used, so need to locate the closest time (and corresponding index) to the desired start/end time
                        
                        #Convert APR times to datetimes
                        time_dates = np.empty(time.shape, dtype=object)
                        for i in np.arange(0, time.shape[0]):
                            #hour, second automatically revert to midnight (hour = 0, seconds = 0) for '%Y%m%d'
                            time_dates[i] = datetime.strptime(desired_date, '%Y%m%d') + timedelta(seconds = float(time[i].values))
                        
                        unique_apr_times = time_dates[:]  #all the times in the given APR file
                        
                        if len(apr_files_use) == 1:  #i.e. apr_filepath == apr_files_use[first_good_Ku_file_index] and apr_filepath == apr_files_use[-1]
                        
                            #find the closest time (and its corresponding index) to the desired start_time
                            desired_start_time = datetime.strptime(desired_date + start_time, '%Y%m%d%H%M%S')
                            start_time_idx = np.argmin(abs(unique_apr_times - desired_start_time))
                        
                            #find the closest time (and its corresponding index) to the desired end_time
                            desired_end_time = datetime.strptime(desired_date + end_time, '%Y%m%d%H%M%S')
                            end_time_idx = np.argmin(abs(unique_apr_times - desired_end_time))
                                    
                        elif apr_filepath == apr_files_use[first_good_Ku_file_index]:  #i.e. the first APR file in the apr_files_use list
                            end_time_idx = time.shape[0] - 1
                            
                            #find the closest time (and its corresponding index) to the desired start_time
                            desired_start_time = datetime.strptime(desired_date + start_time, '%Y%m%d%H%M%S')
                            start_time_idx = np.argmin(abs(unique_apr_times - desired_start_time))
                                    
                        else:  #apr_filepath == apr_files_use[-1]  #i.e. the last APR file in the apr_files_use list
                            start_time_idx = 0
                            
                            #find the closest time (and its corresponding index) to the desired end_time
                            desired_end_time = datetime.strptime(desired_date + end_time, '%Y%m%d%H%M%S')
                            end_time_idx = np.argmin(abs(unique_apr_times - desired_end_time))                
                            
                    else:  #the entire APR file is within the desired time range, so set the start/end indices to the first/last indices of the APR file
                        start_time_idx = 0
                        end_time_idx = time.shape[0] - 1
                            
                    #numpy.histogram2d(dBZ, z) (Zagrodnik et al., 2019) CFAD method:
                    
########################################################################################################################### 
###########################################################################################################################
###########################################################################################################################
###########################################################################################################################                       
                    
                    #loop through the given APR file’s valid times/scans/profiles
                    for time_idx in range(start_time_idx, end_time_idx + 1):          #time_idx represents the scan number
                        
                        profile_time = datetime.strptime(desired_date, '%Y%m%d') + timedelta(seconds = float(time[time_idx].values))
                        ecco_ip = np.argmin(abs(ecco_df['Full_Datetime'] - profile_time))  #abs() necessary to properly subtract datetime objects
                                                                                           #returns the integer position of the minimum value
                        #print (ecco_df['Full_Datetime'].iloc[ecco_index])           #to check that the correct time was chosen
                        classification = ecco_df['Classification'].iloc[ecco_ip]     #grab the classification of the ECCO-V time closest to the given profile time (may/should be the exact time)
                        
                        if classification not in ['0', '1', '2']:
                            raise ValueError(f'{classification} is not a valid ECCO-V classification.')
                    
                        # #skip profiles with an ECCO-V classification of "no classification" (0) or "stratiform" (1)
                        # #(i.e., only use "convective" (2) profiles)
                        # if classification == '0' or classification == '1':
                        #     continue
                        
                        #skip profiles with an ECCO-V classification of "no classification" (0) or "convective" (2)
                        #(i.e., only use "stratiform" (1) profiles)
                        if classification == '0' or classification == '2':
                            continue
                        
                        total_apr_profiles += 1
                        
                        #choose the "nadir" ray factoring in aircraft roll
                        ac_roll = np.nanmean(roll[time_idx,:])  #roll varies slightly w/ray, so take the average roll value for a given scan and use that for ray adjustment
                        ray_use = np.argmin(np.abs(ray_angles - ac_roll))  #the index of the ray whose angle is closest to that of ac_roll; i.e., the "pseudo-nadir" ray
                        
                        if abs(ac_roll) >= 10:
                            apr_profile_roll10 += 1
                        
                        prof_height = alt3d[time_idx,ray_use,:].values
                        prof_dbz = ku_data[time_idx,ray_use,:].values
                        
                        #grab the height/dbz data above 1.5km
                        idx_1500m = np.argmin(np.abs(prof_height - 1500))
                        prof_height_above1500m = np.flip(prof_height[:idx_1500m + 1])  #height profile above 1.5km, in ascending order
                        prof_dbz_above1500m = np.flip(prof_dbz[:idx_1500m + 1])  #Ku-band profile above 1.5km, in ascending order
                        
                        #store all the profile's height/dbz data in long 1-D concatenated arrays
                        if new_ku_array:  #if this is the first qualifying time and ray of the first APR file of the first case in case_dict
                            height_concat = prof_height.copy()
                            dbz_concat = prof_dbz.copy()
                            
                            #create a DataFrame of values above 1.5km, in order to calculate median/quartile reflectivity profile for the given case
                                #NOTE: WE ARE ASSUMING THAT EACH ROW CORRESPONDS TO A NEAR IDENTICAL HEIGHT LEVEL
                                    #this is the best we can do, and should be fine given that each profile is starting from ~1500m and the height resolution is the same for each profile     
                            height_profiles_above1500m_df = pd.DataFrame(prof_height_above1500m)  #height profile from 1500m to top of profile
                            dbz_profiles_above1500m_df = pd.DataFrame(prof_dbz_above1500m)        #dbz profile from 1500m to top of profile
                            new_ku_array = False
                        else:
                            height_concat = np.concatenate((height_concat, prof_height))
                            dbz_concat = np.concatenate((dbz_concat, prof_dbz))
                            
                            #NOTE: WE ARE ASSUMING THAT EACH ROW CORRESPONDS TO A NEAR IDENTICAL HEIGHT LEVEL
                                #this is the best we can do, and should be fine given that each profile is starting from ~1500m and the height resolution is the same for each profile
                            height_profiles_above1500m_df = pd.concat((height_profiles_above1500m_df, pd.Series(prof_height_above1500m)), axis = 1, ignore_index = True)  #add profile as a new column to the df; differences in profile lengths are filled in with NaNs
                            dbz_profiles_above1500m_df = pd.concat((dbz_profiles_above1500m_df, pd.Series(prof_dbz_above1500m)), axis = 1, ignore_index = True)  #add profile as a new column to the df; differences in profile lengths are filled in with NaNs
                        
                        if vel_good:
                            prof_vel = vel_data[time_idx,ray_use,:].values
                            
                            #store all the profile's velocity data in long 1-D concatenated arrays if 
                            #the profile has Ku data > 0 dBZ above 1.5km (i.e. omits clear profiles)
                            
                            if np.nanmax(prof_dbz_above1500m) > 0:
                                if new_vel_array:  #if this is the first qualifying time and ray of the first APR file of the first case in case_dict
                                    vel_concat = prof_vel.copy()
                                    height_vel_concat = prof_height.copy()
                                    new_vel_array = False
                                else:
                                    vel_concat = np.concatenate((vel_concat, prof_vel))
                                    height_vel_concat = np.concatenate((height_vel_concat, prof_height))
                            else:  #clear profiles (which may have noisy velocity data) are omitted from the velocity CFAD
                                pass
                        else:
                            pass
                            
########################################################################################################################### 
###########################################################################################################################
###########################################################################################################################
###########################################################################################################################                            
                            
                else:
                    first_good_Ku_file_index += 1    #used to determine first usable Ku-band file for CPEX-CV cases below; without this, an error is brought up due to the first APR file in apr_files_use only being W-band data (i.e., _Wn.nc)
                    
                apr_file.close()    
                
            else:
                sys.exit('Not a CPEX or CPEX-AW or CPEX-CV case')
                
        print ('Case {} complete'.format(key))
        
    #calculate median/quantile dBZ profiles for the given case
    
    #replace blank data (-99.99) with NaNs
    dbz_profiles_above1500m_df[dbz_profiles_above1500m_df <= -99] = np.nan
    height_profiles_above1500m_df[height_profiles_above1500m_df < 0] = np.nan
    
    #convert all the dropsonde's Ku-band data from dBZ to mm^6 m^-3
    Z_df = 10**(dbz_profiles_above1500m_df / 10)
    
    #calculate the dropsonde's median/quantile heights and reflectivity (in mm^6 m^-3) and standard deviation at each height level
    #NOTE: WE ARE ASSUMING THAT EACH ROW CORRESPONDS TO A NEAR IDENTICAL HEIGHT LEVEL
        #this is the best we can do, and should be fine given that each profile is starting from ~1500m and the height resolution is the same for each profile
        
    median_height_profile_above1500m = height_profiles_above1500m_df.median(axis = 1)  #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    Z_median = Z_df.median(axis = 1)  #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    Z_Q1 = Z_df.quantile(q = 0.25, axis = 1)
    Z_Q3 = Z_df.quantile(q = 0.75, axis = 1)
    #Z_std = Z_df.std(axis = 1)
    
    #convert the dropsonde's median/quartile reflectivity and standard deviation profile back to dBZ
    medianDBZ_profile_above1500m = 10 * np.log10(Z_median)
    Q1DBZ_profile_above1500m = 10 * np.log10(Z_Q1)
    Q3DBZ_profile_above1500m = 10 * np.log10(Z_Q3)
    #stdDBZ_profile_above1500m = 10 * np.log10(Z_std)      
        
    
    #setting the height, reflectivity, and velocity bin edges, along with their associated meshgrids
    height_edges = height_bin_edges
    dbz_edges = dbz_bin_edges
    #dbz_meshgrid, height_meshgrid = np.meshgrid(dbz_edges, height_edges)
    
    vel_edges = vel_bin_edges
    #vel_meshgrid, height_vel_meshgrid = np.meshgrid(vel_edges, height_edges)
    
    #setting the height, reflectivity, and velocity bin centers and their associated meshgrids for contourf plotting
    height_centers = (height_edges[:-1] + height_edges[1:]) / 2    
    dbz_centers = (dbz_edges[:-1] + dbz_edges[1:]) / 2
    dbz_centers_meshgrid, height_centers_meshgrid = np.meshgrid(dbz_centers, height_centers)
    
    vel_centers = (vel_edges[:-1] + vel_edges[1:]) / 2
    vel_centers_meshgrid, height_vel_centers_meshgrid = np.meshgrid(vel_centers, height_centers)
  
    #create the 2D histogram of values/frequencies for Ku-band data
    cfad_array, xedges, yedges = np.histogram2d(dbz_concat, height_concat, bins = (dbz_edges,height_edges))
    
    #transpose cfad_array shape (rows, columns) to be (height, dbz) instead of (dbz, height)
    cfad_array = cfad_array.T
    #cfad_array = np.log10(cfad_array)  #creates log-weighted CFADs
    contours = np.linspace(0, np.nanmax(cfad_array), 21)
    colorbar_label = 'Total Frequency [#]'
    var_label = 'Ku-band Reflectivity [dBZ]'
    save_label = 'CFAD_Cases' + '-'.join(map(str, list(case_dict.keys()))) + '_Ku.png'
    plot_CFAD(cfad_array, contours, colorbar_label, save_label, var_label, dbz_centers_meshgrid, height_centers_meshgrid, dbz_edges, case_dict, median_height_profile_above1500m, medianDBZ_profile_above1500m, Q1DBZ_profile_above1500m, Q3DBZ_profile_above1500m, dbz_plot = True)
    
    #normalize the CFAD and make a normalized CFAD plot if that is also desired
    if normalize:
        #cfad_array = cfad_array / np.nanmax(cfad_array) * 100
        height_bin_maxs = np.nanmax(cfad_array, axis = 1)   #normalize the CFAD by max count in each height bin
        cfad_array = cfad_array / height_bin_maxs[:, np.newaxis] * 100
        contours = np.arange(0,101,5)
        colorbar_label = 'Normalized Frequency [%]'
        save_label = 'CFADnorm_Cases' + '-'.join(map(str, list(case_dict.keys()))) + '_Ku.png'
        plot_CFAD(cfad_array, contours, colorbar_label, save_label, var_label, dbz_centers_meshgrid, height_centers_meshgrid, dbz_edges, case_dict, median_height_profile_above1500m, medianDBZ_profile_above1500m, Q1DBZ_profile_above1500m, Q3DBZ_profile_above1500m, dbz_plot = True)
        
        
    #create the 2D histogram of values/frequencies for Doppler Velocity data
    cfad_array_vel, xedges, yedges = np.histogram2d(vel_concat, height_vel_concat, bins = (vel_edges,height_edges))
    
    #transpose cfad_array shape (rows, columns) to be (height, vel) instead of (vel, height)
    cfad_array_vel = cfad_array_vel.T
    #cfad_array_vel = np.log10(cfad_array_vel)  #creates log-weighted CFADs
    contours = np.linspace(0, np.nanmax(cfad_array_vel), 21)
    colorbar_label = 'Total Frequency [#]'
    var_label = 'Mean Doppler Velocity [m/s]'
    save_label = 'CFAD_Cases' + '-'.join(map(str, list(case_dict.keys()))) + '_Vel.png'
    plot_CFAD(cfad_array_vel, contours, colorbar_label, save_label, var_label, vel_centers_meshgrid, height_vel_centers_meshgrid, vel_edges, case_dict, median_height_profile_above1500m, medianDBZ_profile_above1500m, Q1DBZ_profile_above1500m, Q3DBZ_profile_above1500m, dbz_plot = False)
    
    #normalize the CFAD and make a normalized CFAD plot if that is also desired
    if normalize:
        #cfad_array_vel = cfad_array_vel / np.nanmax(cfad_array_vel) * 100
        height_bin_maxs = np.nanmax(cfad_array_vel, axis = 1)   #normalize the CFAD by max count in each height bin
        cfad_array_vel = cfad_array_vel / height_bin_maxs[:, np.newaxis] * 100
        contours = np.arange(0,101,5)
        colorbar_label = 'Normalized Frequency [%]'
        save_label = 'CFADnorm_Cases' + '-'.join(map(str, list(case_dict.keys()))) + '_Vel.png'
        plot_CFAD(cfad_array_vel, contours, colorbar_label, save_label, var_label, vel_centers_meshgrid, height_vel_centers_meshgrid, vel_edges, case_dict, median_height_profile_above1500m, medianDBZ_profile_above1500m, Q1DBZ_profile_above1500m, Q3DBZ_profile_above1500m, dbz_plot = False)
        
    #print ('Percent of profiles with A/C roll >= 10 degrees:', apr_profile_roll10 / total_apr_profiles * 100)
    
    return cfad_array, cfad_array_vel, dbz_centers_meshgrid, height_centers_meshgrid, vel_centers_meshgrid, height_vel_centers_meshgrid, median_height_profile_above1500m, medianDBZ_profile_above1500m, Q1DBZ_profile_above1500m, Q3DBZ_profile_above1500m
    

def plot_diffCFAD(cfad_array, contours, colorbar_label, save_label, var_label, var_centers_meshgrid, height_centers_meshgrid, var_edges, height_edges, case1_num, case2_num, first_median_height_profile_above1500m, first_medianDBZ_profile_above1500m, first_Q1DBZ_profile_above1500m, first_Q3DBZ_profile_above1500m, second_median_height_profile_above1500m, second_medianDBZ_profile_above1500m, second_Q1DBZ_profile_above1500m, second_Q3DBZ_profile_above1500m, dbz_plot = True):
    
    """Plot a contourf difference CFAD given a difference CFAD 2-D array, contour levels, plot/image labels, and variable/height meshgrids"""
    
    #plot the CFAD
    fig, ax = plt.subplots(1,1, figsize=(21,21))
    # cmap = mplc.ListedColormap(['#00429d', '#2855a6', '#3e68af', '#507bb8', '#618fc1', '#73a3ca', '#85b8d3', 
    #                             '#99ccdc', '#b0e0e6', '#cef2f1', '#ffffff', '#ffe5e7', '#ffcbcf', '#ffafb7', 
    #                             '#ff929e', '#f57789', '#e95d76', '#d94364', '#c62a54', '#af1046', '#93003a'])
    
    cmap = mplc.ListedColormap(['#00429d', '#074ca2', '#0e57a8', '#1561ad', '#1d6bb2', '#2475b8', '#2b7fbd', 
                                '#3289c2', '#3994c8', '#409ecd', '#48a8d2', '#4fb3d8', '#56bddd', '#5ec8e3', 
                                '#65d3e8', '#6dddee', '#74e8f4', '#7cf3f9', '#83feff', '#cbffff', '#ffffff', 
                                '#fef3f0', '#fce7e0', '#fbdad0', '#facec1', '#f8c2b1', '#f7b5a1', '#f5a890', 
                                '#f49b7f', '#f28d6e', '#f17f5c', '#ef6f48', '#ed5f33', '#eb4c1b', '#e13f18', 
                                '#d5351e', '#c82b23', '#bc2128', '#af162e', '#a10b34', '#93003a'])
    
    #cs = ax.pcolormesh(dbz_meshgrid, height_meshgrid, cfad_array, cmap = cmap)
    #cs = ax.contour(dbz_centers_meshgrid, height_centers_meshgrid, cfad_array, levels = contours, cmap = cmap, linewidths = 1)
    cs = ax.contourf(var_centers_meshgrid, height_centers_meshgrid, cfad_array, levels = contours, cmap = cmap)
    if dbz_plot:
        ax.plot(second_medianDBZ_profile_above1500m, second_median_height_profile_above1500m, color = 'k', linestyle = '-', linewidth = 5, label = 'Case ' + case2_num + ' Median Profile') 
        ax.plot(second_Q1DBZ_profile_above1500m, second_median_height_profile_above1500m, color = 'k', linestyle = '--', linewidth = 5, label = 'Case ' + case2_num + ' Q1 and Q3 Profiles')
        ax.plot(second_Q3DBZ_profile_above1500m, second_median_height_profile_above1500m, color = 'k', linestyle = '--', linewidth = 5)
        
        ax.plot(first_medianDBZ_profile_above1500m, first_median_height_profile_above1500m, color = 'darkgoldenrod', linestyle = '-', linewidth = 5, label = 'Case ' + case1_num + ' Median Profile') 
        ax.plot(first_Q1DBZ_profile_above1500m, first_median_height_profile_above1500m, color = 'darkgoldenrod', linestyle = '--', linewidth = 5, label = 'Case ' + case1_num + ' Q1 and Q3 Profiles')
        ax.plot(first_Q3DBZ_profile_above1500m, first_median_height_profile_above1500m, color = 'darkgoldenrod', linestyle = '--', linewidth = 5)
        
        ax.legend(loc = 'upper right')
        
    ax.set_ylabel('Altitude [m]', fontsize=30, fontweight = 'bold')
    ax.set_xlabel(var_label, fontsize=30, fontweight = 'bold')
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    #ax.set_title('Difference CFAD (Case ' + case2_num + ' minus ' + 'Case ' + case1_num + ')', fontsize=35, fontweight = 'bold')
    ax.set_title('b) Difference CFAD (Case 12 minus Case 10, Only Stratiform Regions)', fontsize=34.5, fontweight = 'bold')
    ax.set_ylim([height_edges[0] + 250, height_edges[-1] - 250])
    #ax.set_xlim([var_edges[0],var_edges[-1]])
    ax.set_xlim([5,55])  #for Ku-band
    
    #set the colorbar axis
    cax = fig.add_axes([ax.get_position().x0, ax.get_position().y0 - 0.08,
                       ax.get_position().x1-ax.get_position().x0, 0.02])    #Left, bottom, width, height (all [0,1])

    #create the colorbar
    cbar = plt.colorbar(cs, cax = cax, orientation = 'horizontal')
    cbar.ax.tick_params(length = 10, width = 3, labelsize = 23)
    cbar.set_label(label = colorbar_label, fontsize = 30, fontweight = 'bold')
    
    #save the figure
    #plt.savefig(''.join(['/Users/brodenkirch/Desktop/CPEX-CV/Coding/CFAD_plots/Case', case1_num, 'and', case2_num, '_Comparison/', save_label]), bbox_inches = 'tight')
    plt.savefig(''.join(['/Users/ben/Desktop/', save_label]), bbox_inches = 'tight')    
    plt.close()


def difference_CFAD(case_dict_1, case_dict_2, height_bin_edges, dbz_bin_edges, vel_bin_edges):

    """"Calculate and plot Ku-band and Doppler Velocity difference (normalized) CFAD 2-D arrays for 
        the given pair of dictionaries containing given cases and their respective time ranges
    
    Parameters
    ----------
    case_dict_1:  dictionary of cases for which to calculate the first normalized CFAD; 
                  keys should be case numbers;
                  values should be 3-element lists of case date and start/end times, 
                  with date strings formatted as YYYYMMDD and time strings formatted as HHMMSS
                  
    case_dict_2:  dictionary of cases for which to calculate the second normalized CFAD; 
                  keys should be case numbers;
                  values should be 3-element lists of case date and start/end times, 
                  with date strings formatted as YYYYMMDD and time strings formatted as HHMMSS
    
    height_bin_edges:  a 1-D array of height [m] bin edges used to create the 2-D difference CFAD array/histogram    

    dbz_bin_edges:  a 1-D array of reflectivity [dBZ] bin edges used to create the 2-D difference CFAD array/histogram 

    vel_bin_edges:  a 1-D array of Doppler velocity [m/s] bin edges used to create the 2-D difference CFAD array/histogram
                        
    return:  2-D (normalized) difference CFAD plots (case_dict_2 - case_dict_1)"""

    #calculate the Ku-band and Doppler Velocity 2-D normalized CFADs for each case
    first_dbz_CFAD, first_vel_CFAD, dbz_centers_meshgrid, height_centers_meshgrid, vel_centers_meshgrid, height_vel_centers_meshgrid, first_median_height_profile_above1500m, first_medianDBZ_profile_above1500m, first_Q1DBZ_profile_above1500m, first_Q3DBZ_profile_above1500m = CFAD(case_dict_1, height_bin_edges, dbz_bin_edges, vel_bin_edges, normalize = True)
    second_dbz_CFAD, second_vel_CFAD, dbz_centers_meshgrid, height_centers_meshgrid, vel_centers_meshgrid, height_vel_centers_meshgrid, second_median_height_profile_above1500m, second_medianDBZ_profile_above1500m, second_Q1DBZ_profile_above1500m, second_Q3DBZ_profile_above1500m = CFAD(case_dict_2, height_bin_edges, dbz_bin_edges, vel_bin_edges, normalize = True)
    
    #calculate the Ku-band and Doppler Velocity 2-D difference CFADs
    dbz_diff_CFAD = second_dbz_CFAD - first_dbz_CFAD
    vel_diff_CFAD = second_vel_CFAD - first_vel_CFAD
    
    #calculate the maximum magnitude for each CFAD and create contours for diverging colormaps accordingly
    
    #dbz_highest_mag = np.nanmax(np.abs(dbz_diff_CFAD))  #creates unique colorbar range for each difference CFAD plot
    dbz_highest_mag = 100  #creates uniform colorbar range across all difference CFAD plots
    dbz_contours = np.linspace(-dbz_highest_mag, dbz_highest_mag, 41)  #an odd number of intervals (21) guarantees a contour at 0 for a range from -x to x
    
    #vel_highest_mag = np.nanmax(np.abs(vel_diff_CFAD))  #creates unique colorbar range for each difference CFAD plot
    vel_highest_mag = 100  #creates uniform colorbar range across all difference CFAD plots
    vel_contours = np.linspace(-vel_highest_mag, vel_highest_mag, 41)  #an odd number of intervals (21) guarantees a contour at 0 for a range from -x to x
    
    #grab the case numbers of the 2 cases/case sets being differenced and create appropriate image names
    case1_num = ','.join(map(str, list(case_dict_1.keys())))
    case2_num = ','.join(map(str, list(case_dict_2.keys())))
    cases = case1_num + 'and' + case2_num
    colorbar_label = 'Differential Normalized Frequency [%]'
    dbz_save_label = 'diff_CFADnorm_Cases_' + cases + '_Ku.png'
    vel_save_label = 'diff_CFADnorm_Cases_' + cases + '_Vel.png'

    #plot the Ku-band and Doppler Velocity 2-D difference CFADs
    plot_diffCFAD(dbz_diff_CFAD, dbz_contours, colorbar_label, dbz_save_label, 'Ku-band Reflectivity [dBZ]', dbz_centers_meshgrid, height_centers_meshgrid, dbz_bin_edges, height_bin_edges, case1_num, case2_num, first_median_height_profile_above1500m, first_medianDBZ_profile_above1500m, first_Q1DBZ_profile_above1500m, first_Q3DBZ_profile_above1500m, second_median_height_profile_above1500m, second_medianDBZ_profile_above1500m, second_Q1DBZ_profile_above1500m, second_Q3DBZ_profile_above1500m, dbz_plot = False) 
    plot_diffCFAD(vel_diff_CFAD, vel_contours, colorbar_label, vel_save_label, 'Mean Doppler Velocity [m/s]', vel_centers_meshgrid, height_vel_centers_meshgrid, vel_bin_edges, height_bin_edges, case1_num, case2_num, first_median_height_profile_above1500m, first_medianDBZ_profile_above1500m, first_Q1DBZ_profile_above1500m, first_Q3DBZ_profile_above1500m, second_median_height_profile_above1500m, second_medianDBZ_profile_above1500m, second_Q1DBZ_profile_above1500m, second_Q3DBZ_profile_above1500m, dbz_plot = False)
  

#run the CFAD function to plot the CFADs

#Note: CPEX-CV Case 4 does not have any stratiform nor convective data, as identified by the ECCO-V algorithm
#Note: CPEX-CV Cases 5, 7, and 8 do not have any convective data, as identified by the ECCO-V algorithm


#Isolated Difference CFADs
# difference_CFAD(case1_dict, case2_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case1_dict, case3_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case1_dict, case4_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case1_dict, case5_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case1_dict, case6_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case1_dict, case7_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case2_dict, case3_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case2_dict, case4_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case2_dict, case5_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case2_dict, case6_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case2_dict, case7_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case3_dict, case4_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case3_dict, case5_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case3_dict, case6_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case3_dict, case7_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case4_dict, case5_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case4_dict, case6_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case4_dict, case7_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case5_dict, case6_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case5_dict, case7_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case6_dict, case7_dict, height_edges, dbz_edges, vel_edges)

#Organized Difference CFADs
# difference_CFAD(case8_dict, case9_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case10_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case11_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case12_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case13_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case14_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case15_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case16_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case17_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case8_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case9_dict, case10_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case11_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case12_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case13_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case14_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case15_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case16_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case17_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case9_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case10_dict, case11_dict, height_edges, dbz_edges, vel_edges)
difference_CFAD(case10_dict, case12_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case10_dict, case13_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case10_dict, case14_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case10_dict, case15_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case10_dict, case16_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case10_dict, case17_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case10_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case10_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case10_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case10_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case11_dict, case12_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case11_dict, case13_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case11_dict, case14_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case11_dict, case15_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case11_dict, case16_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case11_dict, case17_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case11_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case11_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case11_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case11_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case12_dict, case13_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case12_dict, case14_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case12_dict, case15_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case12_dict, case16_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case12_dict, case17_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case12_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case12_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case12_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case12_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case13_dict, case14_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case13_dict, case15_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case13_dict, case16_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case13_dict, case17_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case13_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case13_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case13_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case13_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case14_dict, case15_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case14_dict, case16_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case14_dict, case17_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case14_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case14_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case14_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case14_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case15_dict, case16_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case15_dict, case17_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case15_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case15_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case15_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case15_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case16_dict, case17_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case16_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case16_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case16_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case16_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case17_dict, case18_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case17_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case17_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case17_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case18_dict, case20_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case18_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case18_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case20_dict, case21_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case20_dict, case22_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case21_dict, case22_dict, height_edges, dbz_edges, vel_edges)

#Scattered Difference CFADs
# difference_CFAD(case23_dict, case24_dict, height_edges, dbz_edges, vel_edges)

#Scattered vs. Isolated Difference CFADs
# difference_CFAD(case1_dict, case23_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case2_dict, case23_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case3_dict, case23_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case4_dict, case23_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case5_dict, case23_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case6_dict, case23_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case7_dict, case23_dict, height_edges, dbz_edges, vel_edges)

# difference_CFAD(case1_dict, case24_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case2_dict, case24_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case3_dict, case24_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case4_dict, case24_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case5_dict, case24_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case6_dict, case24_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case7_dict, case24_dict, height_edges, dbz_edges, vel_edges)

# # CFADs for CPEX(-AW) AGU Paper
# difference_CFAD(case2_dict, case1_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case7_dict, case8_dict, height_edges, dbz_edges, vel_edges)
# difference_CFAD(case13_dict, case16_dict, height_edges, dbz_edges, vel_edges)


### For CPEX-CV AGU Paper (Figure 10)

In [ ]:
#adapted from ERA5_Analysis.ipynb
#Case 10 and 12 GPM IMERG, MIMIC TPW, and ERA5 precip. for top 4 panels, then smoothed TIMPS lifecycle time series for bottom panel

#plot time-mean total precipitation rate with flight track and dropsondes overlaid for each desired flight & time ranges in case_dict_stream

#list of desired flight datetimes to plot IMERG, ERA5 precip, and MIMIC TPW for (format: YYYYMMDDHH)
case_dict_stream = ['2022090617', '2022090717', '2022090621', '2022090721']

#set some baseline plot displays

#matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
matplotlib.rcParams['axes.labelsize'] = 32
matplotlib.rcParams['axes.titlesize'] = 36
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['xtick.labelsize'] = 30
matplotlib.rcParams['ytick.labelsize'] = 30
matplotlib.rcParams['legend.fontsize'] = 30
#matplotlib.rcParams['legend.facecolor'] = 'w'
#matplotlib.rcParams['axes.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'
matplotlib.rcParams['hatch.linewidth'] = 0.3

#group_fig = plt.figure(figsize = (24,24))   #initialize the figure
group_fig = plt.figure(figsize = (24,36))   #initialize the figure
data_proj = ccrs.PlateCarree()

#define a GridSpec with 3 rows and 2 columns
gs = gridspec.GridSpec(3, 2, figure = group_fig)

#file_date is the date and hour on which the desired flight took place
for ii, file_date in enumerate(case_dict_stream):

    print (file_date + ' precip. rate plot in progress...')

    #ax = group_fig.add_subplot(2,2,ii+1, projection = data_proj)
    if ii == 0:
        ax = group_fig.add_subplot(gs[0, 0], projection = data_proj)
        letter_label = 'a)'
    elif ii == 1:
        ax = group_fig.add_subplot(gs[0, 1], projection = data_proj)
        letter_label = 'b)'
    elif ii == 2:
        ax = group_fig.add_subplot(gs[1, 0], projection = data_proj)
        letter_label = 'c)'
    elif ii == 3:
        ax = group_fig.add_subplot(gs[1, 1], projection = data_proj)
        letter_label = 'd)'
    
    ###get locations of the dropsonde/Navigation/ERA5 folder and read the appropriate files in
    day_folder = os.path.join(os.getcwd(), file_date[:-2])

    #dropsonde data
    drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + file_date[:-2] + '.csv')
    drop_csv = pd.read_csv(drop_csv_path)

    if file_date[:4] == '2017':
        campaign = 'CPEX'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
    elif file_date[:4] == '2021':
        campaign = 'CPEXAW'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
    elif file_date[:4] == '2022':
        campaign = 'CPEXCV'
        drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
    else:
        pass

    #ERA5 data
    era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
    era5_path = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Pressure.nc')
    ds = xr.open_dataset(era5_path)

    era5_path_precip = os.path.join(era5_folder, campaign + '_ERA5_Reanalysis_Hourly_Precip_Avg.nc')
    ds_precip = xr.open_dataset(era5_path_precip)

    #Navigation data
    nav_folder = os.path.join(day_folder, 'Nav_files')

    for x in os.listdir(nav_folder):
        if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
            os.remove(os.path.join(nav_folder, x))

    nav_ict_path = os.path.join(nav_folder, os.listdir(nav_folder)[0])  #only one nav file per flight

    if campaign != 'CPEX':  #campaign either CPEXAW or CPEXCV

        nav_ict = icartt.Dataset(nav_ict_path)    #open the ict file with icartt
        flight_lat = nav_ict.data["Latitude"]     #nav latitude, just a normal 1-D array
        flight_lon = nav_ict.data["Longitude"]    #nav longitude, just a normal 1-D array

    else:  #for CPEX navigation files, open the CSV file with pandas
           #(Navigation files for CPEX (2017) are originally .kmz not .ict,
           #so I converted them to CSV for free using https://www.gpsvisualizer.com/convert_input

        nav_ict = pd.read_csv(nav_ict_path)       #open the ict file with pandas instead
        flight_lat = nav_ict["latitude"].values   #nav latitude, just a normal 1-D array
        flight_lon = nav_ict["longitude"].values  #nav longitude, just a normal 1-D array    

    #flight track lat/lon extent [West,East,South,North] for plotting, giving an XX degree buffer around the flight track   
    #campaign_extent = [flight_lon.min() - 2.5, flight_lon.max() + 2.5, flight_lat.min() - 2.5, flight_lat.max() + 2.5]
    campaign_extent = [-42, -22, 4.5, 24.5]


    ###calculate each near-storm dropsonde's mean lat/lon and add the sonde's time and mean lat/lon to a list to be plotted

    drop_coords_and_time = []   #format: longitude, latitude, time (HHSS)

    df_drop = pd.read_csv(drop_metric_filepath)
    df_drop_use = df_drop[df_drop['Date'] == int(file_date[:-2])].copy()

    for x in range(len(df_drop_use)):
        date = str(df_drop_use['Date'].iloc[x])
        time = str(df_drop_use['Time'].iloc[x]).zfill(6)
        sonde_datetime = date[:4] + '-' + date[4:6] + '-' + date[6:] + ' ' + time[:2] + ':' + time[2:4] + ':' + time[4:]

        drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
        drop_mean_lon = drop_csv_use['Longitude [deg]'].mean()
        drop_mean_lat = drop_csv_use['Latitude [deg]'].mean()

        sonde_info = [drop_mean_lon, drop_mean_lat, time[:4]]
        drop_coords_and_time.append(sonde_info)

        
    ###create an XX-panel plot of streamlines at XX-different pressure levels 
        ###for each desired hour of the given day

    #half_pres_levs = math.ceil(len(pressures_to_plot_stream) / 2)   #to determine size of figure and number of subplots to generate (rounded up to the nearest multiple of 2)

    hr = int(file_date[-2:])
    hr2 = str(hr).zfill(2)
    hour_prior = str(hr - 1).zfill(2)

    #MIMIC TPW data
       ##https://bin.ssec.wisc.edu/pub/mtpw2/data/
    tpw_folder = os.path.join(day_folder, 'MIMIC_TPW_files')
    tpw_path = os.path.join(tpw_folder, 'comp' + file_date[:-2] + '.' + hr2 + '0000.nc')
    ds_tpw = xr.open_dataset(tpw_path)

    #GPM IMERG data (see IMERG.ipynb for more how to more generally download and plot IMERG data)
       ##https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Read%20IMERG%20Data%20Using%20Python
       ##https://disc.gsfc.nasa.gov/datasets?keywords=imerg&page=1
       #0.1 x 0.1 gridded data, half-hourly means, using the half hour BEFORE the desired hour
    imerg_folder = os.path.join(day_folder, 'IMERG_files')
    
    for x in os.listdir(imerg_folder):
        if x[0:3] == '.DS':         #delete hidden .DS_Store files if they come up (will show up if you delete a file)
            os.remove(os.path.join(imerg_folder, x))
            
        #minutes and seconds automatically revert to zero (hour = 0, seconds = 0) for '%Y%m%d%H'
        elif (datetime.strftime(datetime.strptime(file_date[:-2] + hr2, '%Y%m%d%H') - timedelta(seconds = 1), '%H%M%S') in x) and (datetime.strftime(datetime.strptime(file_date[:-2] + hr2, '%Y%m%d%H') - timedelta(minutes = 30), '%H%M%S') in x):
            imerg_file = x
            break
        else:
            imerg_file = 'Could not find the desired IMERG file'
    
    #confirm that the IMERG file is from the correct day (if hr2 == '00', then this will be the previous day)
    assert (file_date[:-2] in imerg_file) or (hr2 == '00'), 'IMERG file not from the correct day'
    
    imerg_path = os.path.join(imerg_folder, imerg_file)
    ds_imerg = h5py.File(imerg_path, 'r')
    
    imerg_lons = ds_imerg['Grid/lon'][:]   #Longitude Shape: (3600,)
    imerg_lats = ds_imerg['Grid/lat'][:]   #Latitude Shape: (1800,)
    imerg_lons, imerg_lats = np.meshgrid(imerg_lons, imerg_lats)  #Long and lat grid shape: (1800, 3600) 
    
    imerg_precip = ds_imerg['Grid/precipitation'][0][:][:]  #Original Precip Shape: (1, 3600, 1800) = (time, lon, lat)
    imerg_precip = np.transpose(imerg_precip)               #New Precip Shape after transpose: (1800, 3600)
    
    #mask blank data
    imerg_precip_masked = np.ma.masked_where(imerg_precip < 0, imerg_precip)  #masks blank and bad data first (if blank data is -999 instead of NaN)
    imerg_precip_masked = np.ma.masked_where(np.isnan(imerg_precip_masked), imerg_precip_masked)  #masks NaN values (not masked in previous line)        

    precip_rate = ds_precip.avg_tprate.sel(valid_time = file_date[:-2])                            #Time-mean total precipitation rate (kg/m2/s)
    precip_rate = precip_rate.sel(valid_time = precip_rate.valid_time.dt.hour.isin(hr)) * 3600     #Time-mean total precipitation rate (mm/hr)        

    # uwnd = ds.u.sel(time = file_date[:-2]).sel(level = pres_lev)  #zonal winds (m/s)
    # uwnd = uwnd.sel(time = uwnd.time.dt.hour.isin(hr))       #zonal winds (m/s)

    # vwnd = ds.v.sel(time = file_date[:-2]).sel(level = pres_lev)  #meridional winds (m/s)
    # vwnd = vwnd.sel(time = vwnd.time.dt.hour.isin(hr))       #meridional winds (m/s)
    
    # rh = ds.r.sel(time = file_date[:-2]).sel(level = pres_lev)    #relative humidity (%)
    # rh = rh.sel(time = rh.time.dt.hour.isin(hr))             #relative humidity (%)

    # ##Smoothing (source: Hannah Zanowski) --> not recommended, see top of document
    #     ##Metpy smooth_n_point (data to be smoothed, number of points to use in smoothing (5 to 9 are valid), and number of times the smoother is applied)
    #         ##see https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.smooth_n_point.html for more info
    # #uwnd_smoothed = mpcalc.smooth_n_point(uwnd,9,10)
    # #vwnd_smoothed = mpcalc.smooth_n_point(vwnd,9,10)

    if file_date[:-2] == '20220906':
        case = '10'
    elif file_date[:-2] == '20220907':
        case = '12'
    else:
        sys.exit('Case is not Case 10 or Case 12')
        
    ax.set_title('%s Case %s, %s UTC' % (letter_label, case, hr2))
    #ax.set_title('MIMIC TPW, GPM IMERG, and \nERA5 Precip. Rate (%s, %s UTC)' % (file_date[:-2], hr2))
    #ax.set_title('ERA5 %i hPa RH, GPM IMERG, and \nERA5 Precip. Rate (%s, %s UTC)' % (pres_lev, file_date[:-2], hr2))
    ax.set_extent(campaign_extent, ccrs.PlateCarree()) #lat/lon bounds are [West,East,South,North]

    # Add land, coastlines, and borders
    #ax.add_feature(cfeature.LAND, facecolor='0.8')
    ax.coastlines(ls = '-', linewidth = 4, color = 'gray')
    
    #plot MIMIC TPW
    tpw_levels = np.arange(0, 70.5, 2)
    pm0 = ax.contourf(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, levels = tpw_levels,
                      extend = 'max', cmap = cm.jet, transform = data_proj, zorder = 1)
#             pm0 = ax.pcolormesh(ds_tpw.lonArr, ds_tpw.latArr, ds_tpw.tpwGrid, vmin = 0, vmax = 70,
#                                 cmap = cm.jet, transform = data_proj, zorder = 1)

#     #plot ERA5 RH
#     tpw_levels = np.arange(0, 100.1, 2)  #actually RH levels, but keeping the tpw_levels name because we use it elsewhere
#     pm0 = ax.contourf(ds.longitude, ds.latitude, rh[0].values, levels = tpw_levels,
#                       extend = 'max', cmap = cm.jet, transform = data_proj, alpha = 0.6)
# #             pm0 = ax.pcolormesh(ds.longitude, ds.latitude, rh[0].values, vmin = 0, vmax = 70,
# #                                 cmap = cm.jet, transform = data_proj, zorder = 1)
    
    #plot IMERG Rain Rate
    pm1 = ax.contourf(imerg_lons, imerg_lats, imerg_precip_masked, 
                      levels = np.logspace(np.log10(0.1), np.log10(40), num = len(tpw_levels)), 
                      norm = 'log', extend = 'max', 
                      cmap = cm.jet, transform = data_proj, zorder = 2)
#             pm1 = ax.pcolormesh(imerg_lons, imerg_lats, imerg_precip_masked, 
#                                 norm = mplc.LogNorm(vmin = 0.1, vmax = 40), 
#                                 cmap = cm.jet, transform = data_proj, zorder = 2) 

    #Gridlines
    gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, linewidth = 2,
                      color = 'gray', alpha = 0.5, linestyle = '--', zorder = 3)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size':30, 'color':'black'}
    gl.ylabel_style = {'size':30, 'color':'black'}

    # #plot ERA5 streamlines
    # ax.streamplot(ds.longitude, ds.latitude, uwnd[0].values, vwnd[0].values,
    #               color = 'k', linewidth = 0.6, density = 1.0, transform = data_proj, zorder = 4)

    #plot ERA5 time-mean total precipitation rate (mm/hr)
    ax.contour(ds_precip.longitude, ds_precip.latitude, precip_rate[0].values, levels = [0.1, 1, 5, 10, 20, 40],
               colors = 'k', linewidths = 2.5, transform = data_proj, zorder = 4)

    #plot flight track
    ax.plot(flight_lon, flight_lat, color = 'darkmagenta', linewidth = 4, zorder = 5)
    
    #plot near-storm dropsonde locations for the given flight 
        #if the dropsonde was deployed within 30 minutes (1-hr total range) of the given hour
            #NOTE: Dropsondes with no wind data don't have GPS data either (5 of them total)
    for sonde in drop_coords_and_time:
        if (sonde[2][:2] == hour_prior and int(sonde[2][2:4]) >= 30) or (sonde[2][:2] == hr2 and int(sonde[2][2:4]) < 30):
            #ax.scatter(sonde[0], sonde[1], marker = f'${sonde[2]}$', color = 'b', s = 500)
            ax.scatter(sonde[0], sonde[1], marker = '*', color = 'k', zorder = 5, s = 500)

    #same as above, but labeling the dropsondes by the order that they appear in the
        #Dropsonde_Metric_Calculations.csv, NOT IN CHRONOLOGICAL ORDER!!!
    #for z, sonde in enumerate(drop_coords_and_time):
        #ax.scatter(sonde[0], sonde[1], marker = f'${z + 1}$', color = 'b', s = 120, zorder = 5) 
        
    #plotting the colorbars
    #cbar0 = group_fig.colorbar(pm0, ax = ax, orientation = 'vertical', shrink = 0.75, pad = 0.25)
    #cbar0.set_label('TPW [mm]')
    #cbar0.ax.yaxis.set_ticks_position('left')
    #cbar0.ax.yaxis.set_label_position('left')
        
    # #this works with GeoAxes (i.e., Cartopy's map projections)
    # if ii == 3:  #4th panel (last panel where the colorbar is applicable)
        
    #     #MIMIC TPW colorbar
    #     ticks_tpw = np.arange(0, 70.5, 10, dtype = int)
    #     #cax = group_fig.add_axes([ax.get_position().x1+0.08, ax.get_position().y0 + 0.007, 0.03, ax.get_position().height])
    #     cax0 = group_fig.add_axes([ax.get_position().x1 + 0.08, ax.get_position().y0 + 0.007, 0.03, 0.756])
    #     cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
    #     cbar0.set_label('TPW [mm]')
    #     cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
    #     cbar0.ax.yaxis.set_ticks_position('left')
    #     cbar0.ax.yaxis.set_label_position('left')

    #     # #ERA5 RH colorbar
    #     # ticks_rh = np.arange(0, 100.5, 10, dtype = int)
    #     # #cax = group_fig.add_axes([ax.get_position().x1+0.08, ax.get_position().y0 + 0.007, 0.03, ax.get_position().height])
    #     # cax0 = group_fig.add_axes([ax.get_position().x1 + 0.08, ax.get_position().y0 + 0.007, 0.03, 0.756])
    #     # cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_rh)
    #     # cbar0.set_label('Relative Humidity [%]')
    #     # cbar0.ax.set_yticklabels(list(map(str, list(ticks_rh))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
    #     # cbar0.ax.yaxis.set_ticks_position('left')
    #     # cbar0.ax.yaxis.set_label_position('left')
        
    #     #IMERG colorbar
    #     ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40], dtype = float)
    #     cax1 = group_fig.add_axes([ax.get_position().x1 + 0.08, ax.get_position().y0 + 0.007, 0.03, 0.756])
    #     cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
    #     cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
    #     cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
    #     cbar1.ax.yaxis.set_ticks_position('right')
    #     cbar1.ax.yaxis.set_label_position('right')
                
    ds_tpw.close()
    ds_imerg.close() 
    ds.close()
    ds_precip.close()

    print (file_date + ' precip. rate plot complete!\n')

#TIMPS data
timps_folder = os.path.join(os.getcwd(), 'TIMPS_data')
unique_timps_id = '227341'   #same TIMPS ID for Case 10 and Case 12 (see Dropsonde_Metric_Calculations_CPEXCV.csv)

for filename in os.listdir(timps_folder):
    if unique_timps_id in filename:
        timps_filepath = os.path.join(timps_folder, filename)

case_date1 = case_dict_stream[0][:-2]
case_date2 = case_dict_stream[1][:-2]

timps_ds0 = xr.open_dataset(timps_filepath)
#timps_ds0 = timps_ds0.sel(time = case_date1)
vrr_max = timps_ds0.maxrr.values     #mm/hr
timps_time = timps_ds0.time.values   #datetime

#using scipy.stats.norm for standardized Gaussian weights (mean=0, std=1) to do a 13-point (6-hourly because TIMPS data is 0.5-hourly) moving average:
x_values = np.linspace(-3, 3, 13)    #make sure however many points are used (13), the range is centered on zero for a true Gaussian/bell curve (e.g., see plt.plot(x_values, weights))
                                     #NOTE: to increase the amount of points used in the filtering, just change the last number (13)
                                     #      the first 2 numbers make the plot look most like a Gaussian bell curve, no matter what the last number is
                                     #      (e.g., test other numbers and use plt.plot(x_values, weights))
weights = norm.pdf(x_values, loc = 0, scale = 1)   #mean = 0, standard deviation = 1
#boxcar_weights = np.ones(7)
#plt.plot(x_values, weights)

#using a non-recursive filter for moving average; a recursive filter would use both the raw data and the previously/already filtered data
   #see Coursework/2nd Year/Spring 2022/AOS 575/aos575_applab5_synthetic_data_with_filters.ipynb for more info/examples of moving averages and filtering
vrr_max_gfilt = sig.filtfilt(weights, np.sum(weights), vrr_max)   #applying the moving average to the data (numerator, denominator, data)

ax = group_fig.add_subplot(gs[2, :])
#ax.plot(timps_time, vrr_max, color = 'r', linestyle = '-', linewidth = 2, label = 'Unsmoothed')
ax.plot(timps_time, vrr_max_gfilt, color = 'b', linestyle = '-', linewidth = 2, label = 'Smoothed')
        
ax.axvspan(datetime.strptime(case_date1 + ',161000', '%Y%m%d,%H%M%S'), datetime.strptime(case_date1 + ',180000', '%Y%m%d,%H%M%S'), color = 'orange', alpha = 0.2)   #denotes the sampling time range for Case 10
ax.axvspan(datetime.strptime(case_date2 + ',134500', '%Y%m%d,%H%M%S'), datetime.strptime(case_date2 + ',151300', '%Y%m%d,%H%M%S'), color = 'green', alpha = 0.2)    #denotes the sampling time range for Case 12 (flight leg 1)
ax.axvspan(datetime.strptime(case_date2 + ',161800', '%Y%m%d,%H%M%S'), datetime.strptime(case_date2 + ',174500', '%Y%m%d,%H%M%S'), color = 'green', alpha = 0.2)    #denotes the sampling time range for Case 12 (flight leg 2)

range_start = datetime.strptime(case_date1 + ',133000', '%Y%m%d,%H%M%S')
range_end = datetime.strptime('20220908,003000', '%Y%m%d,%H%M%S')
#range_end = datetime.strptime(case_date2 + ',233000', '%Y%m%d,%H%M%S')

ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%H"))
#ax.xaxis.set_major_locator(ticker.MaxNLocator(18))      #sets number of ticks
ax.set_xlim([np.datetime64(range_start),np.datetime64(range_end)])    
    #use the above line to narrow the plot's time range (even within a file!!)
        #range_start and range_end must be a datetime object or a string with the 
            #format: 'YYYY-MM-DD HH:MM:SS' or 'YYYY-MM-DDTHH:MM:SS'
blah = range_start + timedelta(minutes = 30)
ax.set_xticks([blah + timedelta(hours = 2 * xx) for xx in range(18)])

ax.set_ylim([15,45])
ax.set_yticks(np.arange(15, 45.1, 5))

ax.set_xlabel('Time [UTC]', labelpad = 15)
ax.set_ylabel('Maximum Rain Rate [mm hr$^{-1}$]', labelpad = 15)
ax.set_title('e) TIMPS 0.5-hourly Maximum Rain Rate (6-hourly Smoothed)')

#ax.legend(loc = 'upper right')

#just some tick size settings
ax.tick_params(axis = 'x', rotation = 50)
ax.tick_params(length = 15, width = 5)

#MIMIC TPW colorbar
ticks_tpw = np.arange(0, 70.5, 10, dtype = int)
#cax = group_fig.add_axes([ax.get_position().x1+0.08, ax.get_position().y0 + 0.007, 0.03, ax.get_position().height])
cax0 = group_fig.add_axes([ax.get_position().x1 + 0.08, ax.get_position().y1 + 0.045, 0.03, 0.498])
cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_tpw)
cbar0.set_label('TPW [mm]')
cbar0.ax.set_yticklabels(list(map(str, list(ticks_tpw))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
cbar0.ax.yaxis.set_ticks_position('left')
cbar0.ax.yaxis.set_label_position('left')

# #ERA5 RH colorbar
# ticks_rh = np.arange(0, 100.5, 10, dtype = int)
# #cax = group_fig.add_axes([ax.get_position().x1+0.08, ax.get_position().y0 + 0.007, 0.03, ax.get_position().height])
# cax0 = group_fig.add_axes([ax.get_position().x1 + 0.08, ax.get_position().y1 + 0.045, 0.03, 0.498])
# cbar0 = group_fig.colorbar(pm0, cax = cax0, ticks = ticks_rh)
# cbar0.set_label('Relative Humidity [%]')
# cbar0.ax.set_yticklabels(list(map(str, list(ticks_rh))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
# cbar0.ax.yaxis.set_ticks_position('left')
# cbar0.ax.yaxis.set_label_position('left')

#IMERG colorbar
ticks_imerg = np.array([0.1, 1, 5, 10, 20, 40], dtype = float)
cax1 = group_fig.add_axes([ax.get_position().x1 + 0.08, ax.get_position().y1 + 0.045, 0.03, 0.498])
cbar1 = group_fig.colorbar(pm1, cax = cax1, ticks = ticks_imerg)
cbar1.set_label('IMERG [mm hr$\\bf{^{-1}}$]')
cbar1.ax.set_yticklabels(list(map(str, list(ticks_imerg))))  #labels automatically default to tick values given to ticks parameter in fig.colorbar(), unless you're using a log scale I guess
cbar1.ax.yaxis.set_ticks_position('right')
cbar1.ax.yaxis.set_label_position('right')

#plt.tight_layout()
#plt.subplots_adjust(wspace = 0.1)

#save the figure
plt.subplots_adjust(hspace = 0.2, wspace = 0.25)
plt.savefig('/Users/ben/Desktop/Figure10.png', bbox_inches = 'tight')
#plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
#plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
plt.close()
plt.clf()    #if placing this after plt.close(), may release memory related to the figure (https://stackoverflow.com/questions/741877/how-do-i-tell-matplotlib-that-i-am-done-with-a-plot)

##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
im = Image.open('/Users/ben/Desktop/Figure10.png')

try:
    im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
except:
    #use this for older version of PIL/Pillow if the above line doesn't work, 
    #though this line will have isolated, extremely minor image effects due to 
    #only using 256 colors instead of the 3-element RGB scale
    im2 = im.convert('P')

im2.save('/Users/ben/Desktop/Figure10.png')
im.close()
im2.close()


### For CPEX-CV AGU Paper (Figure 11)

In [ ]:
#Case 10 and 12 convection-relative convergence PDFs
#adapted from CPEXCV_ConvergencePDFs_TIMPS.ipynb

pressures_to_plot_conv = [975, 700]    #desired pressure levels for convergence

#CPEX-CV convective cases to compare against one another for AGU paper
case_dict_conv_aew = {10: ['20220906', [16,17,18]],
                      12: ['20220907', [14,15,16,17,18]]}

#THIS CELL PLOTS A 3-PANEL PLOT OF CONVECTION-RELATIVE CONVERGENCE PDFs COMPARING 2 CASES (1 PANEL PER LIFECYCLE STAGE)

#For each CPEX-CV case, calculate and plot domain-PDFs of low- (975 hPa) and mid- (700 hPa) level convergence 
#(display median, mean, standard deviation, skewness, and kurtosis as well), with the domain being an 
#2-by-2 degree box around TIMPS MCS centroids (e.g., Galarneau et al., 2023)
    #partition the data by lifecycle of the convection (use TIMPS "gmd" variable)
    #for calculating domain-mean convergence, cosine-weight each grid box value (see AOS 573 material)
    #KEEP TRACK OF WHICH GRID CELLS YOU HAVE ALREADY ADDED CONVERGENCE FOR, SO THAT YOU DON'T
        #DOUBLE COUNT GRID CELLS WHEN YOU HAVE A CASE WITH MULTIPLE TIMPS IDS WHOSE BOXES MAY OVERLAP

#CPEX-CV convective cases (average time for dropsondes for a given convective case, per CPEX-CV Well Documented Convection.docx)
case_dict_comp = {10: ['20220906', [16,17,18]],
                  12: ['20220907', [14,15,16,17,18]]}

# case_dict_comp = {8: ['20220906', [11,12]],
#                   9: ['20220906', [13,14,15,16]],
#                   10: ['20220906', [16,17,18]],
#                   11: ['20220907', [13,14]],
#                   12: ['20220907', [14,15,16,17,18]],
#                   13: ['20220907', [15,16]],
#                   14: ['20220914', [10,11,12,14,15]],
#                   15: ['20220914', [12,13,14,15,16,17]],
#                   16: ['20220916', [14,15,16]],
#                   17: ['20220916', [17,18,19]],
#                   18: ['20220922', [6,7,8,9]],
#                   19: ['20220923', [9,10,11,12,13,14,15]],
#                   20: ['20220926', [7,8,9,10,11]],
#                   21: ['20220929', [11,12,13,14]],
#                   22: ['20220930', [14,15]]}        
        
#set some baseline plot displays

#matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
matplotlib.rcParams['axes.labelsize'] = 21
matplotlib.rcParams['axes.titlesize'] = 21
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['xtick.labelsize'] = 21
matplotlib.rcParams['ytick.labelsize'] = 21
matplotlib.rcParams['legend.fontsize'] = 20
#matplotlib.rcParams['legend.facecolor'] = 'w'
#matplotlib.rcParams['axes.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'
matplotlib.rcParams['hatch.linewidth'] = 0.3
        
#Dropsonde data
drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
df_drop = pd.read_csv(drop_metric_filepath)

#ERA5 data
era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
era5_path = os.path.join(era5_folder, 'CPEXCV_ERA5_Reanalysis_Hourly_Pressure.nc')
ds_era5 = xr.open_dataset(era5_path)

#TIMPS data
timps_folder = os.path.join(os.getcwd(), 'TIMPS_data')

conv_bins = np.arange(-15,15.1,0.5)

for pres_lev in pressures_to_plot_conv:          #plot convergence PDFs at low- and mid-levels

    group_fig = plt.figure(figsize = (36, 12))   #initialize the convergence PDF figure for the given case comparison

    first_case_no_lifecycle_data = [False, False, False]
    
    for case_num in case_dict_comp.keys():

        #print (f'Case {case_num} convergence PDF plots in progress...')

        df_drop_case = df_drop[df_drop['Case'] == case_num].copy()

        case_date = case_dict_comp[case_num][0]
        case_hours = case_dict_comp[case_num][1]
        case_timps_ids = df_drop_case['TIMPS ID'].unique()

        for lifecycle in [1,2,3]:                    #partition convergence PDFs by TIMPS convective lifecycle stage

            conv_df = pd.DataFrame()
            conv_lats_df = pd.DataFrame()
            #conv_lons_df = pd.DataFrame()

            for hr in case_hours:
                hr2 = str(hr).zfill(2)

                coords_check_list = []

                for ii, unique_timps_id in enumerate(case_timps_ids):

                    timps_filepath = None
                    if pd.isnull(unique_timps_id):
                        continue
                    else:
                        unique_timps_id = str(int(unique_timps_id))
                        for filename in os.listdir(timps_folder):
                            if unique_timps_id in filename:
                                timps_filepath = os.path.join(timps_folder, filename)
                                break

                    if timps_filepath == None:
                        sys.exit(f'Could not find TIMPS file for TIMP ID {unique_timps_id}')
                    else:
                        timps_ds0 = xr.open_dataset(timps_filepath)
                        timps_ds0 = timps_ds0.sel(time = case_date)
                        timps_ds = timps_ds0.sel(time = timps_ds0.time.dt.hour.isin(hr))   #gives 2 times: minute = 0 and minute = 30
                        timps_ds = timps_ds.sel(time = timps_ds.time.dt.minute.isin(0))    #grab the time on the hour to match with ERA5

                        if len(timps_ds.gmd) == 0:
                            print (f'{hr2} UTC is out of range of the TIMPS ID range ({timps_ds0.time[0].values.astype(str)[:-10]} - {timps_ds0.time[-1].values.astype(str)[:-10]})')
                            timps_ds0.close()
                            continue

                        #TIMPS has an "unidentifiable" lifecycle phase for the collection range for Case 10, so 
                            #we're using our manual characterization of "mature" instead for Case 10
                        if case_num == 10:
                            timps_gmd = 2
                        else:
                            timps_gmd = timps_ds.gmd.item()

                        #only add ERA5 convergence data for the given lifecycle stage
                        if timps_gmd != lifecycle:
                            timps_ds0.close()
                            continue

                        timps_weighted_lat = timps_ds.centlatwgt.item()
                        timps_weighted_lon = timps_ds.centlonwgt.item()

                        #create 5-by-5 degree box around weighted centroid for the given TIMPS ID at the given hour
                        timps_lat_range = slice(timps_weighted_lat + 2.5, timps_weighted_lat - 2.5)
                        timps_lon_range = slice(timps_weighted_lon - 2.5, timps_weighted_lon + 2.5)

                        #grab all the ERA5 low-/mid-level convergence values and corresponding lats/lons within the given convective box at the given hour
                        v700 = ds_era5.v.sel(time = case_date).sel(level = 700)
                        v700 = v700.sel(time = v700.time.dt.hour.isin(hr))
                        v700 = mpcalc.smooth_gaussian(v700, 5)   #smooth ERA5 winds using a 5-point filter (Quinton)
                        v700 = v700.sel(longitude = timps_lon_range, latitude = timps_lat_range)
                        
                        #manually calculating convergence from ERA5 u and v winds (recommended by Brandon Wolding via George Kiladis)
                        u = ds_era5.u.sel(time = case_date).sel(level = pres_lev)
                        u = u.sel(time = u.time.dt.hour.isin(hr))
                        u = mpcalc.smooth_gaussian(u, 5)   #smooth ERA5 winds using a 5-point filter (Quinton)
                        #u = u.sel(longitude = timps_lon_range, latitude = timps_lat_range)
                
                        v = ds_era5.v.sel(time = case_date).sel(level = pres_lev)
                        v = v.sel(time = v.time.dt.hour.isin(hr))
                        v = mpcalc.smooth_gaussian(v, 5)   #smooth ERA5 winds using a 5-point filter (Quinton)
                        #v = v.sel(longitude = timps_lon_range, latitude = timps_lat_range)
                        
                        delta_lons = 0.25   #ERA5 lat/lon resolution is 0.25 degrees
                        delta_lons_meters = (111.3195 * 1000 * delta_lons) * np.cos(u.latitude.values * np.pi/180)  #distance between longitude lines at equator is 111.3195 km and cosine weighting this distance by latitude
                        dudx = (u[:,:,1:].values - u[:,:,:-1].values).squeeze() / np.expand_dims(np.abs(delta_lons_meters), axis=1)  #squeeze() removes dimensions of size 1 from an array, and expand_dims() inserts a new axis that will appear at the axis position
                        dudx = np.column_stack((dudx, dudx[:,-1]))  #duplicate the last column of dudx to match original grid shape (and shape of dvdy)
            
                        delta_lats = 0.25
                        delta_lats_meters = 110.5744 * 1000 * delta_lats  #distance between latitude lines everywhere
                        dvdy = (v[:,:-1,:].values - v[:,1:,:].values).squeeze() / delta_lats_meters  #squeeze() removes dimensions of size 1 from an array
                        dvdy = np.vstack((dvdy, dvdy[-1,:]))  #duplicate the last row of dvdy to match original grid shape (and shape of dudx)
            
                        conv_old = (dudx + dvdy) * -1 * 10**5  #manually calculated convergence from ERA5 u and v winds (times 10**5 1/s)
                        ds_conv = xr.Dataset(data_vars = dict(convergence = (["latitude", "longitude"], conv_old)),
                                             coords = dict(latitude = ("latitude", u.latitude.values), 
                                                           longitude = ("longitude", u.longitude.values)),
                                             attrs = dict(description = "Manually calculated ERA5 convergence data"))
                        
                        conv = ds_conv.convergence.sel(longitude = timps_lon_range, latitude = timps_lat_range)

                        # #grab all the ERA5 low-level convergence values and corresponding lats/lons within the given TIMPS ID box at the given hour
                        # conv = ds_era5.d.sel(time = case_date).sel(level = pres_lev) * -1    #convergence of the wind (1/s)
                        # conv = conv.sel(time = conv.time.dt.hour.isin(hr)) * 10**5  #convergence of the wind (times 10**5 1/s)
                        # conv = conv.sel(longitude = timps_lon_range, latitude = timps_lat_range)

                        lon, lat = np.meshgrid(conv.longitude, conv.latitude)
                        lons = lon.reshape(-1)
                        lats = lat.reshape(-1)
                        conv_values = conv.values.reshape(-1)

                        #add data from each hour as COLUMNS to corresponding df
                        #only add values to conv_df that haven't been already been added from a prior TIMPS ID for the given hour (so no duplicates!)
                        if ii != 0:  #if not the first TIMPS ID for the given hour
                            for xx, coord in enumerate(zip(lats,lons)):
                                if coord in coords_check_list:
                                    print (f'Duplicate coordinate for Case {case_num}, replacing with NaN')
                                    lats[xx] = np.nan
                                    lons[xx] = np.nan
                                    conv_values[xx] = np.nan

                        coords_check_list += list(zip(lats, lons))  #append the lats/lons to the list as tuples (coordinate pairs)

                        conv_df = pd.concat((conv_df, pd.Series(conv_values)), axis = 1, ignore_index = True)
                        conv_lats_df = pd.concat((conv_lats_df, pd.Series(lats)), axis = 1, ignore_index = True)
                        #conv_lons_df = pd.concat((conv_lons_df, pd.Series(lons)), axis = 1, ignore_index = True)

                        timps_ds0.close()
            
            #create or grab the axis of interest
            if case_num == list(case_dict_comp.keys())[0]:
                ax = group_fig.add_subplot(1, 3, lifecycle)
                text_denom = 0
                
                if lifecycle == 1:
                    clc = 'Growth'
                    color = 'limegreen'
                elif lifecycle == 2:
                    clc = 'Mature'
                    color = 'darkred'
                elif lifecycle == 3:
                    clc = 'Decay'
                    color = 'cornflowerblue'
            else:  #case_num == list(case_dict_comp.keys())[-1]
                ax = group_fig.get_axes()[lifecycle - 1]
                text_denom = 0.2

                if lifecycle == 1:
                    clc = 'Growth'
                    color = 'darkgreen'
                elif lifecycle == 2:
                    clc = 'Mature'
                    color = 'navy'
                elif lifecycle == 3:
                    clc = 'Decay'
                    color = 'darkblue'
            
                #ax.set_title('ERA5 %i hPa Convergence PDFs\n(Convection-Relative, %s Stage, 5x5 Degree Box)' % (pres_lev, clc))
                ax.set_title('a) ERA5 %i hPa Convergence PDFs (Convection-Relative)' % (pres_lev))
            
            if (len(conv_df) == 0) and (case_num == list(case_dict_comp.keys())[0]):     #no data for the given lifecycle for the given case for the given hours
                first_case_no_lifecycle_data[lifecycle - 1] = True
                continue
            elif (len(conv_df) == 0) and (case_num == list(case_dict_comp.keys())[-1]):  #no data for the given lifecycle for the given case for the given hours
                if first_case_no_lifecycle_data[lifecycle - 1]:  #no data for the given lifecycle for either case
                    ax.text(0.5, 0.5, f'{clc} stage was not sampled for\nCase {list(case_dict_comp.keys())[0]} nor Case {case_num} during CPEX-CV', 
                            horizontalalignment = 'center', verticalalignment = 'center', 
                            fontsize = 30, bbox = {'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
                    ax.set_xticklabels([])
                    ax.set_yticklabels([])
                    continue
                else:
                    continue
            else:
                pass

            conv_df = conv_df.values  #convert Pandas DataFrame to NumPy array
            conv_lats_df = conv_lats_df.values  #convert Pandas DataFrame to NumPy array

            #mask NaN values in the Dataframes so that the numpy stat calculations work below
            conv_df_masked = np.ma.masked_where(np.isnan(conv_df), conv_df)
            conv_lats_df_masked = np.ma.masked_where(np.isnan(conv_lats_df), conv_lats_df)

            #line plot histogram (clearer to interpret than "step" histogram below)
            hist, bins = np.histogram(conv_df_masked.reshape(-1), bins = conv_bins, density = True, weights = None)
            bin_centers = (bins[:-1] + bins[1:]) / 2  # Midpoints of the bins
            ax.plot(bin_centers, hist, linewidth = 2, linestyle = '-', color = color, label = f'Case {case_num}')

            # #normal "step" histogram
            # ax.hist(conv_df_masked.reshape(-1), bins = conv_bins, density = True, weights = None,
            #         histtype = 'step', align = 'mid', orientation = 'vertical', color = color, 
            #         linewidth = 2, label = f'Case {case_num}')
            #     #density = True returns a probability density: each bin will display the bin's raw count 
            #         #divided by the total number of counts and the bin width
            #         #(density = counts / (sum(counts) * np.diff(bins))), so that the area under the 
            #         #histogram integrates to 1 (np.sum(density * np.diff(bins)) == 1)

            cos_weights = np.sqrt(np.cos(conv_lats_df_masked * np.pi/180))   #cosine weights to apply to conv_df

            conv_count = np.count_nonzero(~np.isnan(conv_df))
            conv_median = np.round(np.nanmedian(conv_df, axis = None), 2)
            conv_mean = np.round(np.nanmean(conv_df, axis = None), 2)                                        #non-weighted mean (1st moment)
            conv_wgt_mean = np.round(np.average(conv_df_masked, axis = None, weights = cos_weights), 2)      #cosine-weighted mean (1st moment)
            conv_std = np.round(np.std(conv_df_masked, axis = None), 2)                                      #standard deviation (2nd moment)
            conv_skew = np.round(scipy.stats.skew(conv_df_masked, axis = None, nan_policy = 'omit'), 4)      #skewness (3rd moment)
            conv_kurt = np.round(scipy.stats.kurtosis(conv_df_masked, axis = None, nan_policy = 'omit'), 4)  #kurtosis (4th moment)

            ax.axvline(x = 0, color = 'k', linestyle = '--', alpha = 0.5)
            
            # ax.text(0.98, 0.89 - text_denom, 
            #         f'Count: {conv_count}\nMedian: {conv_median}\nMean: {conv_mean}\nWeighted Mean: {conv_wgt_mean}\nStandard Deviation: {conv_std}\nSkewness: {conv_skew}\nKurtosis: {conv_kurt}\n', 
            #         transform = ax.transAxes, horizontalalignment = 'right', verticalalignment = 'center', 
            #         fontsize = 16, fontweight = 'bold', color = color)
                    
        #print (f'Case {case_num} convergence PDF plots complete!\n')
                    
    for ax in group_fig.get_axes():
        ax.set_xlabel('Convergence [10$^{-5}$ s$^{-1}$]')
        ax.set_ylabel('Prob(Convergence)')
        ax.set_xlim([-13,13])
        ax.set_xticks(np.arange(-13, 13.1, 2))
        ax.set_ylim(bottom = 0)
        #ax.set_yticks(np.arange(0, 0.401, 0.05))
        ax.grid(axis = 'y')
        ax.legend(loc = 'upper left')

    #plt.tight_layout()
    #plt.subplots_adjust(wspace = 0.1)
    
    #save the figure
    case_nums = '-'.join(map(str, case_dict_comp.keys()))
    plot_save_name = f'Case{case_nums}_{pres_lev}hPa_convergence_PDFs_convection_relative_separate.png'
    plt.savefig(os.path.join('/Users/ben/Desktop', plot_save_name), bbox_inches = 'tight')
    #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
    #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
    plt.close()
    plt.clf()    #if placing this after plt.close(), may release memory related to the figure (https://stackoverflow.com/questions/741877/how-do-i-tell-matplotlib-that-i-am-done-with-a-plot)

    ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
    ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
    im = Image.open(os.path.join('/Users/ben/Desktop', plot_save_name))

    try:
        im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
    except:
        #use this for older version of PIL/Pillow if the above line doesn't work, 
        #though this line will have isolated, extremely minor image effects due to 
        #only using 256 colors instead of the 3-element RGB scale
        im2 = im.convert('P')

    im2.save(os.path.join('/Users/ben/Desktop', plot_save_name))
    im.close()
    im2.close()

ds_era5.close()

#tend = time.time()
#print (f'This script took {np.round((tend - tstart) / 60, 1)} minutes to complete.')


### For CPEX-CV AGU Paper (Figure 12)

In [ ]:
#Case 10 and 12 AEW-relative convergence PDFs
#adapted from CPEXCV_ConvergencePDFs_TIMPS.ipynb

pressures_to_plot_conv = [975, 700]    #desired pressure levels for convergence

#CPEX-CV convective cases to compare against one another for AGU paper
case_dict_conv_aew = {10: ['20220906', [16,17,18]],
                      12: ['20220907', [14,15,16,17,18]]}

#THIS CELL PLOTS A 1-PANEL PLOT OF AEW-RELATIVE CONVERGENCE PDFs COMPARING 2 CASES (1 PANEL PER AEW SECTOR)

#For each CPEX-CV case, calculate and plot domain-PDFs of low- (975 hPa) and mid- (700 hPa) level convergence 
#(display median, mean, standard deviation, skewness, and kurtosis as well), with the domain being a 
#10-by-10 degree box around the AEW center for the given case (get from Quinton’s AEW tracker: https://osf.io/jnv5u, https://zenodo.org/records/13350860)
    #partition the data into the 2 sectors of the AEW for the given case (ahead/behind the AEW)
    #for calculating domain-mean convergence, cosine-weight each grid box value (see AOS 573 material)
    
    #WON'T NEED TO KEEP TRACK OF WHICH GRID CELLS YOU HAVE ALREADY ADDED CONVERGENCE FOR, SINCE YOU'RE NOT
        #WORKING WITH MULTIPLE AEWs AT A GIVEN HOUR (LIKE YOU WERE WITH MULTIPLE TIMPS IDs PER HOUR)
        
    #Also don't need to split up convergence PDFs by convective lifecycle (just ahead/behind the AEW), because
        #it would be difficult to relate convergence of an AEW region to convective lifecycle, since one
        #AEW region could (and likely often does) have convective systems that are in different lifecycle stages

#set some baseline plot displays

#matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
matplotlib.rcParams['axes.labelsize'] = 21
matplotlib.rcParams['axes.titlesize'] = 21
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['xtick.labelsize'] = 21
matplotlib.rcParams['ytick.labelsize'] = 21
matplotlib.rcParams['legend.fontsize'] = 20
#matplotlib.rcParams['legend.facecolor'] = 'w'
#matplotlib.rcParams['axes.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'
matplotlib.rcParams['hatch.linewidth'] = 0.3
        
#Dropsonde data
drop_metric_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
df_drop = pd.read_csv(drop_metric_filepath)

#ERA5 data
era5_folder = os.path.join(os.getcwd(), 'ERA5_Reanalysis_Data')
era5_path = os.path.join(era5_folder, 'CPEXCV_ERA5_Reanalysis_Hourly_Pressure.nc')
ds_era5 = xr.open_dataset(era5_path)

#AEW tracker data, 6-hourly (Quinton’s AEW tracker: https://osf.io/jnv5u, https://zenodo.org/records/13350860)
aew_folder = os.path.join(os.getcwd(), 'AEW_Tracker_Data')
aew_path = os.path.join(aew_folder, 'AEW_tracks_post_processed_year_2022.nc')
ds_aew = xr.open_dataset(aew_path)

#TIMPS data
timps_folder = os.path.join(os.getcwd(), 'TIMPS_data')

conv_bins = np.arange(-15,15.1,0.5)

#plot convergence PDFs at low- and mid-levels for each AEW sector for each case
for pres_lev in pressures_to_plot_conv:
    for sector in ['full', 'south_ahead', 'ahead', 'north_ahead', 'south_behind', 'behind', 'north_behind', 'north']:
    
        group_fig = plt.figure(figsize = (12, 12))   #initialize the convergence PDF figure for the given case
        ax = group_fig.add_subplot(1, 1, 1)
        
        for case_num in case_dict_conv_aew.keys():
        
            #print (f'Case {case_num} convergence PDF plots in progress...')
            
            aew_system_index_use = None
            
            df_drop_case = df_drop[df_drop['Case'] == case_num].copy()
            
            case_date = case_dict_conv_aew[case_num][0]
            case_hours = case_dict_conv_aew[case_num][1]
            case_timps_ids = df_drop_case['TIMPS ID'].unique()
            
            if len(case_timps_ids) == 1 and pd.isnull(case_timps_ids[0]):  #no TIMPS IDs for the given case
                continue
            
            aews_at_given_date = ds_aew.sel(time = case_date)
            day_after = datetime.strptime(case_date, '%Y%m%d') + timedelta(days = 1)  #day after case_date
            aews_at_given_date_plus1 = ds_aew.sel(time = datetime.strftime(day_after, '%Y%m%d'))
            
            #match the convective case to the nearest AEW (longitudinally) from Quinton's tracker
                #and confirm that each TIMPS ID for the given case matches to the same AEW (sanity check)
            case_hour_nearest_multiple6 = case_hours[np.nanargmin(np.abs(np.array(case_hours) % 6))]  #find the hour closest to a multiple of 6 (AEW tracker is only in 6-hourly intervals)
            nearest_6_to_case_hour = 6 * round(case_hour_nearest_multiple6 / 6)  #find the multiple of 6 closest to case_hour_nearest_multiple6
        
            for ii, unique_timps_id in enumerate(case_timps_ids):
                
                unique_timps_id = str(int(unique_timps_id))
                
                timps_filepath = None
                for filename in os.listdir(timps_folder):
                    if unique_timps_id in filename:
                        timps_filepath = os.path.join(timps_folder, filename)
                        break
        
                if timps_filepath == None:
                    sys.exit(f'Could not find TIMPS file for TIMP ID {unique_timps_id}')
                else:
                    timps_ds0 = xr.open_dataset(timps_filepath)
                    timps_ds0 = timps_ds0.sel(time = case_date)
                    timps_ds = timps_ds0.sel(time = timps_ds0.time.dt.hour.isin(case_hour_nearest_multiple6))   #gives 2 times: minute = 0 and minute = 30
                    timps_ds = timps_ds.sel(time = timps_ds.time.dt.minute.isin(0))    #grab the time on the hour to match with ERA5 and AEW tracker
        
                    if len(timps_ds.gmd) == 0:
                        print (f'{str(case_hour_nearest_multiple6).zfill(2)} UTC is out of range of the TIMPS ID range ({timps_ds0.time[0].values.astype(str)[:-10]} - {timps_ds0.time[-1].values.astype(str)[:-10]})')
                        timps_ds0.close()
                        continue
        
                    #timps_weighted_lat = timps_ds.centlatwgt.item()
                    timps_weighted_lon = timps_ds.centlonwgt.item()
                    
                    aews_at_given_hour = aews_at_given_date.sel(time = aews_at_given_date.time.dt.hour.isin(nearest_6_to_case_hour))
                
                    lon_difs = aews_at_given_hour['AEW_lon_smooth'][:,0] - timps_weighted_lon
                    aew_system_index = np.nanargmin(np.abs(lon_difs).values)   #np.argmin() would grab a NaN value!
                    
                    if aew_system_index_use != None:
                        if case_num != 22:
                            assert aew_system_index == aew_system_index_use, 'TIMPS IDs for the given case are not matching to the same AEW'
                        else:  #Case 22 has 2 TIMPS IDs which incorrectly match to different AEWs;
                               #the 2nd TIMPS ID (304352) matches to the correct AEW, so we're forcing the code to choose that AEW here (not ideal, but just one case we need to do this for)
                            aew_system_index_use = aew_system_index * 1   #the number of the AEW in the AEW tracker file (using * 1 so that aew_system_index_use variable doesn't point (i.e., isn't tied to) to the same reference as aew_system_index)
                            print (f"Actual AEW system: {aew_system_index_use + 1}, AEW central longitude and strength at {case_date} {nearest_6_to_case_hour} UTC: {np.round(aews_at_given_hour['AEW_lon_smooth'][aew_system_index_use,0].item(), 2)}, {aews_at_given_hour['AEW_strength'][aew_system_index_use,0].item()} s-1")
                    else:
                        aew_system_index_use = aew_system_index * 1   #the number of the AEW in the AEW tracker file (using * 1 so that aew_system_index_use variable doesn't point (i.e., isn't tied to) to the same reference as aew_system_index)
                        print (f"AEW system: {aew_system_index_use + 1}, AEW central longitude and strength at {case_date} {nearest_6_to_case_hour} UTC: {np.round(aews_at_given_hour['AEW_lon_smooth'][aew_system_index_use,0].item(), 2)}, {aews_at_given_hour['AEW_strength'][aew_system_index_use,0].item()} s-1")
                        
                    timps_ds0.close()
                
            matched_aew_ds = aews_at_given_date.isel(system = aew_system_index_use)
            matched_aew_ds_dayafter = aews_at_given_date_plus1.isel(system = aew_system_index_use)
    
            conv_df = pd.DataFrame()
            conv_lats_df = pd.DataFrame()
            #conv_lons_df = pd.DataFrame()
    
            for hr in case_hours:
    
                #grab/calculate the AEW centroid coordinates for the given hr
                if hr % 6 == 0:  #if hr is a multiple of 6, then don't need to interpolate the AEW centroid at all
                    matched_aew_smoothed_lat = matched_aew_ds.sel(time = matched_aew_ds.time.dt.hour.isin(hr)).AEW_lat_smooth.item()
                    matched_aew_smoothed_lon = matched_aew_ds.sel(time = matched_aew_ds.time.dt.hour.isin(hr)).AEW_lon_smooth.item()
    
                else:  #interpolate the centroid of the matched AEW to the given hr
    
                    #find the multiple of 6 directly below/equal to hr; this will be your starting hour to interpolate the AEW centroid to hr
                    nearest_6_below_hr = 6 * (hr // 6)
                    nearest_6_above_hr = nearest_6_below_hr + 6  #this will be your ending hour to interpolate the AEW centroid to hr
    
                    matched_aew_start_hr_ds = matched_aew_ds.sel(time = matched_aew_ds.time.dt.hour.isin(nearest_6_below_hr))
    
                    if nearest_6_above_hr == 24:  #grab 00 UTC from the next day
                        matched_aew_end_hr_ds = matched_aew_ds_dayafter.sel(time = matched_aew_ds_dayafter.time.dt.hour.isin(0))
                    else:
                        matched_aew_end_hr_ds = matched_aew_ds.sel(time = matched_aew_ds.time.dt.hour.isin(nearest_6_above_hr))
    
                    matched_aew_start_lat = matched_aew_start_hr_ds.AEW_lat_smooth.item()
                    matched_aew_start_lon = matched_aew_start_hr_ds.AEW_lon_smooth.item()
                    matched_aew_end_lat = matched_aew_end_hr_ds.AEW_lat_smooth.item()
                    matched_aew_end_lon = matched_aew_end_hr_ds.AEW_lon_smooth.item()
    
                    #AEW centroid moves XX degrees per hour
                    lat_per_hour = (matched_aew_end_lat - matched_aew_start_lat) / 6
                    lon_per_hour = (matched_aew_end_lon - matched_aew_start_lon) / 6
    
                    #interpolated AEW centroid at the given hr
                    matched_aew_smoothed_lat = matched_aew_start_lat + (lat_per_hour * (hr % 6))
                    matched_aew_smoothed_lon = matched_aew_start_lon + (lon_per_hour * (hr % 6))
    
                #create 10-by-10 degree box around the (interpolated) AEW centroid at the given hr
                aew_lat_range = slice(matched_aew_smoothed_lat + 5, matched_aew_smoothed_lat - 5)
                aew_lon_range = slice(matched_aew_smoothed_lon - 5, matched_aew_smoothed_lon + 5)
    
                #grab all the ERA5 low-/mid-level convergence values and corresponding lats/lons within the given AEW box at the given hour
                    #and separate the data into the 2 sectors (ahead/behind) of the AEW
                v700 = ds_era5.v.sel(time = case_date).sel(level = 700)
                v700 = v700.sel(time = v700.time.dt.hour.isin(hr))
                v700 = mpcalc.smooth_gaussian(v700, 5)   #smooth ERA5 winds using a 5-point filter (Quinton)
                v700 = v700.sel(longitude = aew_lon_range, latitude = aew_lat_range)
            
                #manually calculating convergence from ERA5 u and v winds (recommended by Brandon Wolding via George Kiladis)
                u = ds_era5.u.sel(time = case_date).sel(level = pres_lev)
                u = u.sel(time = u.time.dt.hour.isin(hr))
                u = mpcalc.smooth_gaussian(u, 5)   #smooth ERA5 winds using a 5-point filter (Quinton)
                #u = u.sel(longitude = aew_lon_range, latitude = aew_lat_range)
                
                v = ds_era5.v.sel(time = case_date).sel(level = pres_lev)
                v = v.sel(time = v.time.dt.hour.isin(hr))
                v = mpcalc.smooth_gaussian(v, 5)   #smooth ERA5 winds using a 5-point filter (Quinton)
                #v = v.sel(longitude = aew_lon_range, latitude = aew_lat_range)
                
                delta_lons = 0.25   #ERA5 lat/lon resolution is 0.25 degrees
                delta_lons_meters = (111.3195 * 1000 * delta_lons) * np.cos(u.latitude.values * np.pi/180)  #distance between longitude lines at equator is 111.3195 km and cosine weighting this distance by latitude
                dudx = (u[:,:,1:].values - u[:,:,:-1].values).squeeze() / np.expand_dims(np.abs(delta_lons_meters), axis=1)  #squeeze() removes dimensions of size 1 from an array, and expand_dims() inserts a new axis that will appear at the axis position
                dudx = np.column_stack((dudx, dudx[:,-1]))  #duplicate the last column of dudx to match original grid shape (and shape of dvdy)
    
                delta_lats = 0.25
                delta_lats_meters = 110.5744 * 1000 * delta_lats  #distance between latitude lines everywhere
                dvdy = (v[:,:-1,:].values - v[:,1:,:].values).squeeze() / delta_lats_meters  #squeeze() removes dimensions of size 1 from an array
                dvdy = np.vstack((dvdy, dvdy[-1,:]))  #duplicate the last row of dvdy to match original grid shape (and shape of dudx)
    
                conv_old = (dudx + dvdy) * -1 * 10**5  #manually calculated convergence from ERA5 u and v winds (times 10**5 1/s)
                ds_conv = xr.Dataset(data_vars = dict(convergence = (["latitude", "longitude"], conv_old)),
                                     coords = dict(latitude = ("latitude", u.latitude.values), 
                                                   longitude = ("longitude", u.longitude.values)),
                                     attrs = dict(description = "Manually calculated ERA5 convergence data"))
                
                conv = ds_conv.convergence.sel(longitude = aew_lon_range, latitude = aew_lat_range)
            
                # #using convergence variable from ERA5
                # conv = ds_era5.d.sel(time = case_date).sel(level = pres_lev) * -1    #convergence of the wind (1/s)
                # conv = conv.sel(time = conv.time.dt.hour.isin(hr)) * 10**5  #convergence of the wind (times 10**5 1/s)
                # conv = conv.sel(longitude = aew_lon_range, latitude = aew_lat_range)
                
                #filter the convergence data by the 700-hPa v-component of the wind  
                    #v <= 0: the grid point is ahead of the AEW center 
                    #v > 0: the grid point is behind the AEW center
                        #This dynamically defines ahead/behind AEW centers, which is especially practical for asymmetric AEWs!
                if sector == 'full':
                    conv = conv * 1
                    clc = 'Full'
                elif sector == 'south_ahead':
                    conv = conv.where(conv.latitude < matched_aew_smoothed_lat).where(v700 <= 0)  #returns elements from 'conv' where condition is True, otherwise fill in NaNs by default
                    clc = 'Ahead (South)'
                elif sector == 'ahead':
                    conv = conv.where(v700 <= 0)  #returns elements from 'conv' where condition is True, otherwise fill in NaNs by default
                    clc = 'Ahead'
                elif sector == 'north_ahead':
                    conv = conv.where(conv.latitude >= matched_aew_smoothed_lat).where(v700 <= 0)  #returns elements from 'conv' where condition is True, otherwise fill in NaNs by default
                    clc = 'Ahead (North)'
                elif sector == 'south_behind':
                    conv = conv.where(conv.latitude < matched_aew_smoothed_lat).where(v700 > 0)   #returns elements from 'conv' where condition is True, otherwise fill in NaNs by default
                    clc = 'Behind (South)'
                elif sector == 'behind':
                    conv = conv.where(v700 > 0)   #returns elements from 'conv' where condition is True, otherwise fill in NaNs by default
                    clc = 'Behind'
                elif sector == 'north_behind':
                    conv = conv.where(conv.latitude >= matched_aew_smoothed_lat).where(v700 > 0)   #returns elements from 'conv' where condition is True, otherwise fill in NaNs by default
                    clc = 'Behind (North)'
                elif sector == 'north':
                    conv = conv.where(conv.latitude >= matched_aew_smoothed_lat)   #returns elements from 'conv' where condition is True, otherwise fill in NaNs by default
                    clc = 'North'
    
                lon, lat = np.meshgrid(conv.longitude, conv.latitude)
                lats = lat.reshape(-1)
                conv_values = conv.values.reshape(-1)
    
                #add data from each hour as COLUMNS to corresponding df
                conv_df = pd.concat((conv_df, pd.Series(conv_values)), axis = 1, ignore_index = True)
                conv_lats_df = pd.concat((conv_lats_df, pd.Series(lats)), axis = 1, ignore_index = True)
                #conv_lons_df = pd.concat((conv_lons_df, pd.Series(lons)), axis = 1, ignore_index = True)         
                    
            if case_num == list(case_dict_conv_aew.keys())[0]:
                color = 'darkred'
                text_denom = 0
            elif case_num == list(case_dict_conv_aew.keys())[1]:
                color = 'navy'
                text_denom = 0.2
    
            conv_df = conv_df.values  #convert Pandas DataFrame to NumPy array
            conv_lats_df = conv_lats_df.values  #convert Pandas DataFrame to NumPy array
    
            #mask NaN values in the Dataframes so that the numpy stat calculations work below
            conv_df_masked = np.ma.masked_where(np.isnan(conv_df), conv_df)
            conv_lats_df_masked = np.ma.masked_where(np.isnan(conv_lats_df), conv_lats_df)
    
            #line plot histogram (clearer to interpret than "step" histogram below)
            hist, bins = np.histogram(conv_df_masked.reshape(-1), bins = conv_bins, density = True, weights = None)
            bin_centers = (bins[:-1] + bins[1:]) / 2  # Midpoints of the bins
            ax.plot(bin_centers, hist, linewidth = 2, linestyle = '-', color = color, label = f'Case {case_num}')
    
            # #normal "step" histogram
            # ax.hist(conv_df_masked.reshape(-1), bins = conv_bins, density = True, weights = None,
            #         histtype = 'step', align = 'mid', orientation = 'vertical', color = color,
            #         linewidth = 2, label = f'Case {case_num}')
            #     #density = True returns a probability density: each bin will display the bin's raw count 
            #         #divided by the total number of counts times the bin width
            #         #(density = counts / (sum(counts) * np.diff(bins))), so that the area under the 
            #         #histogram integrates to 1 (np.sum(density * np.diff(bins)) == 1)
    
            cos_weights = np.sqrt(np.cos(conv_lats_df_masked * np.pi/180))   #cosine weights to apply to conv_df_masked
    
            conv_count = np.count_nonzero(~np.isnan(conv_df))
            conv_median = np.round(np.nanmedian(conv_df, axis = None), 2)
            conv_mean = np.round(np.nanmean(conv_df, axis = None), 2)                                        #non-weighted mean (1st moment)
            conv_wgt_mean = np.round(np.average(conv_df_masked, axis = None, weights = cos_weights), 2)      #cosine-weighted mean (1st moment)
            conv_std = np.round(np.std(conv_df_masked, axis = None), 2)                                      #standard deviation (2nd moment)
            conv_skew = np.round(scipy.stats.skew(conv_df_masked, axis = None, nan_policy = 'omit'), 4)      #skewness (3rd moment)
            conv_kurt = np.round(scipy.stats.kurtosis(conv_df_masked, axis = None, nan_policy = 'omit'), 4)  #kurtosis (4th moment)
    
            # ax.text(0.98, 0.89 - text_denom, 
            #         f'Count: {conv_count}\nMedian: {conv_median}\nMean: {conv_mean}\nWeighted Mean: {conv_wgt_mean}\nStandard Deviation: {conv_std}\nSkewness: {conv_skew}\nKurtosis: {conv_kurt}\n', 
            #         transform = ax.transAxes, horizontalalignment = 'right', verticalalignment = 'center', 
            #         fontsize = 16, fontweight = 'bold', color = color)
        
            if case_num == 20:
                ax.text(0.5, 0.5, f'Case {case_num} not being correctly\nmatched to the appropriate AEW', 
                transform = ax.transAxes, horizontalalignment = 'center', verticalalignment = 'center', 
                fontsize = 30, bbox = {'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

        ax.set_title('a) ERA5 %i hPa Convergence PDFs (AEW-Relative, %s)' % (pres_lev, clc))
        ax.axvline(x = 0, color = 'k', linestyle = '--', alpha = 0.5)
        ax.set_xlabel('Convergence [10$^{-5}$ s$^{-1}$]')
        ax.set_ylabel('Prob(Convergence)')
        ax.set_xlim([-11,11])
        ax.set_xticks(np.arange(-11, 11.1, 2))
        ax.set_ylim(bottom = 0)
        #ax.set_yticks(np.arange(0, 0.601, 0.05))
        ax.grid(axis = 'y')
        ax.legend(loc = 'upper left')
                        
        #plt.tight_layout()
        #plt.subplots_adjust(wspace = 0.1)

        #save the figure
        plot_save_name = f'Case{list(case_dict_conv_aew.keys())[0]}-{list(case_dict_conv_aew.keys())[1]}_{pres_lev}hPa_convergence_PDFs_AEW_relative_{sector}.png'
        plt.savefig(os.path.join('/Users/ben/Desktop', plot_save_name), bbox_inches = 'tight')
        #plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
        #plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
        plt.close()
        plt.clf()    #if placing this after plt.close(), may release memory related to the figure (https://stackoverflow.com/questions/741877/how-do-i-tell-matplotlib-that-i-am-done-with-a-plot)

        ##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
        ##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
        im = Image.open(os.path.join('/Users/ben/Desktop', plot_save_name))

        try:
            im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
        except:
            #use this for older version of PIL/Pillow if the above line doesn't work, 
            #though this line will have isolated, extremely minor image effects due to 
            #only using 256 colors instead of the 3-element RGB scale
            im2 = im.convert('P')
    
        im2.save(os.path.join('/Users/ben/Desktop', plot_save_name))
        im.close()
        im2.close()
        
        #print (f'Case {case_num} convergence PDF plots complete!\n')
    
ds_era5.close()
ds_aew.close()

#tend = time.time()
#print (f'This script took {np.round((tend - tstart) / 60, 1)} minutes to complete.')


### For CPEX-CV AGU Paper (Figure 13 and 14)

In [ ]:
# discuss regional variability in this section w/regard to CPEX and CPEX-AW
#     (create figure similar to Figure 18b in your thesis comparing bulk metric spreads for each field campaign)
# adapted from metric_boxwhisker_4panel_CPEXCV.py (see "For CPEX-CV AGU Paper (Figures 4, 6, and 8)" above) and metric_boxwhisker_4panel.py 

# matplotlib.rcParams['axes.labelsize'] = 14
# matplotlib.rcParams['axes.titlesize'] = 14
# matplotlib.rcParams['xtick.labelsize'] = 12
# matplotlib.rcParams['ytick.labelsize'] = 12
# matplotlib.rcParams['legend.fontsize'] = 12
# matplotlib.rcParams['legend.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'

TC_days = [20170619, 20170620, 20210826, 20210828, 20210901, 20210904, 20220923]  #including TDs as TCs (from both CPEX and CPEX-AW and CPEX-CV)

###CPEX-CV files and dataframes
drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
#drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
df = pd.read_csv(drop_filepath)
df_noTC = df[~df['Date'].isin(TC_days)].copy()  #filter out cases associated with a TD/TC

dawn_filepath = os.path.join(os.getcwd(), 'DAWN_Shear_Calculations_CPEXCV.csv')
#dawn_filepath = os.path.join(os.getcwd(), 'DAWN_Shear_Calculations.csv')
df_dawn = pd.read_csv(dawn_filepath)
df1 = pd.concat([df, df_dawn], ignore_index = True)  #concatenates fields with same heading
df1_noTC = df1[~df1['Date'].isin(TC_days)].copy()  #filter out cases associated with a TD/TC

###CPEX(-AW) files and dataframes (W. Atlantic --> _watl)
drop_filepath_watl = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
#drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')
df_watl = pd.read_csv(drop_filepath_watl)
df_noTC_watl = df_watl[~df_watl['Date'].isin(TC_days)].copy()  #filter out cases associated with a TD/TC

dawn_filepath_watl = os.path.join(os.getcwd(), 'DAWN_Shear_Calculations.csv')
#dawn_filepath = os.path.join(os.getcwd(), 'DAWN_Shear_Calculations_CPEXCV.csv')
df_dawn_watl = pd.read_csv(dawn_filepath_watl)
df1_watl = pd.concat([df_watl, df_dawn_watl], ignore_index = True)  #concatenates fields with same heading
df1_noTC_watl = df1_watl[~df1_watl['Date'].isin(TC_days)].copy()  #filter out cases associated with a TD/TC


rh_layers = ['Deep Layer RH [%]', 'PBL RH [%]', 'Mid Layer RH [%]', 'Upper Layer RH [%]']
# speed_shear_layers = ['SHARPpy Direct Method Deep Layer Speed Shear [kts]', 'SHARPpy Direct Method PBL Speed Shear [kts]', 
#                       'SHARPpy Direct Method Mid Layer Speed Shear [kts]', 'SHARPpy Direct Method Upper Layer Speed Shear [kts]']
speed_shear_layers = ['500m Bottom Cap Deep Layer Speed Shear [kts]', 'SHARPpy Direct Method PBL Speed Shear [kts]', 
                      'SHARPpy Direct Method Mid Layer Speed Shear [kts]', 'SHARPpy Direct Method Upper Layer Speed Shear [kts]']
cape_layers = ['Deep Layer MUCAPE [J/kg]', 'Deep Layer MLCAPE [J/kg]', 'Above FZL MUCAPE [J/kg]', 'Above FZL MLCAPE [J/kg]']

use_alpha = 0.25  #alpha for box plots with all sondes, not just inflow sondes

#for convective type comparison of All Lifecycles dropsondes
def box_plot_onlyIsoOrg_RH(plotting_metric_name, ax):  #parameter will be a data column from df
    
    if plotting_metric_name == 'Upper Layer RH [%]':  #include "In Precip" sondes if "Partially In Precip" == Yes
        df_use0 = df_noTC[df_noTC['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
        df_use1 = df_use0[(df_use0['Environment Falling In'] != 'In Precip') | (df_use0['Partially In Precip'] == 'Yes')].copy()  #filter out In Precip, unless "Partially In Precip" == Yes
        df_use = df_use1[(df_use1['Low-level Inflow Sonde'] == 'Yes') | (df_use1['Mid-level Inflow Sonde'] == 'Yes')].copy()
        #df_use = df_noTC[(df_noTC['Environment Falling In'] != 'In Precip') & (df_noTC['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
    else:
        df_use1 = df_noTC[(df_noTC['Environment Falling In'] != 'In Precip') & (df_noTC['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
        df_use = df_use1[(df_use1['Low-level Inflow Sonde'] == 'Yes') | (df_use1['Mid-level Inflow Sonde'] == 'Yes')].copy()

    #all sondes
    df_iso_all = df_use1[df_use1['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all = df_use1[df_use1['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all = df_use1[df_use1['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso = df_use[df_use['Primary Convective Type'] == 'Isolated'].copy()
    df_org = df_use[df_use['Primary Convective Type'] == 'Organized'].copy()
    #df_scat = df_use[df_use['Primary Convective Type'] == 'Scattered'].copy()

######################
    if plotting_metric_name == 'Upper Layer RH [%]':  #include "In Precip" sondes if "Partially In Precip" == Yes
        df_use0_watl = df_noTC_watl[df_noTC_watl['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
        df_use1_watl = df_use0_watl[(df_use0_watl['Environment Falling In'] != 'In Precip') | (df_use0_watl['Partially In Precip'] == 'Yes')].copy()  #filter out In Precip, unless "Partially In Precip" == Yes
        df_use_watl = df_use1_watl[(df_use1_watl['Low-level Inflow Sonde'] == 'Yes') | (df_use1_watl['Mid-level Inflow Sonde'] == 'Yes')].copy()
        #df_use_watl = df_noTC_watl[(df_noTC_watl['Environment Falling In'] != 'In Precip') & (df_noTC_watl['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
    else:
        df_use1_watl = df_noTC_watl[(df_noTC_watl['Environment Falling In'] != 'In Precip') & (df_noTC_watl['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
        df_use_watl = df_use1_watl[(df_use1_watl['Low-level Inflow Sonde'] == 'Yes') | (df_use1_watl['Mid-level Inflow Sonde'] == 'Yes')].copy()

    #all sondes
    df_iso_all_watl = df_use1_watl[df_use1_watl['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all_watl = df_use1_watl[df_use1_watl['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all_watl = df_use1_watl[df_use1_watl['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso_watl = df_use_watl[df_use_watl['Primary Convective Type'] == 'Isolated'].copy()
    df_org_watl = df_use_watl[df_use_watl['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_watl = df_use_watl[df_use_watl['Primary Convective Type'] == 'Scattered'].copy()
    
    #need to filter out NaN values (using dropna()), otherwise the boxplot() won't create anything
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_scat[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                 patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated', 'Organized', 'Scattered'])
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
    #                  df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values, 
    #                  df_scat[plotting_metric_name].dropna().values, df_scat_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                  patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized', 'Scattered\n(Inflow)', 'Scattered'])
    bp = ax.boxplot([df_iso_watl[plotting_metric_name].dropna().values, df_iso[plotting_metric_name].dropna().values, df_iso_all_watl[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
                     df_org_watl[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_org_all_watl[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
                     patch_artist = True, vert = True, widths = 0.65, tick_labels = ['Isolated\n(Inflow)\n(CPEX(-AW))', 'Isolated\n(Inflow)\n(CPEX-CV)', 'Isolated\n(CPEX(-AW))', 'Isolated\n(CPEX-CV)', 'Organized\n(Inflow)\n(CPEX(-AW))', 'Organized\n(Inflow)\n(CPEX-CV)', 'Organized\n(CPEX(-AW))', 'Organized\n(CPEX-CV)'])
    
    print (f'Isolated {plotting_metric_name} median (CPEX(-AW)):', df_iso_watl[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median (CPEX(-AW)):', df_org_watl[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median (CPEX(-AW)):', df_scat_watl[plotting_metric_name].median(skipna = True))
    
    print (f'Isolated {plotting_metric_name} median (CPEX-CV):', df_iso[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median (CPEX-CV):', df_org[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median (CPEX-CV):', df_scat[plotting_metric_name].median(skipna = True))

    colors = ['red', 'red', 'red', 'red', 'blue', 'blue', 'blue', 'blue']
    nums = list(range(len(colors)))
     
    for ii, patch, color in zip(nums, bp['boxes'], colors):
        if ii in [0,1,4,5]:
            patch.set_facecolor(color)
        else:
            patch.set_facecolor(color)
            patch.set_alpha(use_alpha)

    #changing color and linewidth of medians
    for ii, median in enumerate(bp['medians']):
        if ii in [0,1,4,5]:
            median.set(color = 'white', linewidth = 3)
        else:
            #median.set(color = 'white', alpha = use_alpha, linewidth = 3)
            median.set(color = 'white', linewidth = 3)

    #changing color and linewidth of whiskers
    for ii, whisker in enumerate(bp['whiskers']):
        if ii in [0, 1, 2, 3, 8, 9, 10, 11]:  #16 whiskers, not 8
            whisker.set(color = 'k', linewidth = 2, linestyle = "-")
        else:
            whisker.set(color = 'k', alpha = use_alpha, linewidth = 2, linestyle = "-")
     
    #changing color and linewidth of caps
    for ii, cap in enumerate(bp['caps']):
        if ii in [0, 1, 2, 3, 8, 9, 10, 11]:  #16 caps, not 8
            cap.set(color = 'k', linewidth = 2)
        else:
            cap.set(color = 'k', alpha = use_alpha, linewidth = 2)
     
    #changing style of fliers
    for ii, flier, color in zip(nums, bp['fliers'], colors):
        if ii in [0,1,4,5]:
            flier.set(marker = 'o', color = 'k', markersize = 13, markerfacecolor = color)
        else:
            flier.set(marker = 'o', color = 'k', alpha = use_alpha, markersize = 13, markerfacecolor = color)
            
    ax.grid(True, axis = 'y')
    ax.set_title(plotting_metric_name[:-4], fontsize = 33, fontweight = 'bold')
    #ax.set_xlabel('Convective Type', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[%]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([15,100])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
 
    
group_fig = plt.figure(figsize=(48,24))
#group_fig = plt.figure(figsize=(22,24))

for i, layer in enumerate(rh_layers):
    ax = group_fig.add_subplot(2,2,i+1)
    box_plot_onlyIsoOrg_RH(layer, ax)

#custom legend
# legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 's', markersize = 13, label='Isolated (In Precip Profiles Excluded)'),
#                    Line2D([], [], color='blue', linewidth = 0, marker = 's', markersize = 13, label='Organized (In Precip Profiles Excluded)')]
# group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 25)

group_fig.text(0.5, 0.473, 'In Precip Profiles Excluded', horizontalalignment='center', verticalalignment='center', 
               fontsize = 40, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
plt.subplots_adjust(hspace = 0.45, wspace = 0.15)
plt.savefig('/Users/ben/Desktop/RH_box_4panel_allCPEX.png', bbox_inches = 'tight')
plt.close()
print ('')


#for convective type comparison of All Lifecycles dropsondes
def box_plot_onlyIsoOrg_SS(plotting_metric_name, ax):  #parameter will be a data column from df

    if 'Deep Layer Speed Shear' in plotting_metric_name:  #include DAWN speed shear in the deep layer speed shear plot
        df_use1 = df1_noTC[df1_noTC['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
        df_use = df_use1[(df_use1['Low-level Inflow Sonde'] != 'No') | (df_use1['Mid-level Inflow Sonde'] != 'No')].copy()
    
        ###############
        df_use1_watl = df1_noTC_watl[df1_noTC_watl['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
        df_use_watl = df_use1_watl[(df_use1_watl['Low-level Inflow Sonde'] != 'No') | (df_use1_watl['Mid-level Inflow Sonde'] != 'No')].copy()
            
        ax.set_title('Deep Layer Shear (Dropsonde & DAWN, 0.5km - 7.6km)', fontsize = 30, fontweight = 'bold')
        
    else:
    
        df_use1 = df_noTC[df_noTC['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
        df_use = df_use1[(df_use1['Low-level Inflow Sonde'] == 'Yes') | (df_use1['Mid-level Inflow Sonde'] == 'Yes')].copy()

        ######################
        
        df_use1_watl = df_noTC_watl[df_noTC_watl['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
        df_use_watl = df_use1_watl[(df_use1_watl['Low-level Inflow Sonde'] == 'Yes') | (df_use1_watl['Mid-level Inflow Sonde'] == 'Yes')].copy()

        ax.set_title(plotting_metric_name[22:-18] + ' Shear', fontsize = 33, fontweight = 'bold')
        
    #all sondes
    df_iso_all = df_use1[df_use1['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all = df_use1[df_use1['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all = df_use1[df_use1['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso = df_use[df_use['Primary Convective Type'] == 'Isolated'].copy()
    df_org = df_use[df_use['Primary Convective Type'] == 'Organized'].copy()
    #df_scat = df_use[df_use['Primary Convective Type'] == 'Scattered'].copy()
        
    #all sondes
    df_iso_all_watl = df_use1_watl[df_use1_watl['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all_watl = df_use1_watl[df_use1_watl['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all_watl = df_use1_watl[df_use1_watl['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso_watl = df_use_watl[df_use_watl['Primary Convective Type'] == 'Isolated'].copy()
    df_org_watl = df_use_watl[df_use_watl['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_watl = df_use_watl[df_use_watl['Primary Convective Type'] == 'Scattered'].copy()
    
    #need to filter out NaN values (using dropna()), otherwise the boxplot() won't create anything
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_scat[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                 patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated', 'Organized', 'Scattered'])
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
    #                  df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values, 
    #                  df_scat[plotting_metric_name].dropna().values, df_scat_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                  patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized', 'Scattered\n(Inflow)', 'Scattered'])
    bp = ax.boxplot([df_iso_watl[plotting_metric_name].dropna().values, df_iso[plotting_metric_name].dropna().values, df_iso_all_watl[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
                     df_org_watl[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_org_all_watl[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
                     patch_artist = True, vert = True, widths = 0.65, tick_labels = ['Isolated\n(Inflow)\n(CPEX(-AW))', 'Isolated\n(Inflow)\n(CPEX-CV)', 'Isolated\n(CPEX(-AW))', 'Isolated\n(CPEX-CV)', 'Organized\n(Inflow)\n(CPEX(-AW))', 'Organized\n(Inflow)\n(CPEX-CV)', 'Organized\n(CPEX(-AW))', 'Organized\n(CPEX-CV)'])
    
    print (f'Isolated {plotting_metric_name} median (CPEX(-AW)):', df_iso_watl[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median (CPEX(-AW)):', df_org_watl[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median (CPEX(-AW)):', df_scat_watl[plotting_metric_name].median(skipna = True))
    
    print (f'Isolated {plotting_metric_name} median (CPEX-CV):', df_iso[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median (CPEX-CV):', df_org[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median (CPEX-CV):', df_scat[plotting_metric_name].median(skipna = True))

    colors = ['red', 'red', 'red', 'red', 'blue', 'blue', 'blue', 'blue']
    nums = list(range(len(colors)))
     
    for ii, patch, color in zip(nums, bp['boxes'], colors):
        if ii in [0,1,4,5]:
            patch.set_facecolor(color)
        else:
            patch.set_facecolor(color)
            patch.set_alpha(use_alpha)

    #changing color and linewidth of medians
    for ii, median in enumerate(bp['medians']):
        if ii in [0,1,4,5]:
            median.set(color = 'white', linewidth = 3)
        else:
            #median.set(color = 'white', alpha = use_alpha, linewidth = 3)
            median.set(color = 'white', linewidth = 3)

    #changing color and linewidth of whiskers
    for ii, whisker in enumerate(bp['whiskers']):
        if ii in [0, 1, 2, 3, 8, 9, 10, 11]:  #16 whiskers, not 8
            whisker.set(color = 'k', linewidth = 2, linestyle = "-")
        else:
            whisker.set(color = 'k', alpha = use_alpha, linewidth = 2, linestyle = "-")
     
    #changing color and linewidth of caps
    for ii, cap in enumerate(bp['caps']):
        if ii in [0, 1, 2, 3, 8, 9, 10, 11]:  #16 caps, not 8
            cap.set(color = 'k', linewidth = 2)
        else:
            cap.set(color = 'k', alpha = use_alpha, linewidth = 2)
     
    #changing style of fliers
    for ii, flier, color in zip(nums, bp['fliers'], colors):
        if ii in [0,1,4,5]:
            flier.set(marker = 'o', color = 'k', markersize = 13, markerfacecolor = color)
        else:
            flier.set(marker = 'o', color = 'k', alpha = use_alpha, markersize = 13, markerfacecolor = color)

    ax.grid(True, axis = 'y')
    #ax.set_xlabel('Convective Type', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[kts]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([0,50])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    
 
group_fig = plt.figure(figsize=(48,24))
#group_fig = plt.figure(figsize=(22,24))

for i, layer in enumerate(speed_shear_layers):
    ax = group_fig.add_subplot(2,2,i+1)
    box_plot_onlyIsoOrg_SS(layer, ax)

#custom legend
# legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 's', markersize = 13, label='Isolated'),
#                    Line2D([], [], color='blue', linewidth = 0, marker = 's', markersize = 13, label='Organized')]
# group_fig.legend(handles = legend_elements, loc = 'center', fontsize = 25)

group_fig.text(0.5, 0.473, 'In Precip Profiles Included', horizontalalignment='center', verticalalignment='center', 
               fontsize = 40, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
plt.subplots_adjust(hspace = 0.45, wspace = 0.15)
plt.savefig('/Users/ben/Desktop/SS_box_4panel_allCPEX.png', bbox_inches = 'tight')
plt.close()
print ('')

#for convective type comparison of All Lifecycles dropsondes
def box_plot_onlyIsoOrg_SSwDAWN(plotting_metric_name):  #parameter will be a data column from df1

    group_fig = plt.figure(figsize=(48,24))
    #group_fig = plt.figure(figsize=(32,21))
    ax = group_fig.add_subplot(1,2,1)
    
    df_use1 = df1_noTC[df1_noTC['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
    df_use = df_use1[(df_use1['Low-level Inflow Sonde'] != 'No') | (df_use1['Mid-level Inflow Sonde'] != 'No')].copy()
    
    #all sondes
    df_iso_all = df_use1[df_use1['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all = df_use1[df_use1['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all = df_use1[df_use1['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso = df_use[df_use['Primary Convective Type'] == 'Isolated'].copy()
    df_org = df_use[df_use['Primary Convective Type'] == 'Organized'].copy()
    #df_scat = df_use[df_use['Primary Convective Type'] == 'Scattered'].copy()

###############
    df_use1_watl = df1_noTC_watl[df1_noTC_watl['Environment Falling In'] != 'Clear Far'].copy()  #filter out Clear Far dropsondes
    df_use_watl = df_use1_watl[(df_use1_watl['Low-level Inflow Sonde'] != 'No') | (df_use1_watl['Mid-level Inflow Sonde'] != 'No')].copy()
    
    #all sondes
    df_iso_all_watl = df_use1_watl[df_use1_watl['Primary Convective Type'] == 'Isolated'].copy()
    df_org_all_watl = df_use1_watl[df_use1_watl['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_all_watl = df_use1_watl[df_use1_watl['Primary Convective Type'] == 'Scattered'].copy()
        
    #just inflow sondes
    df_iso_watl = df_use_watl[df_use_watl['Primary Convective Type'] == 'Isolated'].copy()
    df_org_watl = df_use_watl[df_use_watl['Primary Convective Type'] == 'Organized'].copy()
    #df_scat_watl = df_use_watl[df_use_watl['Primary Convective Type'] == 'Scattered'].copy()
    
    #need to filter out NaN values (using dropna()), otherwise the boxplot() won't create anything
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_scat[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                 patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated', 'Organized', 'Scattered'])
    # bp = ax.boxplot([df_iso[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
    #                  df_org[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values, 
    #                  df_scat[plotting_metric_name].dropna().values, df_scat_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
    #                  patch_artist = True, vert = True, widths = 0.65, labels = ['Isolated\n(Inflow)', 'Isolated', 'Organized\n(Inflow)', 'Organized', 'Scattered\n(Inflow)', 'Scattered'])
    bp = ax.boxplot([df_iso_watl[plotting_metric_name].dropna().values, df_iso[plotting_metric_name].dropna().values, df_iso_all_watl[plotting_metric_name].dropna().values, df_iso_all[plotting_metric_name].dropna().values, 
                     df_org_watl[plotting_metric_name].dropna().values, df_org[plotting_metric_name].dropna().values, df_org_all_watl[plotting_metric_name].dropna().values, df_org_all[plotting_metric_name].dropna().values], notch = True, bootstrap = 10000,
                     patch_artist = True, vert = True, widths = 0.65, tick_labels = ['Isolated\n(Inflow)\n(CPEX(-AW))', 'Isolated\n(Inflow)\n(CPEX-CV)', 'Isolated\n(CPEX(-AW))', 'Isolated\n(CPEX-CV)', 'Organized\n(Inflow)\n(CPEX(-AW))', 'Organized\n(Inflow)\n(CPEX-CV)', 'Organized\n(CPEX(-AW))', 'Organized\n(CPEX-CV)'])
        
    print (f'Isolated {plotting_metric_name} median (CPEX(-AW)):', df_iso_watl[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median (CPEX(-AW)):', df_org_watl[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median (CPEX(-AW)):', df_scat_watl[plotting_metric_name].median(skipna = True))
    
    print (f'Isolated {plotting_metric_name} median (CPEX-CV):', df_iso[plotting_metric_name].median(skipna = True))
    print (f'Organized {plotting_metric_name} median (CPEX-CV):', df_org[plotting_metric_name].median(skipna = True))
    #print (f'Scattered {plotting_metric_name} median (CPEX-CV):', df_scat[plotting_metric_name].median(skipna = True))

    colors = ['red', 'red', 'red', 'red', 'blue', 'blue', 'blue', 'blue']
    nums = list(range(len(colors)))
     
    for ii, patch, color in zip(nums, bp['boxes'], colors):
        if ii in [0,1,4,5]:
            patch.set_facecolor(color)
        else:
            patch.set_facecolor(color)
            patch.set_alpha(use_alpha)

    #changing color and linewidth of medians
    for ii, median in enumerate(bp['medians']):
        if ii in [0,1,4,5]:
            median.set(color = 'white', linewidth = 3)
        else:
            #median.set(color = 'white', alpha = use_alpha, linewidth = 3)
            median.set(color = 'white', linewidth = 3)

    #changing color and linewidth of whiskers
    for ii, whisker in enumerate(bp['whiskers']):
        if ii in [0, 1, 2, 3, 8, 9, 10, 11]:  #16 whiskers, not 8
            whisker.set(color = 'k', linewidth = 2, linestyle = "-")
        else:
            whisker.set(color = 'k', alpha = use_alpha, linewidth = 2, linestyle = "-")
     
    #changing color and linewidth of caps
    for ii, cap in enumerate(bp['caps']):
        if ii in [0, 1, 2, 3, 8, 9, 10, 11]:  #16 caps, not 8
            cap.set(color = 'k', linewidth = 2)
        else:
            cap.set(color = 'k', alpha = use_alpha, linewidth = 2)
     
    #changing style of fliers
    for ii, flier, color in zip(nums, bp['fliers'], colors):
        if ii in [0,1,4,5]:
            flier.set(marker = 'o', color = 'k', markersize = 13, markerfacecolor = color)
        else:
            flier.set(marker = 'o', color = 'k', alpha = use_alpha, markersize = 13, markerfacecolor = color)
        
    ax.grid(True, axis = 'y')
    ax.set_title('Deep Layer Shear (Dropsonde & DAWN, 0.5km - 7.6km)', fontsize = 30, fontweight = 'bold')
    ax.set_xlabel('Convective Type', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[kts]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([0,50])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    ax.text(0.165, 0.97, 'In Precip Profiles Included', horizontalalignment='center', verticalalignment='center', 
            transform=ax.transAxes, fontsize = 30, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 5})
    
    #plot the DAWN/dropsonde speed shear scatter plot
    ax = group_fig.add_subplot(1,2,2)
    
    plotting_metric = df1[plotting_metric_name]   #both dropsonde and DAWN data
    xlist = []
    for i in range(len(df1)):
        xstring = str(df1['Case'][i])
        xlist.append(xstring)
    
    #custom legend
    # legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 15, label='Isolated'),
    #                     Line2D([], [], color='red', linewidth = 0, marker = '$P$', markersize = 15, label='Isolated (In Precip)'),
    #                     Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 15, label='Organized'),
    #                     Line2D([], [], color='blue', linewidth = 0, marker = '$P$', markersize = 15, label='Organized (In Precip)'),
    #                     Line2D([], [], color='black', linewidth = 0, marker = 'o', markersize = 15, label='Scattered'),
    #                     Line2D([], [], color='black', linewidth = 0, marker = '$P$', markersize = 15, label='Scattered (In Precip)')]
    legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = 'o', markersize = 13, label='Isolated (Clear)'),
                       Line2D([], [], color='red', linewidth = 0, marker = '$C$', markersize = 13, label='Isolated (In Cloud)'),
                       Line2D([], [], color='red', linewidth = 0, marker = '$P$', markersize = 13, label='Isolated (In Precip)'),
                       Line2D([], [], color='blue', linewidth = 0, marker = 'o', markersize = 13, label='Organized (Clear)'),
                       Line2D([], [], color='blue', linewidth = 0, marker = '$C$', markersize = 13, label='Organized (In Cloud)'),
                       Line2D([], [], color='blue', linewidth = 0, marker = '$P$', markersize = 13, label='Organized (In Precip)')]
    
    
    #the lighter shades account for TS Cindy organized cases and, further, TS Cindy cases that were away from the main organized convection and instead near the cyclonic center
    for j in range(len(plotting_metric)):
        #if (df1['Convective Lifecycle'][j] != 'Weakening') and (df1['Environment Falling In'][j] == 'Clear Near' or df1['Environment Falling In'][j] == 'In Cloud' or df1['Environment Falling In'][j] == 'In Precip'):
        if df1['Environment Falling In'][j] == 'Clear Near' or df1['Environment Falling In'][j] == 'In Cloud' or df1['Environment Falling In'][j] == 'In Precip':
        #if (df1['Environment Falling In'][j] == 'Clear Near' or df1['Environment Falling In'][j] == 'In Cloud' or df1['Environment Falling In'][j] == 'In Precip') and (df1['Low-level Inflow Sonde'][j] != 'No' or df1['Mid-level Inflow Sonde'][j] != 'No'):
            if df1['Primary Convective Type'][j] == 'Isolated':
                color = 'red'
                if df1['Date'][j] in TC_days:
                    continue
            elif df1['Primary Convective Type'][j] == 'Organized':
                color = 'blue'
                if df1['Date'][j] in TC_days:
                    continue
            # elif df1['Primary Convective Type'][j] == 'Scattered':
            #     color = 'black'
            #     if df1['Date'][j] in TC_days:
            #         continue 
            else:
                continue
                  
            if df1['Environment Falling In'][j] == 'In Precip':
                if df1['Partially In Precip'][j] == 'Yes':
                    mark = '$*P$'
                    mark = '$P$'
                else:
                    mark = '$P$'
                outline = None
            elif df1['Environment Falling In'][j] == 'In Cloud':
                if df1['Falling Through Weak Stratiform (Onion-ish Profile (Typically On Outer Edge of Storm))'][j] == 'Yes':
                    mark = '$*C$'
                    mark = '$C$'
                else:
                    mark = '$C$'
                outline = None
            else:
                mark = 'o'
                outline = 'black'

            #if the metric is not NaN (need this filter, otherwise Python raises an error when showing/saving the figure)
            if not np.isnan(plotting_metric[j]):  #could also use pd.isna(plotting_metric[j]) 
                if df1['Low-level Inflow Sonde'][j] != 'No' or df1['Mid-level Inflow Sonde'][j] != 'No':               
                    ax.scatter(xlist[j], plotting_metric[j], c = color, s = 150, marker = mark, edgecolor = outline)
                else:
                    ax.scatter(xlist[j], plotting_metric[j], c = color, alpha = use_alpha, s = 150, marker = mark, edgecolor = outline)
    
    ax.grid(True)
    ax.set_title('Deep Layer Shear (Dropsonde & DAWN, 0.5km - 7.6km)', fontsize = 30, fontweight = 'bold')
    ax.set_xlabel('Case', fontsize = 30, fontweight = 'bold', labelpad = 15.0)
    ax.set_ylabel('[kts]', fontsize = 30, fontweight = 'bold', labelpad = 20.0)
    #ax.set_ylim([0,50])
    ax.tick_params(length = 15, width = 5, labelsize = 25)
    ax.legend(handles = legend_elements, fontsize = 22, ncol = 1, loc = 'upper left')
    ax.text(0.5, 0.5, 'IGNORE: JUST CPEX-CV CASES', horizontalalignment='center', verticalalignment='center', 
        transform=ax.transAxes, fontsize = 60, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 5})

box_plot_onlyIsoOrg_SSwDAWN('500m Bottom Cap Deep Layer Speed Shear [kts]')
plt.subplots_adjust(wspace = 0.20)
plt.savefig('/Users/ben/Desktop/SSwDAWN_box_scatter_allCPEX.png', bbox_inches = 'tight')
plt.close()


### For CPEX-CV AGU Paper (Figure 15)

In [ ]:
#adapted from CPEXCV_Metrics_Regression.ipynb
#This script plots a joint distribution scatter plot and regression line for two desired CPEX(-AW,-CV) dropsonde environmental metrics.
    #The points are color coded by any desired CPEX-CV dropsonde metric.
#COLORED BY CONVECTIVE TYPE, LABELED BY FIELD CAMPAIGN, SHADED BY INFLOW (Clear and In Cloud Sondes Only, Inflow Sondes Saturated)

#should only need to change the 9 variables in below
metric_x = 'SHARPpy Direct Method Upper Layer Speed Shear [kts]'  #metric to be placed on the x-axis; should be the independent metric
metric_x_label = 'Upper Layer Shear [kts]'
metric_x_title_label = 'Upper Layer Shear'
metric_y = 'Upper Layer RH [%]'  #metric to be placed on the y-axis; should be the dependent metric
metric_y_label = 'Upper Layer RH [%]'
metric_y_title_label = 'Upper Layer RH'
metric_colorby = 'Primary Convective Type'  #metric for which to color the scatter plot points by
metric_colorby_label = 'Convective Type'
vlims = [0,40]  #if metric_colorby is a numerical metric, give the desired numerical range for the colorbar
filter_out_precip = True  #recommend filtering out In Precip sondes if one of the desired metrics is moisture- or CAPE-related
save_name = '/Users/ben/Desktop/UpperShear_vs_UpperRH_option2.png' #filepath of the location where the plot should be saved

#options for metric_x, metric_y, and metric_colorby above:
# rh_layers = ['Deep Layer RH [%]', 'PBL RH [%]', 'Mid Layer RH [%]', 'Upper Layer RH [%]']
# speed_shear_layers = ['SHARPpy Direct Method Deep Layer Speed Shear [kts]', 'SHARPpy Direct Method PBL Speed Shear [kts]', 
#                       'SHARPpy Direct Method Mid Layer Speed Shear [kts]', 'SHARPpy Direct Method Upper Layer Speed Shear [kts]']
# cape_layers = ['Deep Layer MUCAPE [J/kg]', 'Deep Layer MLCAPE [J/kg]', 'Above FZL MUCAPE [J/kg]', 'Above FZL MLCAPE [J/kg]']


#set some baseline plot displays
matplotlib.rcParams['axes.labelsize'] = 30
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['axes.titlesize'] = 33
matplotlib.rcParams['xtick.labelsize'] = 25
matplotlib.rcParams['ytick.labelsize'] = 25
matplotlib.rcParams['legend.fontsize'] = 25
matplotlib.rcParams['legend.title_fontsize'] = 20
matplotlib.rcParams['legend.facecolor'] = 'w'
matplotlib.rcParams['font.family'] = 'arial'


TC_days = [20170619, 20170620, 20210826, 20210828, 20210901, 20210904, 20220923]  #including TDs as TCs (from both CPEX and CPEX-AW and CPEX-CV)
drop_filepath_cpexaw = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')  #filepath of the dropsonde metric CSV file
drop_filepath_cpexcv = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')  #filepath of the dropsonde metric CSV file

#read in the dropsonde metric CSV files and filter out dropsondes that don't have both desired metrics and/or are associated with a TD/TC
    #also filter out In Precip sondes, if desired (recommended if one of the metrics is moisture- or CAPE-related)

#NOTE: if a dropsonde's "metric_colorby" metric is blank, then the code will run fine BUT Python will skip plotting that dropsonde point
df_cpexaw = pd.read_csv(drop_filepath_cpexaw)
df_cpexcv = pd.read_csv(drop_filepath_cpexcv)
df = pd.concat([df_cpexaw, df_cpexcv], ignore_index = True)  #concatenates fields with same heading
df = df[~df['Date'].isin(TC_days)].copy()  #filter out cases associated with a TD/TC

if filter_out_precip:
    df = df[(~df[metric_x].isna()) & (~df[metric_y].isna()) & (df['Environment Falling In'] != 'In Precip')].copy()
    #df = df[(~df[metric_x].isna()) & (~df[metric_y].isna()) & (df['Environment Falling In'] != 'In Precip') & (df['Environment Falling In'] != 'In Cloud')].copy()
else:
    df = df[(~df[metric_x].isna()) & (~df[metric_y].isna())].copy()

fig,ax = plt.subplots(1,1,figsize=(16,16))

plotting_metric_x = df[metric_x]
plotting_metric_y = df[metric_y]
plotting_color_metric = df[metric_colorby]

# if 'SHARP' in metric_x:  #shorten the shear labels in the plot if needed
#     metric_x_label = metric_x[22:]
# else:
#     metric_x_label = metric_x[:]
    
# if 'SHARP' in metric_y:  #shorten the shear labels in the plot if needed
#     metric_y_label = metric_y[22:]
# else:
#     metric_y_label = metric_y[:]
    
# if 'SHARP' in metric_colorby:  #shorten the shear labels in the plot if needed
#     metric_colorby_label = metric_colorby[22:]
# else:
#     metric_colorby_label = metric_colorby[:]
    
for j in range(len(df)):

    if df['Primary Convective Type'].iloc[j] == 'Scattered':
        continue

    if str(df['Date'].iloc[j])[:4] == '2022':
        mark = '^'
        outline = 'black'
    else:
        mark = 's'
        outline = 'black'

    # if df['Environment Falling In'].iloc[j] == 'In Cloud':
    #     mark = '$C$'
    #     outline = None
    # elif df['Environment Falling In'].iloc[j] == 'In Precip':
    #     mark = '$P$'
    #     outline = None
    # else:
    #     mark = 'o'
    #     outline = 'black'

    #use_alpha = 1
    if (df['Low-level Inflow Sonde'].iloc[j] == 'Yes') or (df['Mid-level Inflow Sonde'].iloc[j] == 'Yes'):   #is it an inflow sonde
    #if str(df['Date'].iloc[j])[:4] == '2022':   #is it a CPEX-CV sonde
        use_alpha = 1
    else:
        use_alpha = 0.25   #a non-inflow sonde or a CPEX(-AW) sonde (same lower alpha value for the box plots and scatter plots)
    
    if metric_colorby == 'Primary Convective Type':
        if df['Primary Convective Type'].iloc[j] == 'Isolated':
            color = 'red'
        elif df['Primary Convective Type'].iloc[j] == 'Organized':
            color = 'blue'
        # elif df['Primary Convective Type'].iloc[j] == 'Scattered':
        #     color = 'black'
        else:
            continue

        #NOTE: if a dropsonde's "metric_colorby" metric is blank, then the code will run fine BUT Python will skip plotting that dropsonde point
        ax.scatter(plotting_metric_x.iloc[j], plotting_metric_y.iloc[j], c = color, s = 150, alpha = use_alpha, marker = mark, edgecolor = outline)
        
    else:
        if 'Direction' in metric_x or 'Direction' in metric_y: #use a cyclical colormap for directional shear
            #NOTE: if a dropsonde's "metric_colorby" metric is blank, then the code will run fine BUT Python will skip plotting that dropsonde point
            pm = ax.scatter(plotting_metric_x.iloc[j], plotting_metric_y.iloc[j], c = plotting_color_metric.iloc[j], cmap = cm.hsv, vmin = 0, vmax = 360, s = 150, alpha = use_alpha, marker = mark, edgecolor = outline)
        else:
            #NOTE: if a dropsonde's "metric_colorby" metric is blank, then the code will run fine BUT Python will skip plotting that dropsonde point
            pm = ax.scatter(plotting_metric_x.iloc[j], plotting_metric_y.iloc[j], c = plotting_color_metric.iloc[j], cmap = cm.Oranges, vmin = vlims[0], vmax = vlims[1], s = 150, alpha = use_alpha, marker = mark, edgecolor = outline)


# # Fit linear regression via least squares with numpy.polyfit
# # It returns a slope (a) and intercept (b)
# # deg = 1 means linear fit (i.e. polynomial of degree 1)
# #a, b = np.polyfit(plotting_metric_x, plotting_metric_y, deg = 1)

# #plotting_metric_x is the independent variable, plotting_metric_y is the independent variable
# slope, intercept, Rvalue, pvalue, stderr = scipy.stats.linregress(plotting_metric_x, plotting_metric_y)

# #the below two lines do not produce the same slope, intercept, and stderr
# #print (scipy.stats.linregress(plotting_metric_x, plotting_metric_y)) (e.g., slope = mb/%)
# #print (scipy.stats.linregress(plotting_metric_y, plotting_metric_x)) (e.g., slope = %/mb)

# #Create sequence of 1000 values of the plotting_metric_x to create the regression line
#     #subtracting/adding a fraction of the max value to make sure the regression line runs through the whole plot range
# xseq = np.linspace(plotting_metric_x.min() - 0.5 * plotting_metric_x.max(), plotting_metric_x.max() + 0.5 * plotting_metric_x.max(), num=1000)

# print ('Linear Regression Coefficient:', slope)
# print ('Pearson Correlation Coefficient:', Rvalue)
# #print ('Pearson Correlation coefficient:', scipy.stats.pearsonr(plotting_metric_x, plotting_metric_y)) #same as above
# #print ('Pearson Correlation coefficient:', scipy.stats.pearsonr(plotting_metric_y, plotting_metric_x)) #same as above

# # Plot regression line
# ax.plot(xseq, slope * xseq + intercept, color = 'k', linestyle = '--', linewidth = 3)         


ax.grid(True)
#ax.set_title(f'{metric_y_label} vs. {metric_x_label}\n(Clear and In Cloud Sondes Only, Inflow Sondes Saturated)')
ax.set_title(f'c) {metric_y_title_label} vs. {metric_x_title_label}')
ax.set_xlabel(metric_x_label, labelpad = 20.0)
ax.set_ylabel(metric_y_label, labelpad = 20.0)
ax.tick_params(length = 15, width = 5)
#ax.set_xlim([plotting_metric_x.min() - 0.05 * plotting_metric_x.min(), plotting_metric_x.max() + 0.05 * plotting_metric_x.max()])
#ax.set_ylim([plotting_metric_y.min() - 0.05 * plotting_metric_y.min(), plotting_metric_y.max() + 0.05 * plotting_metric_y.max()])

#ax.invert_yaxis()
#ax.set_yticks(np.arange(1000,899,-10))

if metric_colorby == 'Primary Convective Type':
    legend_elements = [Line2D([], [], color='red', linewidth = 0, marker = '^', markersize = 13, label='Isolated (CPEX-CV)'),
                       Line2D([], [], color='red', linewidth = 0, marker = 's', markersize = 13, label='Isolated (CPEX(-AW))'),
                       Line2D([], [], color='blue', linewidth = 0, marker = '^', markersize = 13, label='Organized (CPEX-CV)'),
                       Line2D([], [], color='blue', linewidth = 0, marker = 's', markersize = 13, label='Organized (CPEX(-AW))')]
    
    #legend = ax.legend(handles = legend_elements, title = r"TOC Type and Sonde Env't")
    legend = ax.legend(handles = legend_elements, loc = 'lower right')
    
else: 
    legend_elements = [Line2D([], [], color='k', linewidth = 3, linestyle = '--', label=f'$y = {slope:.1f}x {intercept:+.1f}$'),
                       Line2D([], [], color='k', linewidth = 3, linestyle = '', label=f'$R_{{pearson}} = {Rvalue:.3f}$'),
                       Line2D([], [], color='k', linewidth = 0, marker = '^', markersize = 13, label='CPEX-CV'),
                       Line2D([], [], color='k', linewidth = 0, marker = 's', markersize = 13, label='CPEX(-AW)')]
    
    #legend = ax.legend(handles = legend_elements, title = "Sonde Env't")
    legend = ax.legend(handles = legend_elements, loc = 'lower right')
    
    cbar0 = plt.colorbar(pm, ax=ax, pad = 0.02)
    cbar0.set_label(metric_colorby_label)

#plt.setp(legend.get_title(), fontweight = 'bold')
plt.savefig(save_name, bbox_inches = 'tight')

#plt.show()  #plt.show() must come after plt.savefig() in order for the image to save properly
#plt.clf()   #supposedly speeds things up? According to: https://www.youtube.com/watch?v=jGVIZbi9uMY
plt.close()
plt.clf()    #if placing this after plt.close(), may release memory related to the figure (https://stackoverflow.com/questions/741877/how-do-i-tell-matplotlib-that-i-am-done-with-a-plot)

##decrease file size of the image by 66% without noticeable image effects (if using Matplotlib)
##(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
im = Image.open(save_name)

try:
    im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
except:
    #use this for older version of PIL/Pillow if the above line doesn't work, 
    #though this line will have isolated, extremely minor image effects due to 
    #only using 256 colors instead of the 3-element RGB scale
    im2 = im.convert('P')

im2.save(save_name)
im.close()
im2.close()

print ('Done!')
